In [1]:
from pyalgotrade import strategy
from pyalgotrade.barfeed import quandlfeed
import pandas as pd

In [2]:
def conv_to_float(n):
    if type(n)==int or type(n)==float:
        return float(n)
    l = n.split('.')
    if(len(l)==1):
        return float(l[0])
    m = l[1]+"."+l[-2]
    return float(m)




class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        super(MyStrategy, self).__init__(feed)
        self.__instrument = instrument

    def onBars(self, bars):
        bar = bars[self.__instrument]
        self.info(bar.getClose())

In [3]:
#Load the bar feed from the CSV file
feed = quandlfeed.Feed()
feed.addBarsFromCSV("orcl", "WIKI-ORCL-2000-quandl.csv")
k = list(feed)[0][1].getBar('orcl')
print(k.getClose())

# Evaluate the strategy with the feed's bars.
myStrategy = MyStrategy(feed, "orcl")
#myStrategy.run()

FileNotFoundError: [Errno 2] No such file or directory: 'WIKI-ORCL-2000-quandl.csv'

In [4]:
#feed = quandlfeed.Feed()
df = pd.read_csv('ioc.csv')

df["Open"] = [conv_to_float(i) for i in list(df["Open"])]
df["High"] = [conv_to_float(i) for i in list(df["High"])]
df["Low"] = [conv_to_float(i) for i in list(df["Low"])]
df["Close"] = [conv_to_float(i) for i in list(df["Close"])]
df["Volume"] = [float(i) for i in list(df["Volume"])]

df.to_csv("ioc2.csv", sep=',', encoding='utf-8')

#feed.addBarsFromCSV("ioc", "ioc.csv")

FileNotFoundError: File b'ioc.csv' does not exist

In [ ]:
feed = quandlfeed.Feed()
feed.addBarsFromCSV("ioc", "ioc2.csv")

# Evaluate the strategy with the feed's bars.
myStrategy = MyStrategy(feed, "ioc")
myStrategy.run()



In [ ]:
from __future__ import print_function

from pyalgotrade import strategy
from pyalgotrade.barfeed import quandlfeed
from pyalgotrade.technical import ma


class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument, smaPeriod):
        super(MyStrategy, self).__init__(feed, 1000)
        self.__position = None
        self.__instrument = instrument
        # We'll use adjusted close values instead of regular close values.
        self.setUseAdjustedValues(True)
        self.__sma = ma.SMA(feed[instrument].getPriceDataSeries(), smaPeriod)

    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info("BUY at $%.2f" % (execInfo.getPrice()))

    def onEnterCanceled(self, position):
        self.__position = None

    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info("SELL at $%.2f" % (execInfo.getPrice()))
        self.__position = None

    def onExitCanceled(self, position):
        # If the exit was canceled, re-submit it.
        self.__position.exitMarket()

    def onBars(self, bars):
        # Wait for enough bars to be available to calculate a SMA.
        if self.__sma[-1] is None:
            return

        bar = bars[self.__instrument]
        # If a position was not opened, check if we should enter a long position.
        if self.__position is None:
            if bar.getPrice() > self.__sma[-1]:
                # Enter a buy market order for 10 shares. The order is good till canceled.
                self.__position = self.enterLong(self.__instrument, 10, True)
        # Check if we have to exit the position.
        elif bar.getPrice() < self.__sma[-1] and not self.__position.exitActive():
            self.__position.exitMarket()


def run_strategy(smaPeriod):
    # Load the bar feed from the CSV file
    feed = quandlfeed.Feed()
    feed.addBarsFromCSV("orcl", "WIKI-ORCL-2000-quandl.csv")

    # Evaluate the strategy with the feed.
    myStrategy = MyStrategy(feed, "orcl", smaPeriod)
    myStrategy.run()
    print("Final portfolio value: $%.2f" % myStrategy.getBroker().getEquity())

run_strategy(20)

In [ ]:
#My simple algo trading 
import random


l = [random.randint(50,60) for i in range(20)]
#l = [1,2,3,4,5,6,7,8,9,10]
#print(l)
l = [3133.0295, 3169.2458, 3246.7131, 3302.216, 3304.9988, 3315.764, 3198.5261, 3171.4138, 3230.554, 3180.962, 3112.0781, 3141.9407, 3052.759, 3052.1223, 3067.9058, 2979.1697, 2937.5447, 2943.958, 2980.046, 2967.7615, 2878.5671, 2874.5881, 2878.2104, 2863.2664, 2821.2383, 2759.144, 2780.447, 2775.9868, 2799.03, 2855.0051, 2877.21, 2918.134, 2954.8591, 3020.4968, 3048.1653, 2941.8987, 2953.5205, 2853.3418, 2866.2625, 2892.3281, 2950.6555, 2947.0928, 2979.295, 2973.091, 2971.2456, 3014.768, 3043.5735, 2967.6445, 2977.4958, 3068.6514, 3121.3628, 3140.2349, 3171.7517, 3174.867, 3145.9539, 3130.4673, 3145.8035, 3131.3528, 3126.278, 3133.9893, 3145.904, 3139.592, 3103.316, 3074.6882, 3012.4514, 3011.7673, 3077.841, 3038.159, 3076.1404, 3092.087, 3040.6887, 3057.276, 2975.362, 2967.199, 2959.143, 3019.2468, 3002.3032, 3035.8528, 3103.453, 3143.6384, 3131.111, 3055.93, 3060.634, 3081.9402, 3139.8445, 3197.3115, 3173.968, 3192.853, 3168.6338, 3280.473, 3267.5063, 3267.4622, 3356.613, 3376.149, 3390.5579, 3363.1428, 3353.3171, 3279.6047, 3238.9163, 3281.7402, 3222.8828, 3221.8643, 3287.1536, 3315.7068, 3264.7834, 3303.6667, 3345.3154, 3329.3313, 3363.3967, 3331.3276, 3332.5015, 3366.309, 3391.5732, 3364.2573, 3392.9863, 3458.0715, 3490.9937, 3497.8843, 3491.2253, 3493.6836, 3551.7188, 3658.4934, 3708.6746, 3673.6252, 3699.6262, 3682.1975, 3683.4214, 3612.1313, 3577.6067, 3569.666, 3612.288, 3644.6265, 3661.6663, 3718.8572, 3766.1372, 3774.1812, 3734.392, 3707.3457, 3664.6802, 3678.4973, 3745.8757, 3693.564, 3693.2305, 3622.2725, 3618.2568, 3667.8884, 3686.1133, 3689.3757, 3647.6272, 3618.4185, 3651.7278, 3643.0852, 3683.3972, 3691.16, 3707.8857, 3713.0906, 3796.2366, 3798.835, 3800.7483, 3833.5288, 3804.6973, 3827.6995, 3877.7756, 3914.3818, 3879.2917, 3899.5378, 3980.2073, 4027.9128, 4080.7827, 4037.173, 4097.7544, 4070.3206, 4058.0017, 4071.7488, 4021.3447, 4045.3823, 4100.315, 4122.3555, 4188.642, 4183.858, 4093.8743, 4088.7927, 4171.254, 4176.552, 4100.768, 4083.3584, 3977.6838, 3923.4578, 3804.6338, 3837.7588, 3777.9353, 3789.1367, 3888.4695, 3923.1711, 4023.0115, 4004.513, 3965.1057, 4040.2666, 4091.2002, 4122.843, 4098.705, 4063.3057, 3958.7356, 3950.6277, 4037.1326, 4046.8918, 3958.0967, 3926.6536, 3971.6418, 3960.8992, 3915.7922, 3927.5134, 3955.736, 4006.3396, 3995.931, 4132.252, 4209.2246, 4187.678, 4234.991, 4228.643, 4231.063, 4165.549, 4194.776, 4120.879, 4096.958, 4074.2107, 4104.2515, 4022.3823, 4078.6511, 4205.7456, 4321.859, 4334.5547, 4372.148, 4321.888, 4314.721, 4427.933, 4489.4043, 4501.692, 4515.478, 4521.1133, 4517.4995, 4531.0264, 4504.4297, 4527.1953, 4458.5557, 4480.4062, 4521.8945, 4611.6445, 4704.4043, 4602.1157, 4743.633, 4748.546, 4765.8896, 4744.51, 4808.0825, 4782.046, 4816.212, 4818.4634, 4871.402, 4908.508, 4873.211, 4823.2827, 4607.5864, 4465.908, 4487.771, 4536.219, 4494.2603, 4408.2627, 4320.8525, 4336.4233, 4315.6797, 4380.719, 4398.4023, 4398.121, 4435.6475, 4447.742, 4466.699, 4407.5186, 4385.838, 4371.589, 4443.195, 4420.1953, 4592.56, 4624.506, 4735.907, 4956.579, 5023.138, 5029.89, 5043.579, 5059.6274, 5081.778, 5024.1426, 5009.421, 5021.3145, 5056.027, 4900.1616, 4752.621, 4825.5103, 4637.887, 4606.0854, 4611.983, 4724.1875, 4638.971, 4656.6367, 4679.246, 4717.4575, 4740.1187, 4679.2007, 4750.3506, 4696.785, 4590.748, 4664.5967, 4640.6504, 4722.3965, 4714.351, 4567.059, 4549.5586, 4592.653, 4594.9727, 4665.109, 4509.0864, 4536.109, 4544.8394, 4537.1, 4672.7183, 4702.27, 4752.291, 4760.168, 4759.523, 4847.1, 4924.0293, 4904.2905, 4917.602, 4968.99, 4865.6514, 4820.281, 4794.055, 4851.749, 4925.234, 5016.495, 4871.577, 4883.862, 4930.2017, 4967.1753, 5063.039]
money = 0
have_stock = 1
for i in range(1,len(l)):
    if(l[i]>l[i-1] and money>=l[i-1]):
        print("buy at " + str(l[i-1]))
        buy = int(money/l[i-1])
        money = money - buy*l[i-1]
        have_stock = buy + have_stock
    elif(l[i]<l[i-1] and have_stock!=0):
        print("sell at " + str(l[i-1]))
        money = money + have_stock*l[i-1]
        have_stock = 0

print(money)
print(have_stock)

print("Net profit : "+ str(money+have_stock*l[-1] - l[0]))



In [ ]:
l = [3202.9998, 3265.0, 3310.0, 3340.9998, 3329.0, 3178.0, 3212.9998, 3259.0, 3202.9998, 3127.0002, 3159.0, 3081.0, 3051.9998, 3090.0, 2966.0, 2945.0, 2958.9998, 2988.0, 2985.0, 2873.0, 2877.0, 2879.0, 2860.9998, 2813.0, 2741.0, 2776.0, 2777.0, 2801.0, 2860.0, 2892.0, 2918.0, 2968.0, 3043.0, 3068.0, 2938.0, 2947.0, 2863.0, 2875.9998, 2912.0, 2948.9998, 2963.9998, 2990.0, 2968.0, 2973.0, 3028.0, 3036.9998, 2965.0, 2997.0, 3090.0, 3132.0002, 3161.0, 3192.0, 3183.0, 3134.9998, 3147.0, 3143.0, 3149.0, 3144.0, 3139.0, 3157.0, 3128.0, 3127.0002, 3070.0, 3008.0, 3021.0, 3104.0, 3038.0, 3072.0002, 3107.0002, 3041.0, 3058.0, 2970.0, 2962.0, 2957.0, 3038.0, 3021.0, 3049.0, 3122.0002, 3154.9998, 3143.0, 3045.0, 3073.0, 3092.0002, 3162.0, 3215.0002, 3191.0, 3207.9998, 3171.0, 3286.0, 3295.0, 3287.0, 3406.0002, 3402.0, 3416.0002, 3373.0, 3373.0, 3310.9998, 3266.0, 3313.0002, 3240.0, 3256.0, 3330.9998, 3347.0, 3296.0, 3310.0, 3363.0, 3353.0, 3385.0, 3359.0, 3372.0, 3383.0, 3431.0002, 3384.0, 3428.0, 3501.0, 3513.0, 3528.0, 3500.0, 3520.0, 3579.0, 3691.0, 3734.0, 3695.0002, 3737.9998, 3712.0, 3720.0002, 3639.0, 3621.0, 3599.9998, 3661.0, 3663.0, 3678.0, 3747.0, 3798.0002, 3807.0, 3757.0, 3737.9998, 3678.0, 3687.0, 3762.0, 3714.0, 3724.0, 3637.0, 3657.0, 3692.0, 3720.0002, 3692.0, 3656.0, 3651.0, 3669.0, 3648.0, 3697.9998, 3716.0, 3732.0, 3737.0, 3833.0, 3818.0002, 3820.0, 3850.0, 3829.0, 3824.0, 3903.0, 3921.0, 3887.0, 3906.0002, 4003.0, 4036.0, 4100.0, 4065.0, 4116.0, 4086.0, 4084.9998, 4088.0, 4036.9998, 4045.0, 4110.0, 4136.0, 4190.0, 4177.0, 4106.0, 4112.0, 4195.0, 4186.0, 4103.0, 4099.0, 3959.0002, 3931.0, 3807.0, 3842.0, 3813.0002, 3823.0, 3911.0002, 3941.0, 4036.9998, 4034.0, 3999.0002, 4086.0, 4125.0, 4155.0, 4122.0, 4093.0, 3983.0, 3995.0, 4075.0, 4063.0, 3960.0, 3943.0, 4016.9998, 3977.0, 3918.9998, 3948.0, 3984.0002, 4050.0, 4039.0, 4159.0, 4251.0, 4201.0, 4266.0, 4247.0, 4232.0, 4185.0, 4234.0, 4112.0, 4127.0, 4111.0, 4115.0, 4029.0, 4116.0, 4248.0, 4354.0, 4362.0, 4401.0, 4351.0, 4348.0, 4470.0, 4495.0, 4531.0, 4565.0, 4530.0, 4563.0, 4558.0, 4547.0, 4583.0, 4487.0, 4512.0, 4571.0, 4668.0, 4749.0, 4623.0, 4799.0, 4757.0, 4793.0, 4794.0, 4850.0, 4820.0, 4832.0, 4855.0, 4919.0, 4929.0, 4913.0, 4859.0, 4597.0, 4508.0, 4527.0, 4614.0, 4517.0, 4428.0, 4351.0, 4392.0, 4342.0, 4443.0, 4445.0, 4472.0, 4478.0, 4506.0, 4517.0, 4435.0, 4481.0, 4419.0, 4486.0, 4496.0, 4672.0, 4658.0, 4815.0, 5028.0, 5075.0, 5086.0, 5109.0, 5080.0, 5126.0, 5039.0, 5075.0, 5049.0, 5071.0, 4900.0, 4780.0, 4884.0, 4642.0, 4669.0, 4699.0, 4799.0, 4707.0, 4690.0, 4745.0, 4778.0, 4800.0, 4714.0, 4824.0, 4734.0, 4629.0, 4744.0, 4686.0, 4767.0, 4762.0, 4570.0, 4602.0, 4674.0, 4643.0, 4712.0, 4583.0, 4598.0, 4633.0, 4617.0, 4763.0, 4774.0, 4804.0, 4826.0, 4809.0, 4926.0, 4999.0, 4959.0, 5012.0, 5012.0, 4934.0, 4884.0, 4883.0, 4916.0, 5044.0, 5085.0, 4918.0, 4950.0, 4995.0, 5055.0, 5152.0, 5069.0]

money = 0
have_stock = 1
for i in range(1,len(l)):
    if(l[i]>l[i-1] and money>=l[i-1]):
        print("buy at " + str(l[i-1]))
        buy = int(money/l[i-1])
        money = money - buy*l[i-1]
        have_stock = buy + have_stock
    elif(l[i]<l[i-1] and have_stock!=0):
        print("sell at " + str(l[i-1]))
        money = money + have_stock*l[i-1]
        have_stock = 0

print(money)
print(have_stock)

print("Net profit : "+ str(money+have_stock*l[-1] - l[0]))





In [1]:

def algo_trade_sim(l):
    money = 0
    have_stock = 1
    buy_indices = []
    sell_indices = []
    for i in range(1,len(l)):
        if(l[i]>l[i-1] and money>=l[i-1]):
            print("buy at " + str(l[i-1]))
            buy = int(money/l[i-1])
            buy_indices.append((i-1,buy))
            money = money - buy*l[i-1]
            have_stock = buy + have_stock
        elif(l[i]<l[i-1] and have_stock!=0):
            print("sell at " + str(l[i-1]))
            sell_indices.append((i-1,have_stock))
            money = money + have_stock*l[i-1]
            have_stock = 0

    print(money)
    print(have_stock)

    print("Net profit : "+ str(money+have_stock*l[-1] - l[0]))

    net_profit = money+have_stock*l[-1] - l[0]
    return net_profit,buy_indices,sell_indices,have_stock


def accuracy(actual,predicted):
    
    #predicted
    print("As per predictions : ")
    expected_profit,buy_indices,sell_indices,stocks = algo_trade_sim(predicted)
    
    money = 0
    have_stock = 1
    for tup in buy_indices:
        money = money - actual[tup[0]]*tup[1]
    for tup in sell_indices:
        money = money + actual[tup[0]]*tup[1]
    
    print("Net profit of actual : "+ str(money+stocks*actual[-1] - actual[0]))
    

    
    


In [6]:
#Only for crude oil (final_oil.csv)

#pred1 = [3133.0295, 3169.2458, 3246.7131, 3302.216, 3304.9988, 3315.764, 3198.5261, 3171.4138, 3230.554, 3180.962, 3112.0781, 3141.9407, 3052.759, 3052.1223, 3067.9058, 2979.1697, 2937.5447, 2943.958, 2980.046, 2967.7615, 2878.5671, 2874.5881, 2878.2104, 2863.2664, 2821.2383, 2759.144, 2780.447, 2775.9868, 2799.03, 2855.0051, 2877.21, 2918.134, 2954.8591, 3020.4968, 3048.1653, 2941.8987, 2953.5205, 2853.3418, 2866.2625, 2892.3281, 2950.6555, 2947.0928, 2979.295, 2973.091, 2971.2456, 3014.768, 3043.5735, 2967.6445, 2977.4958, 3068.6514, 3121.3628, 3140.2349, 3171.7517, 3174.867, 3145.9539, 3130.4673, 3145.8035, 3131.3528, 3126.278, 3133.9893, 3145.904, 3139.592, 3103.316, 3074.6882, 3012.4514, 3011.7673, 3077.841, 3038.159, 3076.1404, 3092.087, 3040.6887, 3057.276, 2975.362, 2967.199, 2959.143, 3019.2468, 3002.3032, 3035.8528, 3103.453, 3143.6384, 3131.111, 3055.93, 3060.634, 3081.9402, 3139.8445, 3197.3115, 3173.968, 3192.853, 3168.6338, 3280.473, 3267.5063, 3267.4622, 3356.613, 3376.149, 3390.5579, 3363.1428, 3353.3171, 3279.6047, 3238.9163, 3281.7402, 3222.8828, 3221.8643, 3287.1536, 3315.7068, 3264.7834, 3303.6667, 3345.3154, 3329.3313, 3363.3967, 3331.3276, 3332.5015, 3366.309, 3391.5732, 3364.2573, 3392.9863, 3458.0715, 3490.9937, 3497.8843, 3491.2253, 3493.6836, 3551.7188, 3658.4934, 3708.6746, 3673.6252, 3699.6262, 3682.1975, 3683.4214, 3612.1313, 3577.6067, 3569.666, 3612.288, 3644.6265, 3661.6663, 3718.8572, 3766.1372, 3774.1812, 3734.392, 3707.3457, 3664.6802, 3678.4973, 3745.8757, 3693.564, 3693.2305, 3622.2725, 3618.2568, 3667.8884, 3686.1133, 3689.3757, 3647.6272, 3618.4185, 3651.7278, 3643.0852, 3683.3972, 3691.16, 3707.8857, 3713.0906, 3796.2366, 3798.835, 3800.7483, 3833.5288, 3804.6973, 3827.6995, 3877.7756, 3914.3818, 3879.2917, 3899.5378, 3980.2073, 4027.9128, 4080.7827, 4037.173, 4097.7544, 4070.3206, 4058.0017, 4071.7488, 4021.3447, 4045.3823, 4100.315, 4122.3555, 4188.642, 4183.858, 4093.8743, 4088.7927, 4171.254, 4176.552, 4100.768, 4083.3584, 3977.6838, 3923.4578, 3804.6338, 3837.7588, 3777.9353, 3789.1367, 3888.4695, 3923.1711, 4023.0115, 4004.513, 3965.1057, 4040.2666, 4091.2002, 4122.843, 4098.705, 4063.3057, 3958.7356, 3950.6277, 4037.1326, 4046.8918, 3958.0967, 3926.6536, 3971.6418, 3960.8992, 3915.7922, 3927.5134, 3955.736, 4006.3396, 3995.931, 4132.252, 4209.2246, 4187.678, 4234.991, 4228.643, 4231.063, 4165.549, 4194.776, 4120.879, 4096.958, 4074.2107, 4104.2515, 4022.3823, 4078.6511, 4205.7456, 4321.859, 4334.5547, 4372.148, 4321.888, 4314.721, 4427.933, 4489.4043, 4501.692, 4515.478, 4521.1133, 4517.4995, 4531.0264, 4504.4297, 4527.1953, 4458.5557, 4480.4062, 4521.8945, 4611.6445, 4704.4043, 4602.1157, 4743.633, 4748.546, 4765.8896, 4744.51, 4808.0825, 4782.046, 4816.212, 4818.4634, 4871.402, 4908.508, 4873.211, 4823.2827, 4607.5864, 4465.908, 4487.771, 4536.219, 4494.2603, 4408.2627, 4320.8525, 4336.4233, 4315.6797, 4380.719, 4398.4023, 4398.121, 4435.6475, 4447.742, 4466.699, 4407.5186, 4385.838, 4371.589, 4443.195, 4420.1953, 4592.56, 4624.506, 4735.907, 4956.579, 5023.138, 5029.89, 5043.579, 5059.6274, 5081.778, 5024.1426, 5009.421, 5021.3145, 5056.027, 4900.1616, 4752.621, 4825.5103, 4637.887, 4606.0854, 4611.983, 4724.1875, 4638.971, 4656.6367, 4679.246, 4717.4575, 4740.1187, 4679.2007, 4750.3506, 4696.785, 4590.748, 4664.5967, 4640.6504, 4722.3965, 4714.351, 4567.059, 4549.5586, 4592.653, 4594.9727, 4665.109, 4509.0864, 4536.109, 4544.8394, 4537.1, 4672.7183, 4702.27, 4752.291, 4760.168, 4759.523, 4847.1, 4924.0293, 4904.2905, 4917.602, 4968.99, 4865.6514, 4820.281, 4794.055, 4851.749, 4925.234, 5016.495, 4871.577, 4883.862, 4930.2017, 4967.1753, 5063.039]

pred = [3153.5925, 3197.0393, 3270.3062, 3315.041, 3326.92, 3311.2224, 3180.1357, 3169.5806, 3248.2014, 3203.2234, 3118.4827, 3130.7656, 3052.226, 3059.2056, 3058.6174, 2984.5945, 2945.6787, 2961.0667, 2995.9153, 2993.3916, 2902.8906, 2923.5847, 2941.864, 2937.352, 2888.1482, 2803.7776, 2838.6511, 2830.7197, 2861.4717, 2901.494, 2918.3723, 2966.7798, 3004.875, 3055.2942, 3091.8188, 2991.6316, 2991.9111, 2894.218, 2910.4146, 2935.09, 2988.7024, 2996.4402, 3028.5283, 3038.5613, 3026.1614, 3070.1768, 3093.0154, 3034.1672, 3030.5808, 3109.5273, 3156.6528, 3164.5388, 3207.6252, 3200.0544, 3161.0027, 3132.8103, 3158.16, 3152.7302, 3144.7915, 3151.7607, 3167.0833, 3146.584, 3130.6416, 3098.8628, 3047.864, 3044.9138, 3100.5405, 3084.3286, 3098.1257, 3120.8438, 3089.703, 3118.7996, 3034.6887, 3004.3025, 3010.6157, 3052.3108, 3056.9458, 3105.117, 3145.3716, 3185.7913, 3167.4424, 3121.6714, 3112.4084, 3133.2024, 3191.6187, 3247.4978, 3234.992, 3236.9058, 3233.4475, 3300.8506, 3304.069, 3309.6987, 3384.3389, 3394.1677, 3410.4749, 3391.6462, 3387.481, 3328.5435, 3292.9722, 3330.0547, 3285.58, 3289.193, 3330.3318, 3358.9912, 3315.228, 3350.5125, 3388.7957, 3382.505, 3429.5913, 3423.306, 3413.0376, 3428.9133, 3441.6145, 3436.373, 3478.1396, 3527.4292, 3555.9355, 3568.536, 3551.4695, 3560.4697, 3596.2742, 3677.3926, 3726.4976, 3704.6858, 3730.8044, 3735.0112, 3741.1978, 3682.3533, 3655.664, 3644.451, 3673.559, 3699.725, 3710.9314, 3759.9744, 3804.6516, 3816.8625, 3763.4812, 3744.4192, 3704.3337, 3713.6685, 3770.2268, 3735.58, 3733.0293, 3677.389, 3672.119, 3708.3328, 3729.886, 3733.3667, 3700.912, 3672.0957, 3716.7102, 3715.4373, 3743.7678, 3759.0786, 3766.8867, 3781.858, 3852.218, 3868.2715, 3869.628, 3893.4106, 3897.55, 3897.704, 3935.8992, 3975.8252, 3949.4722, 3971.146, 4023.1965, 4072.3772, 4115.659, 4078.5862, 4136.665, 4121.699, 4121.6865, 4127.6235, 4096.214, 4106.3477, 4139.3325, 4159.058, 4224.767, 4226.6016, 4136.941, 4115.5146, 4173.396, 4179.081, 4099.063, 4076.7656, 3975.3044, 3946.9688, 3861.706, 3884.2366, 3845.9333, 3836.5159, 3916.8655, 3967.016, 4068.8118, 4052.7185, 4026.5725, 4070.0984, 4115.5747, 4136.7676, 4118.6597, 4085.0654, 3998.54, 3997.1836, 4045.7258, 4075.3276, 3978.8564, 3962.3071, 4001.4292, 3999.7302, 3954.3213, 3948.8909, 4006.4463, 4068.2375, 4066.2585, 4170.8647, 4239.3867, 4232.319, 4254.2266, 4262.6235, 4246.2275, 4187.7095, 4233.5547, 4161.024, 4127.4844, 4098.206, 4153.156, 4092.7751, 4144.797, 4252.1025, 4332.269, 4366.024, 4415.7026, 4385.332, 4378.393, 4472.2183, 4530.931, 4539.7847, 4553.241, 4556.273, 4555.089, 4562.285, 4553.48, 4555.382, 4538.558, 4539.01, 4564.0645, 4657.7017, 4754.581, 4640.615, 4773.8604, 4795.774, 4814.858, 4801.291, 4839.8574, 4836.4697, 4863.649, 4872.3154, 4908.1294, 4914.412, 4881.1675, 4837.8076, 4641.676, 4532.325, 4529.948, 4562.269, 4519.011, 4450.734, 4369.038, 4382.652, 4362.6006, 4429.0024, 4453.602, 4451.6655, 4489.3774, 4482.7515, 4511.5654, 4454.6035, 4439.2705, 4434.4536, 4492.59, 4476.5986, 4627.339, 4701.608, 4784.7925, 4989.9805, 5058.0933, 5076.7363, 5082.0576, 5075.584, 5119.2627, 5068.8774, 5052.196, 5034.6313, 5063.8325, 4937.105, 4771.8916, 4842.228, 4702.2954, 4655.385, 4680.215, 4757.4717, 4695.4565, 4710.9956, 4728.6284, 4748.533, 4790.2534, 4730.173, 4775.0977, 4745.414, 4645.2427, 4703.472, 4695.198, 4743.144, 4741.069, 4615.0903, 4598.3555, 4635.0913, 4626.3135, 4681.926, 4571.84, 4596.2085, 4609.2534, 4607.8623, 4716.1025, 4757.7554, 4803.1885, 4809.997, 4814.432, 4886.631, 4962.6655, 4953.0337, 4968.6978, 4993.091, 4905.5522, 4853.741, 4833.49, 4885.2095, 4957.747, 5040.8696, 4930.822, 4916.9297, 4966.487, 5001.5166, 5096.7305]
actual = [3202.9998, 3265.0, 3310.0, 3340.9998, 3329.0, 3178.0, 3212.9998, 3259.0, 3202.9998, 3127.0002, 3159.0, 3081.0, 3051.9998, 3090.0, 2966.0, 2945.0, 2958.9998, 2988.0, 2985.0, 2873.0, 2877.0, 2879.0, 2860.9998, 2813.0, 2741.0, 2776.0, 2777.0, 2801.0, 2860.0, 2892.0, 2918.0, 2968.0, 3043.0, 3068.0, 2938.0, 2947.0, 2863.0, 2875.9998, 2912.0, 2948.9998, 2963.9998, 2990.0, 2968.0, 2973.0, 3028.0, 3036.9998, 2965.0, 2997.0, 3090.0, 3132.0002, 3161.0, 3192.0, 3183.0, 3134.9998, 3147.0, 3143.0, 3149.0, 3144.0, 3139.0, 3157.0, 3128.0, 3127.0002, 3070.0, 3008.0, 3021.0, 3104.0, 3038.0, 3072.0002, 3107.0002, 3041.0, 3058.0, 2970.0, 2962.0, 2957.0, 3038.0, 3021.0, 3049.0, 3122.0002, 3154.9998, 3143.0, 3045.0, 3073.0, 3092.0002, 3162.0, 3215.0002, 3191.0, 3207.9998, 3171.0, 3286.0, 3295.0, 3287.0, 3406.0002, 3402.0, 3416.0002, 3373.0, 3373.0, 3310.9998, 3266.0, 3313.0002, 3240.0, 3256.0, 3330.9998, 3347.0, 3296.0, 3310.0, 3363.0, 3353.0, 3385.0, 3359.0, 3372.0, 3383.0, 3431.0002, 3384.0, 3428.0, 3501.0, 3513.0, 3528.0, 3500.0, 3520.0, 3579.0, 3691.0, 3734.0, 3695.0002, 3737.9998, 3712.0, 3720.0002, 3639.0, 3621.0, 3599.9998, 3661.0, 3663.0, 3678.0, 3747.0, 3798.0002, 3807.0, 3757.0, 3737.9998, 3678.0, 3687.0, 3762.0, 3714.0, 3724.0, 3637.0, 3657.0, 3692.0, 3720.0002, 3692.0, 3656.0, 3651.0, 3669.0, 3648.0, 3697.9998, 3716.0, 3732.0, 3737.0, 3833.0, 3818.0002, 3820.0, 3850.0, 3829.0, 3824.0, 3903.0, 3921.0, 3887.0, 3906.0002, 4003.0, 4036.0, 4100.0, 4065.0, 4116.0, 4086.0, 4084.9998, 4088.0, 4036.9998, 4045.0, 4110.0, 4136.0, 4190.0, 4177.0, 4106.0, 4112.0, 4195.0, 4186.0, 4103.0, 4099.0, 3959.0002, 3931.0, 3807.0, 3842.0, 3813.0002, 3823.0, 3911.0002, 3941.0, 4036.9998, 4034.0, 3999.0002, 4086.0, 4125.0, 4155.0, 4122.0, 4093.0, 3983.0, 3995.0, 4075.0, 4063.0, 3960.0, 3943.0, 4016.9998, 3977.0, 3918.9998, 3948.0, 3984.0002, 4050.0, 4039.0, 4159.0, 4251.0, 4201.0, 4266.0, 4247.0, 4232.0, 4185.0, 4234.0, 4112.0, 4127.0, 4111.0, 4115.0, 4029.0, 4116.0, 4248.0, 4354.0, 4362.0, 4401.0, 4351.0, 4348.0, 4470.0, 4495.0, 4531.0, 4565.0, 4530.0, 4563.0, 4558.0, 4547.0, 4583.0, 4487.0, 4512.0, 4571.0, 4668.0, 4749.0, 4623.0, 4799.0, 4757.0, 4793.0, 4794.0, 4850.0, 4820.0, 4832.0, 4855.0, 4919.0, 4929.0, 4913.0, 4859.0, 4597.0, 4508.0, 4527.0, 4614.0, 4517.0, 4428.0, 4351.0, 4392.0, 4342.0, 4443.0, 4445.0, 4472.0, 4478.0, 4506.0, 4517.0, 4435.0, 4481.0, 4419.0, 4486.0, 4496.0, 4672.0, 4658.0, 4815.0, 5028.0, 5075.0, 5086.0, 5109.0, 5080.0, 5126.0, 5039.0, 5075.0, 5049.0, 5071.0, 4900.0, 4780.0, 4884.0, 4642.0, 4669.0, 4699.0, 4799.0, 4707.0, 4690.0, 4745.0, 4778.0, 4800.0, 4714.0, 4824.0, 4734.0, 4629.0, 4744.0, 4686.0, 4767.0, 4762.0, 4570.0, 4602.0, 4674.0, 4643.0, 4712.0, 4583.0, 4598.0, 4633.0, 4617.0, 4763.0, 4774.0, 4804.0, 4826.0, 4809.0, 4926.0, 4999.0, 4959.0, 5012.0, 5012.0, 4934.0, 4884.0, 4883.0, 4916.0, 5044.0, 5085.0, 4918.0, 4950.0, 4995.0, 5055.0, 5152.0, 5069.0]
accuracy(actual,pred)

As per predictions : 
sell at 3326.92
buy at 3169.5806
sell at 3248.2014
buy at 3118.4827
sell at 3130.7656
buy at 3052.226
sell at 3059.2056
buy at 2945.6787
sell at 2995.9153
buy at 2902.8906
sell at 2941.864
buy at 2803.7776
sell at 2838.6511
buy at 2830.7197
sell at 3091.8188
buy at 2991.6316
sell at 2991.9111
buy at 2894.218
sell at 3038.5613
buy at 3026.1614
sell at 3093.0154
buy at 3030.5808
sell at 3207.6252
buy at 3132.8103
sell at 3158.16
buy at 3144.7915
sell at 3167.0833
buy at 3044.9138
sell at 3100.5405
buy at 3084.3286
sell at 3120.8438
buy at 3089.703
sell at 3118.7996
buy at 3004.3025
sell at 3185.7913
buy at 3112.4084
sell at 3247.4978
buy at 3234.992
sell at 3236.9058
buy at 3233.4475
sell at 3410.4749
buy at 3292.9722
sell at 3330.0547
buy at 3285.58
sell at 3358.9912
buy at 3315.228
sell at 3388.7957
buy at 3382.505
sell at 3429.5913
buy at 3413.0376
sell at 3441.6145
buy at 3436.373
sell at 3568.536
buy at 3551.4695
sell at 3726.4976
buy at 3704.6858
sell at 3741.

In [3]:
#IOC Using OHLC data

actual = [75.475, 75.275, 75.95, 76.8375, 75.9375, 75.75, 75.3375, 75.7625, 75.425, 72.675, 72.2, 71.8875, 71.7375, 72.5625, 72.1, 70.7125, 72.45, 71.1875, 70.35, 70.4, 68.325, 69.8625, 72.5125, 74.8, 75.0, 76.3, 77.675, 79.3875, 77.45, 77.175, 77.625, 78.45, 77.9875, 76.8, 76.7625, 77.025, 75.95, 74.7125, 73.175, 76.6, 73.4875, 76.65, 78.0875, 78.55, 79.475, 78.6125, 78.7, 79.1375, 80.05, 79.2625, 78.1, 78.8, 81.55, 83.2, 83.1125, 82.15, 84.2125, 81.825, 78.4125, 77.6125, 79.65, 80.8875, 78.4625, 81.4875, 83.35, 87.325, 78.975, 74.2375, 74.3875, 72.025, 71.1875, 71.5375, 71.5875, 70.3375, 70.3125, 70.2875, 67.8875, 68.0, 67.475, 67.2125, 65.25, 64.8, 64.35, 64.925, 64.9625, 65.45, 65.2875, 65.325, 65.25, 65.15, 65.175, 65.9, 66.2125, 66.0625, 66.0375, 65.675, 65.6, 65.4625, 65.9375, 65.45, 65.0875, 64.8625, 64.7625, 64.725, 64.525, 63.962505, 64.5, 64.625, 65.375, 65.525, 66.375, 65.875, 65.5375, 65.15, 65.625, 65.025, 65.125, 64.95, 65.075, 65.05, 65.0375, 64.9375, 64.85, 64.475, 64.1875, 63.624996, 63.925003, 64.325, 64.7125, 64.6125, 64.65, 65.0125, 64.7875, 65.075, 63.525, 63.2375, 62.625, 62.2, 63.65, 63.5375, 64.05, 63.174995, 62.0375, 62.0875, 61.95, 62.2, 63.65, 63.262505, 63.3875, 63.0625, 62.075, 61.862503, 61.375, 60.9875, 60.6375, 61.562496, 61.012497, 61.7875, 62.112503, 62.4, 63.212505, 64.0125, 62.9875, 63.012505, 62.625, 63.725002, 63.624996, 62.5875, 62.9, 64.025, 63.774998, 64.425, 65.65, 65.975, 66.875, 67.475, 66.35, 65.1625, 65.2875, 66.85, 67.45, 67.6125, 67.7125, 68.5375, 67.3125, 67.925, 67.75, 67.8125, 68.1125, 66.8375, 66.575, 66.4125, 67.4625, 66.3625, 66.4875, 63.962505, 63.9375, 63.275, 64.6875, 65.05, 64.975, 65.4625, 65.4375, 63.5875, 62.525, 60.325, 60.025, 60.812496, 61.412502, 61.8, 61.812504, 62.112503, 62.4875, 63.4875, 64.3125, 64.9625, 64.0375, 63.9, 63.4375, 65.125, 66.4375, 67.55, 68.3, 67.8875, 67.5125, 67.775, 67.1375, 66.8625, 66.5625, 65.9, 65.55, 66.45, 66.7375, 66.275, 66.275, 64.35, 64.7, 64.8125, 66.3875, 64.8, 63.9, 65.0125, 65.95, 64.8875, 65.8875, 65.4125, 65.65, 64.05, 63.1375, 62.9, 63.2, 62.762505, 62.762505, 64.125, 65.7125, 66.6, 65.1, 65.675, 63.5, 64.9375, 65.3875, 65.7125, 66.925, 66.625, 67.675, 67.525, 67.3375, 68.3625, 68.1875, 68.925, 69.0125, 68.875, 68.975, 68.225, 70.025, 69.3, 69.425, 69.4125, 70.95, 68.9625, 68.375, 69.125, 70.8875, 69.2625, 68.2, 67.7, 67.3625, 69.1625, 68.4875, 68.9375, 68.925, 67.9375, 68.1375, 67.8, 68.7125, 68.0, 68.525, 67.875, 72.2, 70.25, 72.3125, 70.85, 69.5125, 69.5375, 68.6625, 68.1, 67.95, 67.35, 66.825, 66.7125, 66.175, 64.675, 63.8, 63.9375, 63.2875, 63.15, 62.6875, 63.4375, 63.1375, 63.262505, 65.2125, 68.2875, 68.4875, 67.375, 66.3375, 66.3625, 66.7375, 67.15, 67.4125, 69.2125, 67.275]
pred = [74.32254, 74.97709, 75.34151, 75.86418, 75.96869, 75.407845, 75.63673, 75.14667, 75.63532, 74.25932, 71.93045, 71.905075, 71.46233, 71.853065, 72.24583, 71.31289, 71.33337, 71.36527, 70.317535, 70.10794, 69.07963, 69.37584, 72.1584, 72.82218, 74.74005, 75.265526, 76.568275, 77.65283, 78.0824, 76.91165, 77.097755, 77.52859, 77.840744, 77.257515, 76.912926, 76.606476, 75.81355, 75.37247, 73.73625, 74.66191, 74.98903, 74.69527, 76.97835, 78.06348, 78.642006, 78.917435, 78.005905, 78.328804, 80.3806, 79.3866, 78.41895, 78.04869, 79.61209, 81.88905, 82.63755, 81.9875, 82.8225, 82.50886, 79.69146, 78.106766, 78.329575, 79.72836, 79.14442, 79.70119, 81.42256, 84.0047, 83.11651, 75.85721, 74.628395, 72.79674, 71.01855, 71.12906, 71.81478, 71.30107, 70.468216, 69.316605, 69.21443, 67.90919, 67.26342, 67.21099, 66.26445, 64.75726, 64.56343, 64.587364, 64.65188, 65.01477, 65.215256, 65.26136, 65.33417, 65.187744, 65.25009, 65.33576, 65.87803, 66.17511, 66.142166, 65.7735, 65.53539, 65.48701, 65.5643, 65.823944, 66.38826, 65.36054, 64.7945, 64.65935, 64.511475, 64.25541, 63.871326, 64.71956, 64.80201, 65.53734, 65.959885, 65.88333, 65.48867, 65.49644, 65.44494, 65.32567, 65.21058, 64.462105, 64.97241, 65.1037, 65.06212, 64.74925, 64.78067, 64.42402, 64.357605, 64.100204, 64.77322, 64.0575, 64.59279, 64.697495, 64.41907, 64.79363, 65.11107, 64.80995, 64.67492, 63.557682, 63.052525, 62.594334, 62.961956, 63.637424, 63.686157, 63.669983, 62.518337, 60.99246, 62.125683, 62.443172, 64.33661, 63.7067, 63.018417, 63.523506, 62.72598, 62.071686, 61.795635, 61.037663, 61.00288, 61.192715, 61.397434, 61.59468, 61.954773, 62.34446, 63.005306, 63.43718, 63.431705, 63.145103, 63.082813, 63.062305, 63.695084, 63.065792, 62.224346, 63.2702, 64.116554, 63.677654, 65.031425, 65.55773, 66.313354, 67.02813, 66.69449, 65.59234, 65.37078, 65.8975, 66.866234, 67.6037, 67.550545, 68.05572, 68.050575, 67.524284, 67.76484, 67.64249, 67.97428, 67.38332, 66.19774, 66.550575, 67.042755, 66.91238, 66.208496, 65.1438, 63.88201, 63.511055, 63.737953, 64.86215, 64.858864, 65.32171, 65.85874, 64.843544, 63.20315, 61.406155, 60.5846, 60.826553, 61.152077, 61.744385, 61.924225, 62.137245, 62.27335, 63.065327, 63.982475, 64.59963, 64.34502, 63.925793, 63.27621, 64.01004, 65.85748, 62.902996, 69.055016, 68.225845, 67.895065, 67.90491, 67.24557, 67.21949, 66.72748, 66.12642, 65.748245, 65.94513, 66.52288, 66.2238, 66.063286, 65.31798, 64.19762, 64.71502, 65.55609, 65.81063, 64.33372, 64.461365, 65.65776, 65.2381, 65.385216, 65.74019, 65.358826, 65.0591, 63.674675, 63.09218, 63.19758, 63.09341, 62.83868, 63.62486, 64.57488, 65.84727, 65.888245, 65.30248, 64.64816, 63.916718, 65.25625, 65.528694, 66.268875, 67.13385, 67.11195, 67.36152, 67.48322, 67.12445, 68.36784, 68.32333, 68.94266, 68.91836, 69.03941, 68.83499, 68.50348, 70.0199, 69.61951, 69.56535, 69.45193, 69.96164, 68.438805, 68.709366, 70.19254, 69.6302, 68.72098, 67.72645, 67.96634, 67.99587, 68.84028, 69.096085, 68.99979, 69.07341, 67.68837, 67.9501, 68.23343, 68.37943, 68.24455, 68.01507, 69.61297, 71.07821, 71.082146, 71.44575, 70.155975, 69.45795, 68.94306, 68.439476, 68.036385, 67.686966, 66.55497, 66.90351, 66.48019, 65.17526, 64.36793, 63.268005, 63.57093, 63.155724, 62.865868, 62.660446, 63.405983, 63.023613, 63.853058, 66.19662, 68.15182, 67.66863, 66.78056, 66.023224, 66.14314, 66.92137, 67.34357, 68.051216, 67.815094]


accuracy(actual,pred)


As per predictions : 
sell at 75.96869
buy at 75.407845
sell at 75.63673
buy at 75.14667
sell at 75.63532
buy at 71.46233
sell at 72.24583
buy at 71.31289
sell at 71.36527
buy at 69.07963
sell at 78.0824
buy at 76.91165
sell at 77.840744
buy at 73.73625
sell at 74.98903
buy at 74.69527
sell at 78.917435
buy at 78.005905
sell at 80.3806
buy at 78.04869
sell at 82.63755
buy at 81.9875
sell at 82.8225
buy at 78.106766
sell at 79.72836
buy at 79.14442
sell at 84.0047
buy at 71.01855
sell at 71.81478
buy at 64.56343
sell at 65.33417
buy at 65.187744
sell at 66.17511
buy at 65.48701
sell at 66.38826
buy at 63.871326
sell at 65.959885
buy at 65.48867
sell at 65.49644
buy at 64.462105
sell at 65.1037
buy at 64.74925
sell at 64.78067
buy at 64.100204
sell at 64.77322
buy at 64.0575
sell at 64.697495
buy at 64.41907
sell at 65.11107
buy at 62.594334
sell at 63.686157
buy at 60.99246
sell at 64.33661
buy at 63.018417
sell at 63.523506
buy at 61.00288
sell at 63.43718
buy at 63.062305
sell at 63.6

In [4]:
#oil

#With extra parameters

actual = [3202.9998, 3265.0, 3310.0, 3340.9998, 3329.0, 3178.0, 3212.9998, 3259.0, 3202.9998, 3127.0002, 3159.0, 3081.0, 3051.9998, 3090.0, 2966.0, 2945.0, 2958.9998, 2988.0, 2985.0, 2873.0, 2877.0, 2879.0, 2860.9998, 2813.0, 2741.0, 2776.0, 2777.0, 2801.0, 2860.0, 2892.0, 2918.0, 2968.0, 3043.0, 3068.0, 2938.0, 2947.0, 2863.0, 2875.9998, 2912.0, 2948.9998, 2963.9998, 2990.0, 2968.0, 2973.0, 3028.0, 3036.9998, 2965.0, 2997.0, 3090.0, 3132.0002, 3161.0, 3192.0, 3183.0, 3134.9998, 3147.0, 3143.0, 3149.0, 3144.0, 3139.0, 3157.0, 3128.0, 3127.0002, 3070.0, 3008.0, 3021.0, 3104.0, 3038.0, 3072.0002, 3107.0002, 3041.0, 3058.0, 2970.0, 2962.0, 2957.0, 3038.0, 3021.0, 3049.0, 3122.0002, 3154.9998, 3143.0, 3045.0, 3073.0, 3092.0002, 3162.0, 3215.0002, 3191.0, 3207.9998, 3171.0, 3286.0, 3295.0, 3287.0, 3406.0002, 3402.0, 3416.0002, 3373.0, 3373.0, 3310.9998, 3266.0, 3313.0002, 3240.0, 3256.0, 3330.9998, 3347.0, 3296.0, 3310.0, 3363.0, 3353.0, 3385.0, 3359.0, 3372.0, 3383.0, 3431.0002, 3384.0, 3428.0, 3501.0, 3513.0, 3528.0, 3500.0, 3520.0, 3579.0, 3691.0, 3734.0, 3695.0002, 3737.9998, 3712.0, 3720.0002, 3639.0, 3621.0, 3599.9998, 3661.0, 3663.0, 3678.0, 3747.0, 3798.0002, 3807.0, 3757.0, 3737.9998, 3678.0, 3687.0, 3762.0, 3714.0, 3724.0, 3637.0, 3657.0, 3692.0, 3720.0002, 3692.0, 3656.0, 3651.0, 3669.0, 3648.0, 3697.9998, 3716.0, 3732.0, 3737.0, 3833.0, 3818.0002, 3820.0, 3850.0, 3829.0, 3824.0, 3903.0, 3921.0, 3887.0, 3906.0002, 4003.0, 4036.0, 4100.0, 4065.0, 4116.0, 4086.0, 4084.9998, 4088.0, 4036.9998, 4045.0, 4110.0, 4136.0, 4190.0, 4177.0, 4106.0, 4112.0, 4195.0, 4186.0, 4103.0, 4099.0, 3959.0002, 3931.0, 3807.0, 3842.0, 3813.0002, 3823.0, 3911.0002, 3941.0, 4036.9998, 4034.0, 3999.0002, 4086.0, 4125.0, 4155.0, 4122.0, 4093.0, 3983.0, 3995.0, 4075.0, 4063.0, 3960.0, 3943.0, 4016.9998, 3977.0, 3918.9998, 3948.0, 3984.0002, 4050.0, 4039.0, 4159.0, 4251.0, 4201.0, 4266.0, 4247.0, 4232.0, 4185.0, 4234.0, 4112.0, 4127.0, 4111.0, 4115.0, 4029.0, 4116.0, 4248.0, 4354.0, 4362.0, 4401.0, 4351.0, 4348.0, 4470.0, 4495.0, 4531.0, 4565.0, 4530.0, 4563.0, 4558.0, 4547.0, 4583.0, 4487.0, 4512.0, 4571.0, 4668.0, 4749.0, 4623.0, 4799.0, 4757.0, 4793.0, 4794.0, 4850.0, 4820.0, 4832.0, 4855.0, 4919.0, 4929.0, 4913.0, 4859.0, 4597.0, 4508.0, 4527.0, 4614.0, 4517.0, 4428.0, 4351.0, 4392.0, 4342.0, 4443.0, 4445.0, 4472.0, 4478.0, 4506.0, 4517.0, 4435.0, 4481.0, 4419.0, 4486.0, 4496.0, 4672.0, 4658.0, 4815.0, 5028.0, 5075.0, 5086.0, 5109.0, 5080.0, 5126.0, 5039.0, 5075.0, 5049.0, 5071.0, 4900.0, 4780.0, 4884.0, 4642.0, 4669.0, 4699.0, 4799.0, 4707.0, 4690.0, 4745.0, 4778.0, 4800.0, 4714.0, 4824.0, 4734.0, 4629.0, 4744.0, 4686.0, 4767.0, 4762.0, 4570.0, 4602.0, 4674.0, 4643.0, 4712.0, 4583.0, 4598.0, 4633.0, 4617.0, 4763.0, 4774.0, 4804.0, 4826.0, 4809.0, 4926.0, 4999.0, 4959.0, 5012.0, 5012.0, 4934.0, 4884.0, 4883.0, 4916.0, 5044.0, 5085.0, 4918.0, 4950.0, 4995.0, 5055.0, 5152.0, 5069.0]

#Test RMSE: 130.667
#pred_extra = [3204.489, 3252.451, 3332.209, 3382.6648, 3394.4558, 3383.1438, 3252.129, 3228.753, 3310.6511, 3262.0933, 3175.8079, 3188.836, 3106.6812, 3110.476, 3110.0889, 3034.507, 2987.048, 2999.49, 3038.082, 3033.8591, 2940.099, 2952.429, 2967.355, 2960.683, 2912.4565, 2828.528, 2857.48, 2849.9229, 2882.8044, 2927.077, 2947.2666, 2997.2864, 3038.4128, 3097.0637, 3134.9248, 3030.8743, 3027.9119, 2926.8389, 2940.5786, 2967.1494, 3024.4336, 3031.045, 3063.2214, 3071.118, 3059.363, 3106.6187, 3133.0105, 3068.256, 3065.4055, 3152.3066, 3204.4006, 3215.5332, 3259.549, 3254.9644, 3218.3596, 3184.72, 3209.808, 3202.7373, 3194.913, 3201.324, 3217.3313, 3201.1528, 3178.617, 3146.812, 3089.7883, 3087.1304, 3146.9492, 3128.0725, 3144.7463, 3167.367, 3131.793, 3159.0654, 3070.9956, 3041.5598, 3042.4863, 3091.879, 3092.5925, 3141.0708, 3189.6448, 3232.8955, 3216.1208, 3163.269, 3153.745, 3174.5378, 3236.6528, 3298.011, 3282.804, 3289.8513, 3281.5208, 3363.6982, 3364.7422, 3370.5054, 3454.2163, 3469.096, 3484.9438, 3462.939, 3456.866, 3391.385, 3349.2139, 3389.7512, 3342.4277, 3343.577, 3392.1477, 3423.8733, 3375.133, 3410.7344, 3453.523, 3445.7932, 3494.2578, 3488.2383, 3475.4636, 3493.6194, 3509.4348, 3499.102, 3540.8157, 3597.6355, 3628.6145, 3641.4326, 3624.4702, 3632.8508, 3677.057, 3768.3577, 3821.9592, 3794.5884, 3824.1814, 3825.5251, 3831.2043, 3766.967, 3733.6968, 3724.1165, 3756.6194, 3788.9834, 3798.62, 3851.5283, 3903.4214, 3915.173, 3862.6848, 3838.4062, 3791.049, 3800.1667, 3865.4216, 3827.4036, 3824.6045, 3762.3037, 3756.7468, 3795.7456, 3818.066, 3820.9702, 3785.688, 3754.4666, 3798.6714, 3798.3306, 3830.5996, 3845.3838, 3854.492, 3870.667, 3946.3855, 3960.7776, 3965.1313, 3990.1826, 3997.993, 3991.4526, 4032.9604, 4073.6418, 4046.0593, 4068.186, 4129.4395, 4180.869, 4229.274, 4191.478, 4255.984, 4235.111, 4233.1504, 4240.567, 4204.3115, 4215.9414, 4254.919, 4274.137, 4344.2764, 4349.121, 4251.1367, 4229.748, 4298.1865, 4306.356, 4221.5464, 4196.75, 4089.8486, 4052.5059, 3957.496, 3979.0386, 3940.2417, 3928.6714, 4018.4033, 4073.4556, 4183.9707, 4160.98, 4132.9795, 4184.5615, 4239.198, 4261.7373, 4240.161, 4201.666, 4106.888, 4108.7812, 4164.047, 4190.179, 4088.0776, 4070.0403, 4112.935, 4111.4863, 4057.1504, 4053.5364, 4111.906, 4176.7656, 4176.54, 4289.2847, 4364.3184, 4354.823, 4383.2427, 4389.664, 4374.9434, 4310.8755, 4363.402, 4283.0415, 4246.951, 4214.9067, 4267.6196, 4198.1553, 4256.8223, 4374.8535, 4464.854, 4500.3096, 4549.633, 4517.6924, 4511.0796, 4614.674, 4678.784, 4690.83, 4707.153, 4706.1997, 4704.3423, 4715.8833, 4704.371, 4705.343, 4686.153, 4682.258, 4714.616, 4812.17, 4919.3364, 4794.993, 4943.6377, 4963.841, 4985.342, 4972.461, 5011.618, 5006.3867, 5035.109, 5049.2544, 5094.0586, 5103.4854, 5067.7266, 5022.2075, 4807.161, 4684.56, 4683.417, 4723.0337, 4675.842, 4597.816, 4509.5054, 4525.268, 4501.7793, 4576.5376, 4601.828, 4599.855, 4638.701, 4635.81, 4665.7563, 4610.075, 4589.3945, 4584.884, 4638.2974, 4625.463, 4785.0854, 4855.1226, 4951.8345, 5175.2812, 5246.4263, 5266.794, 5270.433, 5268.768, 5318.918, 5254.689, 5240.328, 5230.109, 5260.3135, 5120.619, 4946.0977, 5027.7275, 4871.409, 4818.3335, 4842.85, 4938.506, 4860.614, 4875.5103, 4896.7886, 4921.379, 4964.6, 4901.01, 4954.456, 4917.5376, 4806.479, 4874.819, 4858.938, 4919.812, 4916.0913, 4777.707, 4759.863, 4804.4243, 4797.248, 4856.156, 4737.837, 4760.369, 4773.2197, 4768.1226, 4893.192, 4934.0874, 4982.9536, 4992.954, 4992.9067, 5073.581, 5158.7124, 5147.2993, 5166.1533, 5186.969, 5094.7466, 5045.7207, 5024.987, 5080.402, 5158.6104, 5241.994, 5130.0215, 5110.0527, 5166.61, 5202.1196, 5304.525]

#Test RMSE: 97.142
#pred_extra = [3183.7554, 3234.3784, 3318.6306, 3368.1472, 3379.937, 3370.4148, 3223.9133, 3210.9788, 3301.6655, 3253.226, 3165.1233, 3173.8508, 3090.4802, 3100.7688, 3100.116, 3028.0818, 2982.9495, 2998.8887, 3035.3225, 3031.485, 2929.1514, 2955.0122, 2971.8623, 2963.7637, 2907.797, 2819.5107, 2857.0, 2855.9543, 2890.9917, 2926.455, 2942.7227, 2995.2751, 3035.43, 3090.5237, 3132.688, 3020.9092, 3016.5483, 2917.843, 2931.5627, 2949.0981, 3001.6904, 3012.908, 3043.9995, 3057.2979, 3045.5342, 3091.7788, 3119.869, 3056.7085, 3049.321, 3133.4067, 3184.0232, 3196.1633, 3245.1292, 3242.2803, 3197.1006, 3165.2349, 3193.671, 3182.3992, 3179.6443, 3184.5557, 3198.484, 3178.22, 3163.8784, 3135.049, 3081.891, 3080.8694, 3129.8325, 3120.242, 3128.4739, 3154.5798, 3119.5078, 3151.1843, 3058.4883, 3027.271, 3033.3655, 3072.827, 3080.3335, 3132.6165, 3169.1465, 3215.5088, 3198.173, 3149.0623, 3133.4414, 3155.8127, 3210.3223, 3274.175, 3263.973, 3265.2405, 3265.4417, 3334.4, 3345.3826, 3356.2097, 3427.9666, 3449.6973, 3466.0208, 3438.545, 3434.5051, 3377.0996, 3332.2278, 3367.5823, 3324.4023, 3328.3506, 3370.1606, 3403.1746, 3354.3945, 3386.7134, 3429.8308, 3424.242, 3472.766, 3473.3804, 3456.01, 3469.3472, 3478.0562, 3472.5935, 3510.7854, 3561.9102, 3596.1536, 3609.9148, 3589.3838, 3601.7295, 3641.452, 3725.4595, 3783.9858, 3753.9275, 3786.19, 3792.9023, 3795.0098, 3731.2295, 3699.391, 3691.6445, 3718.3232, 3752.2878, 3761.037, 3806.458, 3861.3623, 3876.4856, 3823.5813, 3800.037, 3749.9072, 3754.7378, 3819.439, 3791.698, 3785.1672, 3726.2202, 3720.4802, 3753.7273, 3774.9639, 3776.911, 3743.935, 3712.0571, 3755.2786, 3757.7148, 3785.4568, 3801.1687, 3808.18, 3827.2012, 3891.7725, 3913.1216, 3918.2622, 3939.4749, 3947.7998, 3935.7388, 3971.6367, 4014.2034, 3988.5107, 4009.836, 4064.907, 4119.1626, 4163.701, 4130.5806, 4192.6265, 4174.523, 4171.138, 4176.4478, 4143.0547, 4152.6733, 4188.8823, 4208.148, 4273.47, 4279.8687, 4190.4414, 4165.8364, 4232.2114, 4243.9644, 4166.0103, 4139.4434, 4030.1045, 3996.5398, 3907.0896, 3923.809, 3891.0835, 3868.5317, 3954.1458, 4017.8694, 4125.035, 4103.9673, 4077.8667, 4120.015, 4179.6494, 4203.7217, 4184.6724, 4139.259, 4049.0874, 4058.726, 4105.6587, 4133.1323, 4028.099, 4018.2852, 4056.5798, 4058.466, 3998.6848, 3990.006, 4054.671, 4115.0083, 4122.9946, 4217.4727, 4293.809, 4293.4683, 4315.191, 4325.8223, 4312.777, 4248.4043, 4304.653, 4223.2544, 4191.9116, 4150.33, 4203.8843, 4134.234, 4189.227, 4300.06, 4385.5737, 4430.687, 4476.827, 4452.39, 4444.252, 4538.8423, 4609.8657, 4620.836, 4635.4604, 4636.61, 4635.223, 4650.1797, 4641.055, 4630.6694, 4623.4414, 4610.018, 4642.807, 4733.9536, 4844.24, 4718.1216, 4860.7256, 4890.6553, 4912.2144, 4901.043, 4936.4375, 4935.4707, 4961.94, 4978.677, 5021.476, 5035.4253, 5003.831, 4959.879, 4744.6895, 4624.2563, 4621.6377, 4657.847, 4618.033, 4539.9814, 4455.92, 4464.1704, 4441.413, 4515.133, 4544.823, 4539.7964, 4579.837, 4579.307, 4609.9336, 4552.692, 4529.685, 4534.3438, 4571.499, 4561.4893, 4702.6777, 4786.3506, 4865.8755, 5086.3047, 5167.1387, 5190.1167, 5193.4375, 5186.5415, 5239.916, 5181.8325, 5162.5767, 5158.9062, 5188.7666, 5051.0396, 4880.485, 4957.964, 4812.4297, 4751.946, 4778.694, 4872.0923, 4804.9385, 4814.6865, 4832.387, 4855.638, 4899.684, 4841.1904, 4887.789, 4855.6997, 4747.7036, 4803.928, 4798.2646, 4856.848, 4857.783, 4717.4526, 4703.06, 4744.149, 4746.891, 4799.995, 4692.214, 4713.3364, 4723.2637, 4718.9443, 4832.116, 4880.332, 4926.633, 4939.1562, 4937.2773, 5009.4214, 5099.318, 5093.5186, 5110.4375, 5128.2886, 5046.8623, 4994.0576, 4974.8735, 5033.2705, 5101.801, 5182.9155, 5084.604, 5054.3853, 5116.528, 5146.923, 5246.0254]

#Test RMSE: 65.508
pred_extra = [3155.1035, 3194.0376, 3265.9148, 3318.4082, 3327.1624, 3325.9795, 3209.555, 3180.2837, 3244.7732, 3205.811, 3129.0398, 3146.667, 3068.429, 3065.993, 3070.6648, 2993.8635, 2950.4685, 2957.4927, 2993.177, 2988.562, 2905.856, 2906.6938, 2916.8308, 2908.7673, 2866.507, 2793.5454, 2814.8604, 2809.3843, 2832.5151, 2881.4, 2905.037, 2950.5042, 2986.3022, 3043.1917, 3077.9465, 2984.4878, 2983.6643, 2889.3945, 2898.915, 2925.6328, 2982.7668, 2985.742, 3018.6565, 3021.7124, 3011.649, 3052.3853, 3081.3923, 3017.331, 3016.769, 3096.8018, 3152.4023, 3165.4048, 3200.0896, 3199.9998, 3168.3586, 3143.7202, 3163.8066, 3155.0637, 3148.0327, 3154.9402, 3169.3147, 3158.0513, 3130.469, 3100.137, 3042.8792, 3037.207, 3094.6577, 3072.7292, 3098.2712, 3115.7478, 3080.114, 3097.5945, 3020.4758, 2997.96, 2995.865, 3042.4563, 3043.9229, 3079.2334, 3133.244, 3176.832, 3165.189, 3104.6729, 3099.53, 3122.266, 3178.401, 3234.6235, 3222.1287, 3231.5396, 3217.215, 3301.759, 3302.1255, 3304.1394, 3379.2192, 3402.9639, 3418.0298, 3397.9224, 3390.7537, 3324.3823, 3285.7932, 3320.817, 3275.4685, 3272.5137, 3321.6523, 3354.9702, 3311.7607, 3345.083, 3381.443, 3374.7468, 3413.3184, 3392.588, 3388.9446, 3418.0317, 3436.6655, 3425.031, 3454.9946, 3511.6016, 3546.346, 3555.5002, 3545.0938, 3548.3672, 3592.7, 3684.3113, 3739.9583, 3715.6262, 3736.5532, 3731.7458, 3737.2478, 3675.916, 3643.4302, 3632.9087, 3663.0488, 3695.7922, 3709.9211, 3762.12, 3805.1765, 3818.519, 3772.5264, 3749.9016, 3711.06, 3722.2732, 3778.5337, 3739.0283, 3735.965, 3676.4768, 3666.8323, 3710.424, 3730.9966, 3737.6428, 3701.239, 3672.105, 3709.998, 3703.901, 3737.7507, 3750.5955, 3763.6738, 3772.5903, 3846.5503, 3862.2603, 3861.1477, 3889.503, 3874.7544, 3892.871, 3933.1392, 3975.3643, 3945.848, 3965.285, 4028.1633, 4080.384, 4126.2104, 4089.622, 4143.8794, 4128.6123, 4121.8696, 4132.978, 4092.9338, 4107.998, 4149.0586, 4172.166, 4237.597, 4235.5664, 4149.7573, 4133.8657, 4198.35, 4206.637, 4129.6055, 4108.981, 4011.933, 3967.2185, 3866.7495, 3888.5195, 3841.021, 3844.1267, 3929.4568, 3972.346, 4070.4224, 4062.185, 4030.407, 4083.4614, 4130.903, 4157.77, 4140.941, 4109.96, 4017.692, 4002.2886, 4067.7883, 4094.0254, 4006.4563, 3975.8328, 4011.9326, 4011.9854, 3971.1104, 3973.53, 4013.7068, 4068.2266, 4063.347, 4183.7163, 4256.835, 4247.3467, 4277.3384, 4281.3857, 4271.957, 4214.2544, 4241.075, 4177.3647, 4142.5293, 4124.0234, 4165.035, 4098.151, 4144.047, 4259.981, 4360.5605, 4387.1577, 4431.503, 4392.683, 4383.799, 4480.907, 4542.5586, 4557.128, 4570.0293, 4577.749, 4575.575, 4584.4688, 4565.1064, 4578.3657, 4537.1914, 4552.613, 4579.932, 4671.479, 4764.718, 4670.105, 4789.8936, 4812.5693, 4827.582, 4810.143, 4863.085, 4851.574, 4879.2954, 4882.259, 4922.8613, 4944.2754, 4911.1694, 4864.9897, 4665.532, 4536.936, 4545.254, 4576.8975, 4543.695, 4465.85, 4380.587, 4394.287, 4376.4097, 4434.4087, 4460.3647, 4458.2695, 4496.917, 4497.8423, 4522.1255, 4464.9517, 4443.9517, 4438.1636, 4507.4785, 4487.499, 4645.1235, 4705.7993, 4798.0405, 5006.3887, 5080.4883, 5091.1826, 5102.6787, 5109.6816, 5136.8975, 5088.943, 5070.1553, 5065.4937, 5095.243, 4960.4956, 4804.2856, 4862.6455, 4709.729, 4670.167, 4682.3257, 4769.031, 4705.32, 4720.564, 4739.4106, 4770.2686, 4801.869, 4743.14, 4790.3945, 4759.6665, 4658.8296, 4715.5815, 4706.4385, 4763.7554, 4761.5854, 4633.8936, 4608.7075, 4644.9355, 4645.608, 4706.2817, 4577.6123, 4599.0205, 4610.137, 4607.6685, 4722.227, 4766.3965, 4811.2686, 4819.456, 4824.526, 4901.457, 4975.467, 4966.065, 4974.3853, 5020.9307, 4927.3887, 4877.0215, 4851.8296, 4904.832, 4973.8027, 5067.8857, 4936.854, 4937.417, 4984.142, 5019.9097, 5114.304]

accuracy(actual,pred_extra)

#Conclusion - Lower the RMSE better the accuracy (profit)




As per predictions : 
sell at 3327.1624
buy at 3180.2837
sell at 3244.7732
buy at 3129.0398
sell at 3146.667
buy at 3065.993
sell at 3070.6648
buy at 2950.4685
sell at 2993.177
buy at 2905.856
sell at 2916.8308
buy at 2793.5454
sell at 2814.8604
buy at 2809.3843
sell at 3077.9465
buy at 2889.3945
sell at 3021.7124
buy at 3011.649
sell at 3081.3923
buy at 3016.769
sell at 3200.0896
buy at 3143.7202
sell at 3163.8066
buy at 3148.0327
sell at 3169.3147
buy at 3037.207
sell at 3094.6577
buy at 3072.7292
sell at 3115.7478
buy at 3080.114
sell at 3097.5945
buy at 2995.865
sell at 3176.832
buy at 3099.53
sell at 3234.6235
buy at 3222.1287
sell at 3231.5396
buy at 3217.215
sell at 3418.0298
buy at 3285.7932
sell at 3320.817
buy at 3272.5137
sell at 3354.9702
buy at 3311.7607
sell at 3381.443
buy at 3374.7468
sell at 3413.3184
buy at 3388.9446
sell at 3436.6655
buy at 3425.031
sell at 3555.5002
buy at 3545.0938
sell at 3739.9583
buy at 3715.6262
sell at 3736.5532
buy at 3731.7458
sell at 3737.2

In [5]:
#oil

#with just OHLC

actual = [3202.9998, 3265.0, 3310.0, 3340.9998, 3329.0, 3178.0, 3212.9998, 3259.0, 3202.9998, 3127.0002, 3159.0, 3081.0, 3051.9998, 3090.0, 2966.0, 2945.0, 2958.9998, 2988.0, 2985.0, 2873.0, 2877.0, 2879.0, 2860.9998, 2813.0, 2741.0, 2776.0, 2777.0, 2801.0, 2860.0, 2892.0, 2918.0, 2968.0, 3043.0, 3068.0, 2938.0, 2947.0, 2863.0, 2875.9998, 2912.0, 2948.9998, 2963.9998, 2990.0, 2968.0, 2973.0, 3028.0, 3036.9998, 2965.0, 2997.0, 3090.0, 3132.0002, 3161.0, 3192.0, 3183.0, 3134.9998, 3147.0, 3143.0, 3149.0, 3144.0, 3139.0, 3157.0, 3128.0, 3127.0002, 3070.0, 3008.0, 3021.0, 3104.0, 3038.0, 3072.0002, 3107.0002, 3041.0, 3058.0, 2970.0, 2962.0, 2957.0, 3038.0, 3021.0, 3049.0, 3122.0002, 3154.9998, 3143.0, 3045.0, 3073.0, 3092.0002, 3162.0, 3215.0002, 3191.0, 3207.9998, 3171.0, 3286.0, 3295.0, 3287.0, 3406.0002, 3402.0, 3416.0002, 3373.0, 3373.0, 3310.9998, 3266.0, 3313.0002, 3240.0, 3256.0, 3330.9998, 3347.0, 3296.0, 3310.0, 3363.0, 3353.0, 3385.0, 3359.0, 3372.0, 3383.0, 3431.0002, 3384.0, 3428.0, 3501.0, 3513.0, 3528.0, 3500.0, 3520.0, 3579.0, 3691.0, 3734.0, 3695.0002, 3737.9998, 3712.0, 3720.0002, 3639.0, 3621.0, 3599.9998, 3661.0, 3663.0, 3678.0, 3747.0, 3798.0002, 3807.0, 3757.0, 3737.9998, 3678.0, 3687.0, 3762.0, 3714.0, 3724.0, 3637.0, 3657.0, 3692.0, 3720.0002, 3692.0, 3656.0, 3651.0, 3669.0, 3648.0, 3697.9998, 3716.0, 3732.0, 3737.0, 3833.0, 3818.0002, 3820.0, 3850.0, 3829.0, 3824.0, 3903.0, 3921.0, 3887.0, 3906.0002, 4003.0, 4036.0, 4100.0, 4065.0, 4116.0, 4086.0, 4084.9998, 4088.0, 4036.9998, 4045.0, 4110.0, 4136.0, 4190.0, 4177.0, 4106.0, 4112.0, 4195.0, 4186.0, 4103.0, 4099.0, 3959.0002, 3931.0, 3807.0, 3842.0, 3813.0002, 3823.0, 3911.0002, 3941.0, 4036.9998, 4034.0, 3999.0002, 4086.0, 4125.0, 4155.0, 4122.0, 4093.0, 3983.0, 3995.0, 4075.0, 4063.0, 3960.0, 3943.0, 4016.9998, 3977.0, 3918.9998, 3948.0, 3984.0002, 4050.0, 4039.0, 4159.0, 4251.0, 4201.0, 4266.0, 4247.0, 4232.0, 4185.0, 4234.0, 4112.0, 4127.0, 4111.0, 4115.0, 4029.0, 4116.0, 4248.0, 4354.0, 4362.0, 4401.0, 4351.0, 4348.0, 4470.0, 4495.0, 4531.0, 4565.0, 4530.0, 4563.0, 4558.0, 4547.0, 4583.0, 4487.0, 4512.0, 4571.0, 4668.0, 4749.0, 4623.0, 4799.0, 4757.0, 4793.0, 4794.0, 4850.0, 4820.0, 4832.0, 4855.0, 4919.0, 4929.0, 4913.0, 4859.0, 4597.0, 4508.0, 4527.0, 4614.0, 4517.0, 4428.0, 4351.0, 4392.0, 4342.0, 4443.0, 4445.0, 4472.0, 4478.0, 4506.0, 4517.0, 4435.0, 4481.0, 4419.0, 4486.0, 4496.0, 4672.0, 4658.0, 4815.0, 5028.0, 5075.0, 5086.0, 5109.0, 5080.0, 5126.0, 5039.0, 5075.0, 5049.0, 5071.0, 4900.0, 4780.0, 4884.0, 4642.0, 4669.0, 4699.0, 4799.0, 4707.0, 4690.0, 4745.0, 4778.0, 4800.0, 4714.0, 4824.0, 4734.0, 4629.0, 4744.0, 4686.0, 4767.0, 4762.0, 4570.0, 4602.0, 4674.0, 4643.0, 4712.0, 4583.0, 4598.0, 4633.0, 4617.0, 4763.0, 4774.0, 4804.0, 4826.0, 4809.0, 4926.0, 4999.0, 4959.0, 5012.0, 5012.0, 4934.0, 4884.0, 4883.0, 4916.0, 5044.0, 5085.0, 4918.0, 4950.0, 4995.0, 5055.0, 5152.0, 5069.0]
#Test RMSE: 67.674
pred_normal = [3136.928, 3180.1829, 3248.783, 3305.713, 3313.1929, 3326.8132, 3220.2856, 3180.0154, 3230.907, 3190.7673, 3124.8838, 3151.0718, 3066.857, 3060.9229, 3074.3232, 2992.9443, 2943.8467, 2945.1465, 2984.2588, 2971.7053, 2889.324, 2878.1824, 2876.767, 2863.5632, 2829.9531, 2770.2527, 2784.0232, 2778.094, 2795.8013, 2855.0671, 2881.5962, 2922.456, 2956.7893, 3025.8594, 3051.9438, 2959.2725, 2963.2053, 2867.1995, 2873.9863, 2901.5059, 2959.0276, 2955.6863, 2986.3306, 2979.2979, 2975.741, 3015.9812, 3048.3743, 2975.8694, 2983.1245, 3070.9578, 3127.1658, 3144.119, 3170.7458, 3178.0833, 3155.303, 3130.0945, 3144.9058, 3131.0867, 3126.8394, 3132.6208, 3146.4167, 3147.3271, 3107.2598, 3080.0115, 3017.535, 3013.1333, 3077.357, 3041.0103, 3079.1453, 3091.696, 3044.9348, 3051.5828, 2979.2537, 2970.832, 2958.3318, 3017.5322, 3003.7554, 3029.2493, 3102.939, 3143.248, 3134.6992, 3057.333, 3061.6714, 3083.4414, 3139.8645, 3197.6035, 3176.2163, 3198.2634, 3171.839, 3285.0635, 3278.3083, 3275.6104, 3366.6301, 3396.6792, 3412.3193, 3386.1565, 3372.39, 3299.428, 3253.0552, 3294.023, 3243.7063, 3235.4688, 3299.7075, 3330.225, 3281.8772, 3313.0854, 3351.395, 3341.9128, 3371.1204, 3331.273, 3338.0225, 3377.9421, 3406.3179, 3378.0093, 3399.0125, 3468.688, 3501.3164, 3504.7947, 3501.3423, 3498.0103, 3557.5889, 3666.172, 3723.3965, 3690.642, 3711.07, 3693.9407, 3699.5623, 3633.1, 3592.297, 3584.5962, 3621.3442, 3658.3928, 3672.1975, 3730.0461, 3770.1401, 3779.508, 3743.935, 3714.008, 3674.7078, 3689.5203, 3753.38, 3699.0054, 3698.1592, 3632.7551, 3623.5264, 3674.9668, 3691.5735, 3699.0928, 3657.7485, 3626.968, 3653.2764, 3644.9756, 3683.7686, 3691.783, 3709.7876, 3711.188, 3795.7773, 3799.892, 3798.794, 3830.0933, 3792.4507, 3824.2646, 3873.1628, 3909.4333, 3874.8728, 3893.551, 3978.0522, 4025.6885, 4079.0986, 4037.7307, 4090.8586, 4073.347, 4056.9536, 4073.4421, 4021.498, 4046.1055, 4099.246, 4121.6606, 4185.4526, 4182.185, 4095.1965, 4089.0798, 4174.2227, 4183.887, 4110.28, 4093.057, 3997.882, 3935.9946, 3818.477, 3844.5657, 3784.0486, 3799.2607, 3894.033, 3928.1846, 4022.5618, 4018.5715, 3978.7444, 4055.68, 4102.341, 4134.623, 4114.862, 4087.4353, 3985.5796, 3965.753, 4053.1335, 4063.3965, 3982.6868, 3945.533, 3986.554, 3978.7214, 3936.4487, 3948.1357, 3971.2778, 4019.223, 4009.8892, 4150.791, 4226.78, 4205.308, 4251.132, 4242.6284, 4247.103, 4188.052, 4203.284, 4143.2397, 4109.669, 4097.353, 4120.078, 4042.4294, 4091.024, 4218.451, 4342.827, 4353.927, 4388.8345, 4339.6157, 4334.111, 4444.586, 4506.0063, 4525.7744, 4544.071, 4550.645, 4551.0312, 4562.532, 4529.0234, 4557.117, 4485.8823, 4512.898, 4550.98, 4640.692, 4731.638, 4647.7446, 4774.956, 4781.5415, 4795.1245, 4775.9995, 4849.62, 4818.8286, 4851.154, 4851.9253, 4905.237, 4943.4326, 4912.9995, 4864.245, 4655.4907, 4501.9756, 4529.8643, 4568.5933, 4534.8687, 4442.73, 4355.595, 4371.686, 4351.577, 4413.9683, 4434.434, 4433.9287, 4470.5854, 4484.5347, 4502.024, 4455.152, 4428.319, 4417.3457, 4490.6426, 4466.7856, 4636.1196, 4666.991, 4782.8975, 4999.309, 5068.6587, 5066.04, 5085.0396, 5107.738, 5120.3955, 5069.919, 5052.912, 5064.4307, 5097.4995, 4951.025, 4805.6465, 4865.2227, 4688.0293, 4654.503, 4656.522, 4770.922, 4685.301, 4704.5103, 4727.6875, 4766.679, 4783.928, 4725.918, 4790.652, 4742.44, 4638.0547, 4710.904, 4686.9106, 4768.4966, 4758.033, 4625.149, 4597.1167, 4640.2827, 4659.7637, 4727.2637, 4574.9355, 4595.584, 4600.143, 4594.2324, 4727.386, 4761.404, 4807.469, 4817.2114, 4818.719, 4910.59, 4984.668, 4970.888, 4981.096, 5046.8755, 4945.7583, 4906.962, 4875.8555, 4938.2617, 5011.0303, 5099.527, 4951.0127, 4964.945, 5015.0815, 5054.444, 5141.799]

accuracy(actual,pred_normal)


As per predictions : 
sell at 3326.8132
buy at 3180.0154
sell at 3230.907
buy at 3124.8838
sell at 3151.0718
buy at 3060.9229
sell at 3074.3232
buy at 2943.8467
sell at 2984.2588
buy at 2770.2527
sell at 2784.0232
buy at 2778.094
sell at 3051.9438
buy at 2959.2725
sell at 2963.2053
buy at 2867.1995
sell at 2959.0276
buy at 2955.6863
sell at 2986.3306
buy at 2975.741
sell at 3048.3743
buy at 2975.8694
sell at 3178.0833
buy at 3130.0945
sell at 3144.9058
buy at 3126.8394
sell at 3147.3271
buy at 3013.1333
sell at 3077.357
buy at 3041.0103
sell at 3091.696
buy at 3044.9348
sell at 3051.5828
buy at 2958.3318
sell at 3017.5322
buy at 3003.7554
sell at 3143.248
buy at 3057.333
sell at 3197.6035
buy at 3176.2163
sell at 3198.2634
buy at 3171.839
sell at 3285.0635
buy at 3275.6104
sell at 3412.3193
buy at 3253.0552
sell at 3294.023
buy at 3235.4688
sell at 3330.225
buy at 3281.8772
sell at 3351.395
buy at 3341.9128
sell at 3371.1204
buy at 3331.273
sell at 3406.3179
buy at 3378.0093
sell at 35

In [9]:
#ARIMAX


#ONGC Close


actual = [174.649994, 173.699997, 174.149994, 175.75, 175.75, 178.649994, 177.0, 174.0, 174.0, 175.75, 171.449997, 172.050003, 170.550003, 169.199997, 168.5, 169.100006, 169.5, 167.149994, 167.149994, 166.050003, 169.100006, 164.800003, 160.449997, 158.100006, 159.949997, 158.100006, 157.850006, 157.300003, 159.100006, 159.899994, 162.800003, 161.699997, 159.600006, 162.399994, 160.25, 162.850006, 158.899994, 158.350006, 160.050003, 161.699997, 163.0, 165.850006, 164.350006, 163.25, 163.300003, 164.0, 162.899994, 164.75, 169.399994, 167.399994, 165.399994, 165.699997, 166.399994, 165.899994, 164.0, 165.699997, 163.699997, 159.649994, 161.699997, 160.949997, 160.600006, 160.899994, 157.149994, 159.449997, 159.699997, 159.449997, 160.600006, 157.25, 158.399994, 156.949997, 159.949997, 161.800003, 162.800003, 162.300003, 162.300003, 161.850006, 163.050003, 162.300003, 161.050003, 159.350006, 166.75, 165.199997, 165.899994, 167.699997, 165.899994, 164.449997, 164.800003, 171.399994, 170.199997, 171.550003, 170.949997, 170.649994, 172.100006, 171.199997, 173.850006, 170.699997, 170.350006, 169.149994, 169.699997, 169.649994, 172.050003, 171.199997, 174.449997, 172.050003, 171.699997, 176.0, 176.850006, 177.350006, 184.0, 186.600006, 191.100006, 191.0, 188.0, 191.850006, 199.25, 193.800003, 192.550003, 189.600006, 191.100006, 184.199997, 182.0, 177.550003, 179.149994, 177.600006, 180.0, 180.899994, 181.300003, 181.699997, 181.0, 184.050003, 181.25, 182.449997, 180.649994, 179.649994, 181.600006, 178.399994, 175.850006, 177.050003, 180.25, 178.899994, 183.300003, 183.899994, 184.350006, 183.0, 183.050003, 185.199997, 188.0, 188.0, 193.399994, 193.699997, 192.600006, 193.0, 195.199997, 192.350006, 196.850006, 193.449997, 199.5, 198.449997, 197.399994, 197.0, 197.899994, 197.899994, 200.75, 196.600006, 197.199997, 195.699997, 194.100006, 193.699997, 200.699997, 207.350006, 210.850006, 208.5, 205.149994, 204.800003, 203.449997, 194.899994, 192.699997, 188.75, 185.449997, 190.0, 188.699997, 187.350006, 191.0, 185.75, 188.300003, 186.899994, 185.300003, 187.0, 190.149994, 186.100006, 189.800003, 190.350006, 188.5, 188.300003, 189.699997, 185.550003, 185.399994, 181.5, 180.149994, 179.899994, 183.850006, 183.600006, 180.600006, 180.300003, 177.149994, 176.850006, 174.100006, 175.550003, 178.899994, 177.399994, 178.899994, 179.600006, 177.800003, 179.949997, 177.5, 175.399994, 177.449997, 176.949997, 178.399994, 180.199997, 182.149994, 181.75, 182.5, 181.5, 180.800003, 182.100006, 183.949997, 182.350006, 181.050003, 182.649994, 179.449997, 180.800003, 180.699997, 180.550003, 180.050003, 181.100006, 180.25, 183.899994, 183.100006, 182.850006, 188.25, 187.5, 187.25, 189.0, 186.199997, 188.600006, 185.050003, 185.75, 185.199997, 175.399994, 167.699997, 175.449997, 175.600006, 175.550003, 174.0, 177.899994, 173.199997, 171.100006, 171.199997, 170.300003, 172.649994, 173.699997, 173.300003, 171.399994, 170.0, 168.5, 165.449997, 164.600006, 164.800003, 162.75, 159.699997, 159.550003, 156.649994, 158.300003, 156.25, 153.5, 158.399994, 155.5, 157.699997, 156.0, 156.399994, 155.449997, 157.399994, 157.949997, 158.100006, 158.949997, 154.100006, 153.800003, 156.149994, 160.300003, 161.75, 157.949997, 156.550003, 157.199997, 159.050003, 162.550003, 161.300003, 164.100006, 165.699997, 167.199997, 166.399994, 166.800003, 169.25, 167.550003, 172.5, 168.899994, 169.25, 166.050003, 166.25, 164.449997, 163.050003, 168.699997, 170.199997, 171.899994, 174.75, 176.050003, 174.600006, 177.0, 177.449997, 180.050003, 177.300003, 174.350006, 173.699997, 174.5, 171.899994, 168.649994, 167.800003, 168.5, 172.550003, 171.699997, 173.899994, 177.0]

predicted = [174.86682455830154, 173.7960164616196, 173.87730644728796, 176.24825706464821, 173.33270537081734, 176.22949592799648, 175.58131722286828, 172.8335848289425, 173.06848111265066, 177.036797449645, 171.62048344633172, 172.64311871735572, 169.2726461285975, 168.20572332612488, 168.45659518619817, 167.94533448334, 168.59407197970143, 167.52281421381224, 166.35639337384538, 164.98239577723638, 168.19059213105007, 164.64252028243868, 160.83292340252933, 157.79187423344, 156.88688588660776, 157.98378599088315, 158.26099741489324, 156.7999908664313, 157.2587330150745, 160.4048776357984, 161.96862818703715, 160.48238040650546, 159.46222885543798, 162.07286851430575, 160.57338326708276, 164.35075728881128, 159.30071976593814, 157.8681712986101, 159.13240266662064, 160.3011687521661, 160.87407163515866, 166.02767547058505, 163.75276309122023, 161.85247426530896, 161.70936183767847, 163.22843950496735, 162.0917592766431, 163.08055397360363, 168.3928844290735, 167.00744571653715, 164.2321270747438, 164.3168073156063, 164.87305916335143, 165.53265113695807, 162.8999029551237, 164.10839593341012, 162.97214796685432, 158.92106227830408, 161.0492885665583, 159.4387225016858, 160.2159366210234, 160.2470784384042, 159.4723234011268, 158.96180271564046, 159.149243008209, 158.7308142511797, 159.986862343555, 157.05369463893112, 157.7594196688026, 156.52905160290595, 158.79411183899504, 160.3018406966387, 161.66745936533778, 162.52447228703608, 161.314405563773, 160.15076481159096, 161.97696192849583, 161.5462077059659, 161.0650363714654, 159.19231104143768, 165.28724990484602, 165.38686750436875, 165.36263516570114, 167.6025285149155, 164.4030709797496, 163.11332007739202, 162.21253228363202, 170.65269400817414, 170.33925174326663, 170.0089988414415, 171.81617585199524, 170.27625631307893, 171.25216876554833, 170.56272865725498, 173.2488706840644, 170.45384529106673, 169.80801296775996, 168.71977204184032, 168.2643467258061, 168.8126948774588, 171.4657226429489, 170.28086785920274, 173.1624308870758, 171.87679739182022, 170.81693000594402, 175.1626325410382, 175.6931815250494, 178.22110574426222, 182.36138952420265, 188.45433702981734, 188.84296295953072, 190.7590025179362, 188.57653400928154, 189.80595961559487, 196.93317967033795, 196.77088622484638, 191.73071147336725, 187.81639173200688, 190.1415814051942, 184.57182703389117, 181.60656847026308, 176.85098319465703, 177.75601523959722, 178.58280392176323, 178.65150621085763, 179.00950718475212, 180.82782836933166, 180.79900927448261, 180.24886894508995, 182.72869555379538, 180.36795856677213, 182.93309581650686, 179.9116102996053, 179.87437818737914, 182.30455864077373, 177.7284695644763, 176.04467816378633, 176.17006461040768, 179.93857021292723, 177.96892143813062, 182.63027448054464, 185.20256488075222, 183.1016211488211, 183.00493188230172, 178.40181780313338, 184.693453929412, 187.81654415131544, 187.34262002796845, 191.3804366950306, 194.30441886053433, 192.06587168622968, 192.561810123494, 193.63128583111148, 192.37987879243752, 195.82759631291782, 193.24223201952827, 198.2820172170933, 196.58561778312747, 197.76873619862744, 196.58911426746278, 198.0965807407315, 197.8361621838327, 199.6405214118522, 196.71748719489835, 195.35325369469695, 195.75065544480492, 194.10234688398276, 193.36209854027402, 203.69489367207478, 205.4320450426332, 209.7343519088281, 207.8578840808347, 204.66633784623974, 206.15807257440014, 200.27656295894036, 197.96846798876925, 191.68039920226585, 188.30228007293357, 183.8955893566088, 189.31253569301106, 189.01805660687285, 186.79102241508508, 192.01228818197217, 185.91460697693253, 186.29695736014145, 187.4005649425162, 182.3979330781592, 186.49874920589338, 187.00459410807804, 185.76588812511682, 188.9854777874179, 190.25024137066384, 189.21154878557854, 187.92115423309906, 188.6104676893251, 184.89360817358374, 184.5546223059751, 179.71149402127014, 179.85568834798494, 181.1334924551968, 181.4967650149867, 182.494465862918, 180.7955262520672, 178.82550229913153, 176.48514489405005, 176.3793413709344, 174.65349311712436, 174.73600324520982, 179.78495230422658, 178.19594306347972, 176.09254871744355, 177.7514215989312, 176.36027465260736, 179.32270143204016, 177.2229053254896, 174.87744336931218, 176.13829074830153, 176.37708229835505, 177.45738458252123, 178.74876364297137, 183.0778500560354, 181.46904373013484, 181.93067112541718, 179.76061132256888, 179.43684716034164, 181.84588291830391, 183.81908787665742, 181.2749808929878, 179.88952677725064, 181.69547375859793, 179.34891871549792, 179.0052827094496, 180.83126929752854, 180.45304865173313, 179.0564328104818, 181.00378231789506, 182.08363489666215, 182.83725618284626, 181.03192952921256, 183.08293900148544, 186.9843464667557, 186.44517431516215, 185.5399542055752, 187.01110265402235, 178.10283070081914, 187.25909028234173, 187.1148170925611, 186.54698544609346, 185.64956479612857, 176.9810767056603, 159.60416935182172, 171.2333288706099, 176.40678925239385, 174.216841546203, 175.15950324115875, 177.12237705312396, 171.15802231666208, 170.86768222985444, 170.29562744660473, 170.2758815307783, 172.4026780259756, 172.58787481973883, 172.37719490596902, 172.83387873206487, 169.41432646803003, 167.97041096660894, 165.8706697229285, 165.01888520144922, 164.42897317384242, 162.1604538302004, 158.8680418195263, 158.48062187435926, 156.73823146251976, 157.55113084784665, 155.1371695024805, 154.10027705744017, 157.1912240421978, 154.00835470623832, 159.50493436030985, 155.22853074912302, 155.30570315271018, 155.380801914022, 156.72553037490968, 158.05091313484303, 156.8360347797211, 155.73382096992017, 154.23636364667425, 153.41723604006603, 154.96369863584698, 162.6776365927994, 162.6354840964271, 157.35982423507699, 155.5006257022071, 156.35804176877116, 159.18779222428998, 161.7139346692561, 160.29877319179877, 164.65575200050245, 163.76943747961226, 167.37278346022555, 166.68458417880464, 166.99067546417422, 168.32884595960095, 166.67979796443404, 171.25634963180218, 168.9285950510835, 168.9940328210335, 165.6225405833946, 165.0717076456064, 164.2416588955044, 162.72268488703708, 168.15919132865434, 168.0072501026187, 169.93469345914542, 174.65219724574126, 175.5941121565907, 175.11273383757373, 176.63265361112417, 179.69917802823863, 179.3137262070319, 176.88807095253844, 174.18725292239947, 171.54414420517048, 172.45350339384595, 171.31438226525086, 168.63608211166866, 168.0954906784495, 169.29176073355325, 171.08209852613643, 172.07568166347215, 173.04263520355647, 177.00626563547598]

accuracy(actual,predicted)

As per predictions : 
sell at 174.86682455830154
buy at 173.7960164616196
sell at 176.24825706464821
buy at 173.33270537081734
sell at 176.22949592799648
buy at 172.8335848289425
sell at 177.036797449645
buy at 171.62048344633172
sell at 172.64311871735572
buy at 168.20572332612488
sell at 168.45659518619817
buy at 167.94533448334
sell at 168.59407197970143
buy at 164.98239577723638
sell at 168.19059213105007
buy at 156.88688588660776
sell at 158.26099741489324
buy at 156.7999908664313
sell at 161.96862818703715
buy at 159.46222885543798
sell at 162.07286851430575
buy at 160.57338326708276
sell at 164.35075728881128
buy at 157.8681712986101
sell at 166.02767547058505
buy at 161.70936183767847
sell at 163.22843950496735
buy at 162.0917592766431
sell at 168.3928844290735
buy at 164.2321270747438
sell at 165.53265113695807
buy at 162.8999029551237
sell at 164.10839593341012
buy at 158.92106227830408
sell at 161.0492885665583
buy at 159.4387225016858
sell at 160.2470784384042
buy at 158.96

In [ ]:
#ARIMAX
actual_ioc = [212.125, 215.274994, 219.475006, 213.024994, 209.625, 209.774994, 214.725006, 206.925003, 207.399994, 211.024994, 212.300003, 213.675003, 209.375, 210.024994, 211.824997, 209.875, 210.350006, 203.524994, 202.300003, 203.600006, 205.850006, 204.125, 196.75, 191.699997, 195.199997, 193.574997, 193.425003, 192.600006, 193.449997, 190.925003, 192.850006, 190.800003, 191.125, 191.725006, 193.75, 196.300003, 190.475006, 186.550003, 186.774994, 186.074997, 187.574997, 187.274994, 190.449997, 189.074997, 187.399994, 186.824997, 184.449997, 183.899994, 183.824997, 185.175003, 185.050003, 193.524994, 209.475006, 215.600006, 206.625, 207.024994, 205.149994, 205.899994, 209.725006, 211.149994, 213.149994, 213.675003, 206.600006, 211.600006, 213.425003, 214.100006, 220.350006, 217.25, 226.425003, 227.350006, 224.149994, 214.524994, 215.925003, 214.125, 216.100006, 214.574997, 214.524994, 217.449997, 207.975006, 209.850006, 207.824997, 207.824997, 209.149994, 206.975006, 206.625, 204.199997, 199.5, 199.824997, 197.824997, 198.699997, 200.25, 203.274994, 204.375, 203.375, 207.725006, 207.074997, 205.425003, 209.050003, 207.449997, 207.300003, 206.774994, 206.949997, 205.800003, 204.824997, 202.225006, 203.850006, 208.625, 218.25, 207.475006, 208.125, 207.75, 208.399994, 207.475006, 206.625, 205.324997, 200.100006, 198.5, 197.300003, 197.425003, 198.425003, 193.475006, 194.75, 196.5, 197.574997, 196.475006, 198.074997, 198.0, 197.375, 197.699997, 196.300003, 195.300003, 196.850006, 196.75, 193.925003, 195.274994, 195.25, 194.375, 195.75, 202.425003, 202.199997, 197.550003, 201.100006, 205.0, 206.574997, 204.800003, 204.949997, 205.050003, 205.475006, 205.125, 204.975006, 200.75, 196.0, 194.274994, 194.975006, 193.199997, 193.149994, 193.649994, 191.925003, 194.274994, 196.199997, 194.774994, 194.449997, 194.524994, 195.425003, 192.074997, 192.024994, 189.024994, 192.975006, 191.25, 194.699997, 195.149994, 196.149994, 199.824997, 208.125, 208.800003, 208.949997, 204.324997, 204.5, 200.649994, 202.375, 192.800003, 189.324997, 189.524994, 188.675003, 188.399994, 187.524994, 184.824997, 183.300003, 184.25, 181.600006, 184.199997, 188.350006, 188.725006, 189.649994, 190.75, 189.625, 191.100006, 189.699997, 191.050003, 192.75, 200.324997, 203.925003, 197.524994, 192.050003, 184.75, 177.899994, 173.850006, 172.449997, 170.100006, 168.850006, 167.199997, 174.850006, 176.600006, 170.449997, 174.149994, 171.149994, 173.550003, 174.25, 179.800003, 179.300003, 167.350006, 168.25, 165.350006, 165.0, 167.199997, 164.949997, 157.949997, 160.100006, 161.699997, 163.050003, 160.600006, 161.149994, 162.899994, 162.300003, 161.149994, 161.699997, 161.5, 164.300003, 166.550003, 167.199997, 167.949997, 169.100006, 168.399994, 167.199997, 165.949997, 164.75, 163.699997, 165.149994, 160.5, 153.5, 155.899994, 163.449997, 170.449997, 172.449997, 170.800003, 174.100006, 172.350006, 173.800003, 173.949997, 174.899994, 174.800003, 175.0, 173.100006, 174.149994, 175.100006, 171.550003, 167.75, 173.899994, 168.600006, 165.399994, 170.25, 170.0, 167.350006, 166.800003, 155.649994, 154.350006, 156.0, 153.699997, 155.300003, 155.25, 155.0, 156.699997, 157.300003, 155.25, 154.550003, 156.850006, 159.100006, 157.5, 164.449997, 167.199997, 165.800003, 161.050003, 162.5, 164.949997, 163.100006, 158.800003, 165.399994, 166.449997, 164.350006, 168.0, 168.699997, 168.550003, 168.149994, 166.649994, 165.899994, 165.399994, 166.350006, 160.649994, 160.699997, 158.5, 159.550003, 158.649994, 157.699997, 153.399994, 154.25, 156.100006, 155.550003, 156.5, 154.25, 155.550003, 155.0, 152.449997, 152.800003, 153.600006, 153.399994, 151.050003, 150.699997, 149.100006, 155.25, 156.550003, 152.600006, 153.550003]
predicted_ioc = [212.52235071842495, 215.23040060946974, 217.12237314678973, 211.15978797830232, 211.27910661804768, 213.2497883055136, 214.98669124539825, 207.63832098903345, 207.15799638012209, 210.4246052337253, 212.50486384023066, 211.8610493406792, 211.35250819392343, 210.11704659663144, 210.7039095527698, 210.1947300425736, 211.1534954690162, 204.70148765364866, 205.6043053454141, 202.91421513252635, 205.89798350358205, 204.6682028606242, 199.25229929938826, 191.03610261243307, 191.43481960666793, 197.5833433901745, 192.93595150274973, 193.37886219170184, 193.100054951429, 192.88561978428947, 191.71551051292886, 191.1213691783903, 189.59120472140114, 192.56848088922845, 191.94032674470202, 198.09189008888893, 191.52118465635533, 186.96052603074622, 185.86596188852525, 187.18109202723974, 187.81627150497695, 187.76031520121205, 189.80396676592818, 190.01087194178422, 187.77140336568164, 186.7104864818, 184.67096196452883, 183.1356605821759, 185.46052457049427, 186.15136101522216, 184.22389948637817, 194.0737946751109, 208.24052582322327, 212.8460731486732, 206.63012646336222, 209.55561129669286, 205.9768458959744, 205.3118426319129, 209.5229616403078, 212.34868247190263, 213.2736151578465, 215.54984812977443, 209.6506140513486, 212.7697660191826, 215.00560254001152, 215.0355826399748, 219.0638930976645, 219.0670431541763, 224.7479972395488, 227.7392508461562, 224.90961554524006, 216.8112677819397, 216.43385335067433, 215.20616952545765, 214.70506146327796, 215.55172768624948, 215.56239309039393, 216.8457102213058, 208.60496321869493, 209.9563953102521, 209.29492056950096, 208.14018258592114, 208.89011442670324, 206.98278401427797, 208.19112458312577, 201.3279919274945, 199.70895007703336, 195.33591700656757, 199.43888988192668, 192.18979898660496, 200.06153973438632, 201.9266983724043, 204.94780375401456, 201.62939289352994, 206.70074766263053, 208.4393365581218, 206.83769488290545, 210.28709746682478, 208.51186169942082, 208.85281107610626, 208.08901388273034, 209.4916431841782, 206.38750358560105, 205.35775610368296, 204.0550874220096, 205.38225562088883, 207.85601963501426, 216.9551461371292, 208.70223783205097, 208.68553491868124, 209.39018589968896, 209.0535435771484, 208.18605548438688, 206.77417795291103, 205.01970047430484, 199.0684773586754, 197.79527636120156, 197.38042542832864, 196.70475543322175, 195.8431424254693, 194.42674102683793, 196.02128407738815, 195.98368049690214, 197.22352350270762, 195.9411692600861, 197.2765679151189, 196.80865201915515, 198.2206039768423, 196.92941490957026, 195.59802349518077, 194.6631785858458, 196.36386352320812, 196.56677677936614, 194.40428109346942, 193.10671485545947, 193.86545023672, 194.84344609999755, 195.3458123252825, 200.85027688693992, 201.72335746513363, 197.84691602737462, 200.403248745543, 203.3128534560899, 206.5777857336766, 202.2178452183491, 204.49126811032852, 205.12415676488877, 205.78319834991976, 205.34252424377928, 205.10329143089882, 201.37343856578605, 197.601535877071, 194.83868690761847, 195.35001263409777, 193.25433644944292, 193.63426836747658, 193.0189734224806, 192.31629338596554, 194.37842019215555, 195.2147259592956, 195.45767916168836, 194.66896937160072, 196.59066023416221, 197.244879867696, 193.04232860383706, 189.62370044925842, 189.7418049313369, 191.70871039319672, 190.7971394997959, 194.65911363177997, 194.5768710478436, 195.98319429463083, 198.92510981448243, 206.6862748198024, 210.12946927449843, 207.18849947174067, 205.1246431932068, 203.32151534675359, 201.8772985083449, 203.45058478507167, 194.07542954267694, 188.81851695554735, 188.90482982715844, 188.76910948135742, 188.34153763075938, 188.216706159777, 184.91193839382584, 183.7763143787763, 183.14587243794028, 181.10669463772433, 183.67681833887605, 189.08570531267753, 189.24892733518448, 188.81936316954432, 190.3783676387186, 189.65571340268298, 194.40527146753402, 190.52186313464256, 190.1002605531296, 191.75577093319737, 198.9437504060994, 205.00840254928778, 199.53844315181595, 191.33220852536408, 185.51195564622242, 177.96115167403013, 174.00712358012186, 173.50130571972295, 170.51975222432293, 168.44974531654782, 165.93266842195476, 173.3309318802667, 175.88059455776738, 171.97416608258777, 173.78285774188225, 171.2386196424377, 172.90601743321167, 174.72598212038122, 179.0558284493277, 177.9930906773452, 168.54612687865858, 165.28142592600727, 165.9213488041761, 164.63379301069847, 166.5167795761571, 165.27750665098438, 158.6067953400734, 159.45896213074843, 162.34294494895357, 162.70931626801166, 162.03130461850424, 159.59426768933417, 161.96706818058567, 161.89317557110564, 160.87664809505463, 161.60720167583622, 160.74023317498302, 162.93436360947726, 164.8366022901978, 165.27945583702478, 166.80607426464678, 171.7023530175485, 170.56763291237758, 167.11427476715198, 164.21350494274702, 165.2889601866566, 163.771489725607, 164.37422321820816, 159.96018068590683, 154.71094721263057, 153.26290644055712, 162.38510080615345, 174.5755978459971, 171.45270209878936, 170.52318254503507, 173.4396872173288, 173.01392041876284, 172.6023725575003, 172.68635491944212, 173.94470939865562, 174.42908998749317, 173.82804703390354, 174.96805706422344, 174.5790249819372, 173.9220095690771, 171.92589997062805, 168.29784682713648, 175.09467372964036, 169.28658310947446, 166.70473742824106, 168.6796325988223, 170.7025763550937, 166.65948771700357, 168.72374104550647, 156.96273196623815, 156.44849701349608, 156.16979811506815, 154.24005858726605, 154.72213258967417, 155.34408260505052, 156.3491659200879, 158.73300630109566, 158.02577409615355, 155.57350286083943, 152.57713082240707, 159.7649540176758, 156.97364557618766, 158.27632690954306, 163.63511578822465, 167.36992502210728, 163.64248774193084, 161.35919457797445, 164.2637620378664, 163.64812374218013, 161.02340579966256, 160.0603580864581, 164.4358922434393, 167.5101238545908, 164.92180470043502, 168.1227160259309, 169.82779480646855, 168.6802796373671, 167.84047071323687, 167.80659182488887, 166.70332151789614, 166.13555623995592, 165.941624710038, 161.7917459581629, 160.06707350494628, 159.24152967970906, 159.49663527379357, 159.0704371767132, 158.83403202600402, 153.15857587671633, 153.2828627892195, 156.07274585071332, 155.7523728453658, 156.13551227038283, 154.51961724074658, 154.896800449902, 155.16606367840313, 152.75533615551524, 151.56465762004518, 152.48216391695848, 153.00909625514578, 151.0103074998699, 151.67366109107525, 146.49672128057193, 154.45858074801583, 156.80786912379267, 153.43815530472205, 153.78255208405596]
accuracy(actual_ioc,predicted_ioc)


In [7]:
#ARIMAX
predicted_tci = [261.8304869363691, 260.90223283094736, 256.9456902428551, 266.9600691741243, 259.8589950301497, 267.17056248662357, 309.75968461959366, 325.28945967753134, 333.758483358071, 337.676634754911, 336.29642475917717, 335.85617840707357, 329.2750085346207, 324.99000371444845, 314.07817082931615, 319.51965941142487, 318.0804144155263, 318.7292928898304, 320.6695922077887, 319.4810263421948, 315.45854852824834, 313.19062541077085, 311.5367535540929, 307.1637724241067, 311.8473704315179, 297.8589748500989, 333.37310181773523, 337.19939680208233, 337.06198630245296, 347.7545950114628, 347.2261975777094, 353.21161406892236, 350.1742257663918, 348.5859759154582, 345.2954538493941, 343.7173765158493, 345.90225201522196, 337.81283051364574, 329.9369378943246, 332.3180204165859, 332.36805704983794, 331.77834186702455, 328.27776347489703, 326.1775403166747, 330.3088837277445, 326.40273662191396, 326.22318165741353, 319.1059199297131, 321.3851057349003, 318.5814373370382, 315.0687671197393, 320.2232950868897, 311.4914963183626, 306.54460475328415, 302.1244135663442, 292.92256741455435, 284.4148436208684, 288.8760589153561, 291.78062349754657, 294.9174513683656, 291.7069647174034, 288.2385514611133, 284.6939733608593, 279.1988538971974, 278.55433643645085, 281.1556902014939, 293.72767653783114, 294.76515120682853, 290.85743152480006, 297.0552585032022, 293.9776640163791, 287.3052648958733, 289.4673474943393, 292.78547479388567, 291.86483343561156, 287.93739418887435, 296.4129177533607, 317.17634421674654, 308.24383684537634, 302.96896558404364, 302.2105083420935, 302.89266337473896, 308.5027460298762, 296.57977908602334, 292.9260597489654, 283.0140114641399, 269.04915613659796, 268.46556005329506, 280.2950965106663, 272.61706247434233, 274.5999552795023, 275.90226334063607, 280.84737708382954, 275.14718883654393, 274.22105215397727, 273.8364651955357, 274.6888818919266, 279.2825686299615, 278.0015513250793, 295.567337936757, 293.56896360252307, 293.91018664751573, 290.48515347409, 288.8135115286942, 288.6694665846105, 287.18694477382166, 285.65028985944593, 282.8986265669389, 283.2930330083322, 283.7743144124955, 285.6178474700298, 285.5603845656622, 288.4167499413677, 304.33826696732405, 296.94628086762464, 296.28142138162525, 292.68621363563375, 289.08442002702384, 288.73318648696284, 283.34399302099894, 279.88075972260856, 279.15412084579043, 277.0210092392737, 297.038039119538, 300.8029943739856, 306.8076998126825, 295.6939709730931, 299.81572358621736, 303.0537556676278, 317.0228290175678, 317.8319709731644, 317.40220648179786, 311.5128043960526, 313.7251283249433, 304.95009786499304, 300.09487074232516, 296.2841894542245, 301.0772943809683, 305.8330426138506, 300.8703765381476, 298.56999696665434, 296.01891371604677, 293.46157478336374, 293.3043399072504, 301.8542230693617, 298.15776333811965, 302.0920870560459, 307.0407276444052, 304.9317168085014, 308.76932196866244, 303.65904467859, 300.91184841842016, 296.064260841889, 297.9940210080797, 294.0913336678483, 296.08056180920954, 295.16060657486145, 300.82987009502943, 309.4245824552416, 302.75167973478074, 297.23799462320454, 301.02514091028877, 296.45371762645976, 294.34095852157117, 289.4029337591736, 289.3324085360513, 294.305806427485, 289.37337086411645, 286.93079555574445, 285.78156513007684, 283.32621200663203, 283.8651073974975, 284.6612867951681, 304.121587436833, 282.7734161186503, 293.69340771661734, 283.4537329348988, 270.49187817903305, 276.71210752438634, 273.3191691486028, 280.73958782767613, 301.4374692733621, 301.6859495057497, 302.5930899955368, 303.3992311497308, 296.98936479100155, 284.3074910824768, 283.5053784011267, 295.33315351209274, 294.80272243971683, 291.9814935814125, 287.79102524602405, 287.41426677756056, 280.9370980466791, 279.73681092098263, 279.2047483198423, 283.669121632919, 276.82465299546425, 277.125637340436, 276.2390103746574, 267.5670064130276, 267.4568602104582, 267.3106112873003, 269.28408689523394, 270.40900528138536, 262.5026484099014, 267.1044658750895, 274.36614885953554, 269.6345592170072, 270.49684255813145, 268.7803711764508, 272.4028135998068, 276.10864930373543, 282.5801331272524, 288.8158752305751, 300.25285247076755, 296.1310557774672, 290.1289343794973, 294.6571643603274, 290.2817217102203, 296.9687066139082, 292.1304687146058, 286.23725783573695, 282.69109416366905, 288.4182063477058, 280.0886317216682, 279.3888511083922, 280.23876199464553, 278.8365760543252, 298.9744666121509, 300.1186151888944, 288.05184731984764, 296.8419084178205, 297.9100853899762, 294.4465642753188, 285.88828359949656, 286.9955654831291, 289.10251399612355, 282.7727796425394, 287.45388938226756, 282.8772014700134, 282.75784706667173, 289.7316031946415, 279.1374139170314, 277.8672064808536, 301.231241023973, 298.44909158251426, 298.96305382866325, 280.4736649001445, 281.5375802032447, 280.5770407817215, 288.47002009524635, 292.99618746994395, 291.1224501837064, 290.07425416300185, 290.7040790415758, 291.83610754888184, 284.9301633927245, 287.6715040222457, 289.8918204128331, 291.7856978698962, 292.72800741089605, 299.04948266901846, 295.45296551650154, 291.5510558504926, 292.1232900567714, 294.1750822183485, 293.4888019017226, 291.1097166946248, 296.81842839556333, 297.9140918687258, 297.4105127949913, 293.2522600594193, 295.0829399519692, 294.67415189348594, 285.4173252398243, 294.26413536265545, 294.5485244605375, 288.78715731789157, 289.72758074746207, 289.53718662128665, 291.1869615364227, 297.89842381041814, 297.0358599196662, 292.8772102741978, 295.9798460467292, 292.5794382424272, 297.6588736427881, 289.7967602970889, 285.86284220024095, 280.57033904378216, 274.9480346898326, 277.4502242832746, 284.0254254543752, 279.783901324997, 281.87923206569604, 283.82135423610754, 293.635046565873, 287.8370294268991, 287.51016479877444, 310.2201235208753, 327.577617090325, 322.86499391358336, 336.56225112266986, 333.7656757954443, 333.21712105126664, 325.5144272311187, 339.965714334553, 345.5827317885655, 347.1478779297378, 374.0180003868062, 369.2910148152163, 375.3535157470534, 366.96568860124387, 369.6210333369037, 372.75847605222566, 367.3393191598278, 379.0165545361431, 372.6663176677131, 371.3515798081229, 377.38458877758785, 357.60786716807013, 339.7548032774133, 342.8594017833991, 357.08394513723897, 352.9969437121216, 346.12731009159234, 342.2609384220053, 343.1863506253594, 342.17668981169345, 350.7789467162652, 347.50256848807294]
actual_tci = [255.550003, 249.399994, 252.75, 261.25, 253.399994, 263.399994, 296.600006, 318.649994, 324.049988, 329.450012, 316.5, 323.5, 315.299988, 314.350006, 303.049988, 308.049988, 311.799988, 308.049988, 309.600006, 308.649994, 310.0, 304.75, 302.950012, 294.600006, 290.899994, 291.100006, 320.75, 329.950012, 327.549988, 334.850006, 340.600006, 343.549988, 339.850006, 337.450012, 333.950012, 339.049988, 332.799988, 324.950012, 318.649994, 325.149994, 322.950012, 324.25, 320.5, 317.899994, 320.75, 318.299988, 314.899994, 311.700012, 313.700012, 303.950012, 304.25, 301.600006, 298.350006, 297.649994, 290.5, 280.350006, 273.25, 276.600006, 282.299988, 284.649994, 285.950012, 276.950012, 273.649994, 272.549988, 272.649994, 274.649994, 287.700012, 281.600006, 283.450012, 285.850006, 279.950012, 282.25, 281.549988, 286.549988, 284.450012, 282.850006, 290.600006, 302.399994, 290.850006, 297.25, 294.799988, 294.5, 293.850006, 287.850006, 283.899994, 272.75, 260.850006, 262.75, 263.100006, 266.799988, 269.450012, 271.399994, 266.100006, 266.5, 268.950012, 263.0, 270.700012, 273.0, 272.399994, 280.049988, 283.649994, 284.0, 280.299988, 280.5, 282.350006, 278.649994, 276.25, 275.950012, 278.350006, 278.600006, 279.25, 276.649994, 281.100006, 291.5, 289.950012, 286.649994, 280.850006, 282.649994, 277.899994, 275.799988, 274.649994, 267.299988, 271.549988, 286.799988, 291.799988, 287.299988, 290.049988, 293.5, 298.299988, 308.0, 308.100006, 307.549988, 303.75, 299.649994, 297.700012, 295.350006, 293.0, 293.350006, 296.700012, 294.0, 291.799988, 288.299988, 285.399994, 287.149994, 288.649994, 291.25, 297.399994, 300.649994, 295.299988, 299.600006, 294.350006, 292.649994, 289.899994, 291.600006, 287.200012, 290.399994, 289.899994, 294.549988, 297.850006, 293.549988, 291.899994, 289.899994, 288.450012, 287.25, 283.549988, 284.0, 279.399994, 284.700012, 278.200012, 280.0, 277.950012, 274.850006, 280.799988, 276.899994, 278.399994, 281.600006, 272.5, 263.450012, 263.700012, 266.399994, 274.149994, 282.649994, 293.700012, 293.350006, 291.5, 283.5, 278.700012, 280.299988, 286.549988, 286.299988, 282.5, 279.399994, 276.600006, 272.899994, 271.0, 275.75, 276.350006, 269.950012, 270.899994, 261.950012, 256.399994, 260.450012, 261.799988, 266.450012, 264.299988, 256.549988, 258.850006, 262.350006, 264.399994, 259.649994, 263.850006, 263.5, 266.25, 276.200012, 285.100006, 291.450012, 288.350006, 284.450012, 284.200012, 284.850006, 287.399994, 279.0, 279.100006, 278.149994, 275.399994, 272.5, 272.899994, 274.149994, 272.899994, 291.100006, 276.700012, 284.5, 289.049988, 292.600006, 280.5, 279.649994, 279.600006, 279.899994, 277.0, 276.399994, 275.850006, 275.200012, 273.549988, 273.049988, 272.600006, 294.899994, 288.100006, 278.200012, 273.299988, 274.950012, 275.850006, 280.600006, 287.100006, 285.549988, 285.549988, 284.950012, 286.200012, 279.700012, 277.899994, 280.149994, 282.899994, 287.350006, 291.700012, 284.350006, 285.75, 286.0, 287.799988, 285.200012, 286.700012, 292.049988, 287.600006, 289.75, 284.350006, 286.299988, 281.700012, 279.799988, 289.100006, 284.549988, 284.899994, 284.649994, 284.600006, 286.200012, 292.299988, 290.549988, 285.399994, 289.700012, 286.899994, 283.25, 281.149994, 277.600006, 270.200012, 267.549988, 271.549988, 279.450012, 273.75, 274.350006, 277.649994, 282.549988, 281.5, 280.149994, 299.700012, 316.450012, 310.899994, 324.0, 318.5, 321.850006, 308.200012, 336.25, 337.799988, 330.350006, 356.450012, 361.700012, 359.950012, 356.049988, 364.75, 359.450012, 358.049988, 365.75, 363.299988, 365.350006, 349.950012, 338.200012, 327.549988, 335.850006, 348.450012, 341.75, 339.450012, 334.850006, 334.950012, 334.75, 335.149994, 340.600006]
accuracy(actual_tci,predicted_tci)

As per predictions : 
sell at 261.8304869363691
buy at 256.9456902428551
sell at 266.9600691741243
buy at 259.8589950301497
sell at 337.676634754911
buy at 314.07817082931615
sell at 319.51965941142487
buy at 318.0804144155263
sell at 320.6695922077887
buy at 307.1637724241067
sell at 311.8473704315179
buy at 297.8589748500989
sell at 337.19939680208233
buy at 337.06198630245296
sell at 347.7545950114628
buy at 347.2261975777094
sell at 353.21161406892236
buy at 343.7173765158493
sell at 345.90225201522196
buy at 329.9369378943246
sell at 332.36805704983794
buy at 326.1775403166747
sell at 330.3088837277445
buy at 319.1059199297131
sell at 321.3851057349003
buy at 315.0687671197393
sell at 320.2232950868897
buy at 284.4148436208684
sell at 294.9174513683656
buy at 278.55433643645085
sell at 294.76515120682853
buy at 290.85743152480006
sell at 297.0552585032022
buy at 287.3052648958733
sell at 292.78547479388567
buy at 287.93739418887435
sell at 317.17634421674654
buy at 302.21050834209

In [ ]:
#ARIMAX
predicted_ongc_with_co = [218.0132968664133, 217.28636523817738, 215.21745300979947, 217.89618059012025, 215.86195751800352, 218.01325469009737, 216.12282901091834, 214.01019688696624, 213.45584904822095, 216.69920119605217, 211.81066469126313, 211.61487550947032, 207.72817256986303, 206.6994186259044, 207.64053724432063, 207.01352008357532, 206.02439732897022, 205.22278577195084, 204.17853714686026, 202.31340457395947, 205.0002351030675, 200.7507532642784, 197.18034994179243, 194.09835971147294, 194.41885312304555, 195.85131096073462, 196.31182966042613, 195.63286135181716, 197.05979224012918, 200.35523770273292, 200.3664965094901, 199.1030015995641, 196.9143601354158, 199.6603090135019, 198.73300857131443, 202.90230812004663, 197.94508006869142, 196.95738240659074, 197.96650927232778, 199.08821775497864, 200.38121739247813, 205.70507111410052, 202.4714104512297, 200.95412742264324, 202.05651211536025, 204.06152898313735, 203.21213353777307, 204.63878298758743, 209.85216092369552, 207.81061071466877, 205.2041421890881, 205.28136430705604, 205.854428226771, 206.46494672412229, 203.80645730044347, 205.21557180704775, 203.69652295449526, 199.67891317122303, 201.10928188955506, 198.68949952218878, 199.65631205901417, 200.7356257781614, 199.06547814406815, 199.01027367276976, 199.67435320007215, 198.41796314564607, 198.76737306507533, 195.76012336642015, 196.41288970319545, 196.1366788185863, 198.22111705077702, 200.09060885931544, 202.3455655695034, 203.6142339755939, 202.27738980990705, 199.86115973539097, 202.0353152833233, 201.86882591123396, 202.25346684903246, 201.00861215633608, 206.78773930343698, 207.14621260962338, 206.6539358127763, 210.29781664695986, 207.25193286637187, 205.86896964643574, 206.4322415788535, 214.83328212233096, 214.71922061909368, 213.8158639041805, 215.64157495973058, 213.34487981562484, 213.74751791518446, 213.65134422968708, 215.44043133978062, 212.83605134931554, 213.11337925025492, 212.22164068305483, 211.13764454926547, 211.8471961305226, 215.15819057646854, 213.867653762371, 217.1239263347979, 215.5168596728413, 214.76968745697388, 219.6864109354384, 219.6498128675127, 222.7174987243427, 227.74436577026387, 233.9929393040137, 234.5543673142579, 236.14804628349657, 234.19871395848259, 236.1493610036369, 244.6521272552131, 245.0111574888109, 239.5340097116025, 236.1277422784206, 238.16896677279942, 232.68592142830494, 228.73757391080426, 223.74210346925747, 224.38517284442304, 225.95591291327742, 226.07808155078862, 226.6061388904526, 229.28674195299482, 229.8865545884796, 229.45073995298617, 231.29869877906833, 228.69611098314226, 230.519567589178, 227.61588983612393, 228.50004446147585, 230.34018949766198, 225.8821925098606, 223.12153997842933, 223.50984647988162, 227.69648900251866, 226.06662231777057, 230.38616891059695, 232.51477348576083, 230.34379206494532, 230.46603484782952, 225.58577738609006, 232.5191238783101, 235.8631942524803, 235.59307361202679, 239.66938428580204, 243.79501264861756, 241.37156988086986, 241.90680188924995, 243.33325164008772, 241.81445727297648, 245.20243396705234, 243.58416947353385, 248.84215323149192, 246.7233855035147, 248.15591171826594, 248.18937022226316, 250.10391667773945, 250.6392905247336, 251.99634969180312, 249.7021893148496, 247.98879705707475, 248.36048828212364, 246.75063905377306, 245.37764826932414, 255.7876401306671, 258.3245887826806, 262.9758754598125, 261.752239579239, 258.411798642774, 259.0246468618399, 253.20374209794267, 251.88902534603733, 245.5153827077396, 241.14376917312427, 236.67322607424, 240.37237756747186, 239.73723805063895, 235.9610137582821, 241.62364341963365, 235.25999793545225, 236.75259796105865, 238.24597757485844, 234.41541824518447, 238.511039689372, 238.56495142537585, 238.42991766659395, 242.117877824784, 243.7694053355051, 242.28688815782687, 240.67473332075565, 240.0078068968333, 237.40924169214662, 236.93630894176908, 230.787381646474, 230.7489296030045, 232.94420466232862, 232.79486939198205, 233.0870481272728, 231.7355110518015, 230.2274905461736, 228.69357389494684, 228.4726171069778, 228.22133520068007, 229.4472638127085, 233.8646999458756, 233.0897415244499, 230.74481247447613, 232.21919346061645, 230.2579018570218, 232.31740889773226, 230.39749336955413, 227.8476174888487, 229.16238341636176, 228.3324132227898, 230.49321008046957, 233.42441719649412, 239.06065520429598, 237.56529877047012, 238.5133337079812, 235.7284946824746, 235.37226510693904, 239.30148851330415, 241.57265741755725, 239.4888883023359, 238.53163792229248, 239.9190165263612, 237.98217466482117, 237.56815739407466, 239.25925529451547, 239.32116640915223, 237.05989549670244, 239.7361971818696, 242.00361462873127, 243.78438709042416, 240.39580250806142, 244.65752310429963, 248.0201579686277, 247.9517615732392, 247.0585893901984, 249.21261500759556, 239.88535392773485, 249.25017806507083, 249.37985781131545, 249.6344343132704, 248.85127597030942, 239.94982819958852, 221.83322054938276, 230.24811369394513, 234.3634609870761, 232.428735628882, 234.42725792824416, 235.19073280600708, 228.10755506572013, 226.8674392552532, 226.8173575226284, 226.17394186664387, 229.5633777621664, 229.78230572572386, 229.91121143175184, 230.42261063809644, 227.36441530999053, 226.0552810683423, 222.95024321675675, 222.6653359039397, 221.3205468140921, 219.8773818791147, 216.6875373795625, 218.50312182855606, 216.57663291622293, 219.35618103981858, 219.5808429703966, 219.1167573806576, 222.34372740237637, 219.45807473065804, 224.57500193665902, 220.89878653378474, 219.90001684989824, 220.42788068575538, 221.43914507098174, 223.04544600240854, 219.71035351933057, 217.09231690499666, 216.8836428308436, 213.08074884852687, 214.9499048190081, 223.0192255496899, 224.25317687284456, 217.8203420173026, 215.76296365942915, 217.31644123903686, 220.54162456970982, 223.32323736946444, 220.86084431790852, 226.56511504697608, 224.56818234801887, 226.86784019626566, 227.60074995485434, 227.19017257159783, 229.55493790345105, 227.8336642803098, 230.01866354773995, 228.09527430054214, 229.06975201282035, 225.34390618828445, 225.65516755799888, 223.22523392025812, 221.89395298453923, 227.7339862273629, 227.39427279740642, 231.2820209769793, 236.354168469284, 237.60040094967394, 236.8835298328366, 239.87226388388422, 243.84333383766733, 242.98477335902942, 241.21823996273832, 238.5254307177874, 234.92211676836962, 235.20903770521767, 234.0634057488146, 231.81915850177734, 232.87527918341732, 234.55197539437665, 234.6580166593444, 236.2434490173497, 237.96000290002405, 243.0988313254901]
actual_ongc_with_co = [174.649994, 173.699997, 174.149994, 175.75, 175.75, 178.649994, 177.0, 174.0, 174.0, 175.75, 171.449997, 172.050003, 170.550003, 169.199997, 168.5, 169.100006, 169.5, 167.149994, 167.149994, 166.050003, 169.100006, 164.800003, 160.449997, 158.100006, 159.949997, 158.100006, 157.850006, 157.300003, 159.100006, 159.899994, 162.800003, 161.699997, 159.600006, 162.399994, 160.25, 162.850006, 158.899994, 158.350006, 160.050003, 161.699997, 163.0, 165.850006, 164.350006, 163.25, 163.300003, 164.0, 162.899994, 164.75, 169.399994, 167.399994, 165.399994, 165.699997, 166.399994, 165.899994, 164.0, 165.699997, 163.699997, 159.649994, 161.699997, 160.949997, 160.600006, 160.899994, 157.149994, 159.449997, 159.699997, 159.449997, 160.600006, 157.25, 158.399994, 156.949997, 159.949997, 161.800003, 162.800003, 162.300003, 162.300003, 161.850006, 163.050003, 162.300003, 161.050003, 159.350006, 166.75, 165.199997, 165.899994, 167.699997, 165.899994, 164.449997, 164.800003, 171.399994, 170.199997, 171.550003, 170.949997, 170.649994, 172.100006, 171.199997, 173.850006, 170.699997, 170.350006, 169.149994, 169.699997, 169.649994, 172.050003, 171.199997, 174.449997, 172.050003, 171.699997, 176.0, 176.850006, 177.350006, 184.0, 186.600006, 191.100006, 191.0, 188.0, 191.850006, 199.25, 193.800003, 192.550003, 189.600006, 191.100006, 184.199997, 182.0, 177.550003, 179.149994, 177.600006, 180.0, 180.899994, 181.300003, 181.699997, 181.0, 184.050003, 181.25, 182.449997, 180.649994, 179.649994, 181.600006, 178.399994, 175.850006, 177.050003, 180.25, 178.899994, 183.300003, 183.899994, 184.350006, 183.0, 183.050003, 185.199997, 188.0, 188.0, 193.399994, 193.699997, 192.600006, 193.0, 195.199997, 192.350006, 196.850006, 193.449997, 199.5, 198.449997, 197.399994, 197.0, 197.899994, 197.899994, 200.75, 196.600006, 197.199997, 195.699997, 194.100006, 193.699997, 200.699997, 207.350006, 210.850006, 208.5, 205.149994, 204.800003, 203.449997, 194.899994, 192.699997, 188.75, 185.449997, 190.0, 188.699997, 187.350006, 191.0, 185.75, 188.300003, 186.899994, 185.300003, 187.0, 190.149994, 186.100006, 189.800003, 190.350006, 188.5, 188.300003, 189.699997, 185.550003, 185.399994, 181.5, 180.149994, 179.899994, 183.850006, 183.600006, 180.600006, 180.300003, 177.149994, 176.850006, 174.100006, 175.550003, 178.899994, 177.399994, 178.899994, 179.600006, 177.800003, 179.949997, 177.5, 175.399994, 177.449997, 176.949997, 178.399994, 180.199997, 182.149994, 181.75, 182.5, 181.5, 180.800003, 182.100006, 183.949997, 182.350006, 181.050003, 182.649994, 179.449997, 180.800003, 180.699997, 180.550003, 180.050003, 181.100006, 180.25, 183.899994, 183.100006, 182.850006, 188.25, 187.5, 187.25, 189.0, 186.199997, 188.600006, 185.050003, 185.75, 185.199997, 175.399994, 167.699997, 175.449997, 175.600006, 175.550003, 174.0, 177.899994, 173.199997, 171.100006, 171.199997, 170.300003, 172.649994, 173.699997, 173.300003, 171.399994, 170.0, 168.5, 165.449997, 164.600006, 164.800003, 162.75, 159.699997, 159.550003, 156.649994, 158.300003, 156.25, 153.5, 158.399994, 155.5, 157.699997, 156.0, 156.399994, 155.449997, 157.399994, 157.949997, 158.100006, 158.949997, 154.100006, 153.800003, 156.149994, 160.300003, 161.75, 157.949997, 156.550003, 157.199997, 159.050003, 162.550003, 161.300003, 164.100006, 165.699997, 167.199997, 166.399994, 166.800003, 169.25, 167.550003, 172.5, 168.899994, 169.25, 166.050003, 166.25, 164.449997, 163.050003, 168.699997, 170.199997, 171.899994, 174.75, 176.050003, 174.600006, 177.0, 177.449997, 180.050003, 177.300003, 174.350006, 173.699997, 174.5, 171.899994, 168.649994, 167.800003, 168.5, 172.550003, 171.699997, 173.899994, 177.0]
accuracy(actual_ongc_with_co,predicted_ongc_with_co)

In [6]:
#ARIMAX
predicted_tci_with_co = [192.27256528048554, 191.30654963525743, 189.04302957578864, 198.6982214749674, 191.0918491086358, 198.8195587491614, 242.25809366628368, 256.7086881964203, 265.54638997720167, 269.7738651198431, 268.1219649719545, 268.95054454634834, 262.8697559181323, 258.4233209464607, 247.2480523403566, 252.72342450047046, 252.541467162309, 253.04332603991338, 254.95021577921875, 254.00535822394914, 250.60816163310557, 249.0515365271636, 247.01970577657505, 242.93402768712133, 246.70071368559024, 232.43164467923106, 267.4263117567716, 270.1671435513557, 269.0811658835839, 279.37119770971026, 280.1390752190649, 285.9870429720204, 283.99811049517075, 282.34015754667087, 278.7017970252632, 276.77244849063186, 278.8359925567135, 270.6228021565847, 263.10676903916305, 265.38798142407416, 264.82572417764743, 264.0821394864844, 261.4105735041078, 259.01115828153956, 262.1199610804026, 257.77840905927155, 257.4208848451933, 250.0916092420485, 252.3446541408295, 250.30148946256872, 246.87118395969992, 252.09224672967804, 243.35874507731847, 238.4275817939327, 234.22731709593972, 224.9768970123577, 216.8430603020078, 221.44829783283978, 224.63655244988217, 228.38833287488148, 225.0536321881566, 220.80921205068853, 218.11393286009286, 212.3846744743107, 211.2985053193314, 214.51299512872538, 227.94462144114163, 228.86354183780668, 225.00201584604133, 230.35110871648425, 227.48793096886203, 220.48125972922787, 221.77810755928715, 224.82116502165525, 223.90434662368887, 221.23080328099365, 229.33368602399935, 249.61345974808296, 239.97412913649202, 233.99257315884296, 233.5340856125174, 233.99137593018077, 239.95978017948642, 226.87427033384984, 222.95049426308356, 213.36618560879543, 198.22061635634327, 197.50517469482918, 209.010843059889, 201.89491746578733, 203.92123315533615, 205.77971669258727, 211.4625284595702, 205.21325878379025, 204.92628849657308, 204.5323248707631, 204.5165290049453, 208.8343040171252, 208.16457929731484, 225.72203486263885, 222.90558172748854, 223.25122850066703, 219.51369690606361, 218.14421523226974, 217.78753875122186, 215.70798075639368, 214.8634751802001, 211.65920302495329, 211.11564803475318, 211.5212200442271, 213.29831215903727, 213.62917157229708, 216.17568311109164, 231.2686620851706, 222.63007885722675, 221.3193216815505, 218.33299576155014, 214.21079750413156, 214.12233863622612, 208.58853523834517, 206.08843357964327, 205.72528314454962, 203.8438233779002, 223.19006349222974, 226.6187104465232, 232.51795176376527, 220.72827233300814, 224.34462169905498, 227.32258689551685, 241.6836804179627, 242.69212998666876, 243.03161614716112, 237.0167528297664, 238.4773780708456, 230.3222466154965, 225.4266681980373, 222.5281611512895, 227.05807515207113, 231.426134168178, 226.28267143369624, 224.25566537095153, 222.15705082787906, 219.77977317580417, 219.5021479488001, 228.31692159176288, 223.96353717893777, 227.64093695982842, 232.37360996178677, 230.2369497594973, 232.99270662234676, 228.0225798415733, 225.3228911251955, 220.30277763709194, 222.5502640511424, 218.82244428559432, 219.9300482101101, 218.84282701019674, 224.88902219601619, 233.10143250840628, 225.34170497592254, 219.49980980467922, 222.63561993071937, 218.5663723222326, 215.83301596868955, 211.1137907986962, 211.17360554075162, 216.10625797245922, 211.79728352363315, 209.22606871872654, 207.3837169032559, 204.75913845827324, 204.79784795958687, 205.70036415412244, 225.8749810504404, 204.5744215759447, 214.22807973661128, 204.41127284830281, 192.38955133072255, 198.79925843665967, 196.6491732204363, 204.24731098586494, 226.34730907891185, 225.88616439914108, 226.96413795782473, 226.88638069690273, 220.19336201207813, 206.56934190439335, 205.57412811740812, 217.71992198831845, 216.06817677119227, 212.9655745900264, 208.4653059073595, 208.47657801313744, 202.26210005126939, 202.26385373614517, 200.96014935868698, 205.4434825126175, 199.782213472941, 200.22719637409494, 198.61851592786536, 190.57126089897213, 191.14205328696164, 190.61872990000074, 192.07824354902516, 192.55605186024587, 184.75336201072224, 188.00151734843337, 194.17728649980705, 189.96078354302006, 190.38975208869994, 188.78462606407282, 192.50263085946062, 196.56546755834086, 203.75826146966665, 209.74253832635787, 221.20097282198878, 217.01562772373495, 212.0408786530138, 215.62489013048236, 209.747104212855, 215.1094767227155, 210.29421657588432, 204.00452326776934, 200.9346189911139, 206.67624820891746, 197.03790400711455, 195.97666815367944, 196.26294918619004, 194.37386499886543, 214.8854988431653, 215.42054058969143, 203.47898959412544, 212.30319844819246, 212.96623738241982, 210.3472158438986, 201.23327619988225, 201.16155228106692, 202.24417868558925, 197.30025640903756, 199.97162837219582, 195.92509774237215, 195.39114172698828, 202.3018010275149, 190.84819813745287, 190.09701706375705, 213.10616228499384, 209.94517579001695, 209.78720225533112, 191.36161312970705, 192.3707222996578, 192.01217384787083, 203.021710904052, 208.5428044916631, 206.25304393770944, 204.44227256616284, 206.11876389668004, 208.41341774701579, 202.54017104551502, 204.7643145173962, 207.60661258858346, 208.09657306007006, 208.94892539234075, 214.84791627863513, 211.2733412901623, 207.04451110348953, 207.41796474548437, 210.25460217097043, 209.04628301636114, 207.26635561091587, 212.31988143738567, 213.31051754092545, 210.904894239027, 206.8974930022796, 206.85424687374802, 204.1850616253945, 194.2897837489577, 203.01461775861998, 202.95175063989802, 197.56546731753963, 198.06313063075032, 198.58381598844704, 199.8600638829393, 206.9086576652404, 205.6938211594719, 203.5493696827425, 207.9864180591349, 203.4905715398658, 211.26719852360327, 203.27669025424098, 198.89556845751827, 192.48081858242995, 188.05947633520077, 190.71881088945247, 196.45376835939635, 192.03970307532916, 193.8750272111359, 196.79300940289784, 205.20645278434256, 200.5286205175164, 201.4431727191435, 222.73460894343202, 240.37658567973028, 234.4236289745079, 248.2704000540341, 247.40702182907654, 246.4172509529741, 238.1319345733852, 252.48323394594888, 256.7627108364135, 259.902543505583, 286.43121154895437, 281.06451663698726, 287.1910181863817, 277.1545391255788, 279.6101411115137, 282.162379441593, 277.03769422858807, 287.1405034628425, 279.856770075824, 278.87430899760375, 284.44239557278877, 264.72593217504675, 248.02373246565497, 251.45065484645255, 265.69280105331393, 260.8007285079522, 252.44333649917837, 248.43190391116656, 251.00428148936817, 249.16685520378132, 256.9835996023619, 252.74922502469576]
actual_tci_with_co = [255.550003, 249.399994, 252.75, 261.25, 253.399994, 263.399994, 296.600006, 318.649994, 324.049988, 329.450012, 316.5, 323.5, 315.299988, 314.350006, 303.049988, 308.049988, 311.799988, 308.049988, 309.600006, 308.649994, 310.0, 304.75, 302.950012, 294.600006, 290.899994, 291.100006, 320.75, 329.950012, 327.549988, 334.850006, 340.600006, 343.549988, 339.850006, 337.450012, 333.950012, 339.049988, 332.799988, 324.950012, 318.649994, 325.149994, 322.950012, 324.25, 320.5, 317.899994, 320.75, 318.299988, 314.899994, 311.700012, 313.700012, 303.950012, 304.25, 301.600006, 298.350006, 297.649994, 290.5, 280.350006, 273.25, 276.600006, 282.299988, 284.649994, 285.950012, 276.950012, 273.649994, 272.549988, 272.649994, 274.649994, 287.700012, 281.600006, 283.450012, 285.850006, 279.950012, 282.25, 281.549988, 286.549988, 284.450012, 282.850006, 290.600006, 302.399994, 290.850006, 297.25, 294.799988, 294.5, 293.850006, 287.850006, 283.899994, 272.75, 260.850006, 262.75, 263.100006, 266.799988, 269.450012, 271.399994, 266.100006, 266.5, 268.950012, 263.0, 270.700012, 273.0, 272.399994, 280.049988, 283.649994, 284.0, 280.299988, 280.5, 282.350006, 278.649994, 276.25, 275.950012, 278.350006, 278.600006, 279.25, 276.649994, 281.100006, 291.5, 289.950012, 286.649994, 280.850006, 282.649994, 277.899994, 275.799988, 274.649994, 267.299988, 271.549988, 286.799988, 291.799988, 287.299988, 290.049988, 293.5, 298.299988, 308.0, 308.100006, 307.549988, 303.75, 299.649994, 297.700012, 295.350006, 293.0, 293.350006, 296.700012, 294.0, 291.799988, 288.299988, 285.399994, 287.149994, 288.649994, 291.25, 297.399994, 300.649994, 295.299988, 299.600006, 294.350006, 292.649994, 289.899994, 291.600006, 287.200012, 290.399994, 289.899994, 294.549988, 297.850006, 293.549988, 291.899994, 289.899994, 288.450012, 287.25, 283.549988, 284.0, 279.399994, 284.700012, 278.200012, 280.0, 277.950012, 274.850006, 280.799988, 276.899994, 278.399994, 281.600006, 272.5, 263.450012, 263.700012, 266.399994, 274.149994, 282.649994, 293.700012, 293.350006, 291.5, 283.5, 278.700012, 280.299988, 286.549988, 286.299988, 282.5, 279.399994, 276.600006, 272.899994, 271.0, 275.75, 276.350006, 269.950012, 270.899994, 261.950012, 256.399994, 260.450012, 261.799988, 266.450012, 264.299988, 256.549988, 258.850006, 262.350006, 264.399994, 259.649994, 263.850006, 263.5, 266.25, 276.200012, 285.100006, 291.450012, 288.350006, 284.450012, 284.200012, 284.850006, 287.399994, 279.0, 279.100006, 278.149994, 275.399994, 272.5, 272.899994, 274.149994, 272.899994, 291.100006, 276.700012, 284.5, 289.049988, 292.600006, 280.5, 279.649994, 279.600006, 279.899994, 277.0, 276.399994, 275.850006, 275.200012, 273.549988, 273.049988, 272.600006, 294.899994, 288.100006, 278.200012, 273.299988, 274.950012, 275.850006, 280.600006, 287.100006, 285.549988, 285.549988, 284.950012, 286.200012, 279.700012, 277.899994, 280.149994, 282.899994, 287.350006, 291.700012, 284.350006, 285.75, 286.0, 287.799988, 285.200012, 286.700012, 292.049988, 287.600006, 289.75, 284.350006, 286.299988, 281.700012, 279.799988, 289.100006, 284.549988, 284.899994, 284.649994, 284.600006, 286.200012, 292.299988, 290.549988, 285.399994, 289.700012, 286.899994, 283.25, 281.149994, 277.600006, 270.200012, 267.549988, 271.549988, 279.450012, 273.75, 274.350006, 277.649994, 282.549988, 281.5, 280.149994, 299.700012, 316.450012, 310.899994, 324.0, 318.5, 321.850006, 308.200012, 336.25, 337.799988, 330.350006, 356.450012, 361.700012, 359.950012, 356.049988, 364.75, 359.450012, 358.049988, 365.75, 363.299988, 365.350006, 349.950012, 338.200012, 327.549988, 335.850006, 348.450012, 341.75, 339.450012, 334.850006, 334.950012, 334.75, 335.149994, 340.600006]
accuracy(actual_tci_with_co,predicted_tci_with_co)

As per predictions : 
sell at 192.27256528048554
buy at 189.04302957578864
sell at 198.6982214749674
buy at 191.0918491086358
sell at 269.7738651198431
buy at 268.1219649719545
sell at 268.95054454634834
buy at 247.2480523403566
sell at 252.72342450047046
buy at 252.541467162309
sell at 254.95021577921875
buy at 242.93402768712133
sell at 246.70071368559024
buy at 232.43164467923106
sell at 270.1671435513557
buy at 269.0811658835839
sell at 285.9870429720204
buy at 276.77244849063186
sell at 278.8359925567135
buy at 263.10676903916305
sell at 265.38798142407416
buy at 259.01115828153956
sell at 262.1199610804026
buy at 250.0916092420485
sell at 252.3446541408295
buy at 246.87118395969992
sell at 252.09224672967804
buy at 216.8430603020078
sell at 228.38833287488148
buy at 211.2985053193314
sell at 228.86354183780668
buy at 225.00201584604133
sell at 230.35110871648425
buy at 220.48125972922787
sell at 224.82116502165525
buy at 221.23080328099365
sell at 249.61345974808296
buy at 233.53

In [ ]:
#ARIMAX
predicted_ioc_with_co = [216.0083101743239, 218.85665851707967, 220.30426349737064, 214.57826824715443, 214.62212362735642, 216.49930713837716, 218.3900329030202, 210.7439555842061, 210.57269008114167, 213.5807599316458, 215.6301989902061, 215.07782670719052, 214.22326277767792, 213.30112748350302, 213.72688845164646, 213.33470248485708, 214.2303758968303, 207.59359939703143, 208.74168659750342, 205.7382755696184, 208.84527930953078, 207.50583002590463, 202.0855109493508, 194.08517970203422, 194.3112436591967, 200.70321146236915, 195.91018335251601, 196.4204040909707, 196.32457358167835, 195.9666453309742, 194.90880597404055, 194.12719233478464, 192.62707052154917, 195.58777193286073, 194.87253647626034, 201.23846128380706, 194.50004737392925, 190.16773619179037, 188.9590795203623, 190.26809126236083, 191.0632680825811, 190.81306949851955, 192.9627865214307, 193.04638082881303, 190.99778976963307, 190.02759531198672, 187.91997567978984, 186.58706054720244, 188.70322508103004, 189.45932246730047, 187.48792005101097, 197.28595825362868, 211.59607938595474, 216.04638272475876, 210.02148977001866, 212.81831216258104, 209.23415111976783, 208.59987889477708, 212.61869096543404, 215.53938196692934, 216.35033876394982, 218.82176488985266, 212.87040202172636, 215.91927733652574, 218.29965013326313, 218.10147776977954, 222.1777441629617, 222.12088301060135, 227.8007360210757, 230.96464690131086, 228.0025444314491, 220.06164654407758, 219.62478686637053, 218.48339133211647, 217.9865896411111, 218.6683767368752, 218.83310420102055, 220.01726815187072, 211.98465451554785, 213.31970294561756, 212.58190456321927, 211.51758154635502, 212.1232700438503, 210.46596684589872, 211.60939240751503, 204.7871096620338, 203.31328902512786, 198.85417106199145, 203.0548571996472, 195.6779958435518, 203.60039770576964, 205.39840397163903, 208.34722059753886, 205.1506353591816, 210.02846456921577, 211.86001579779617, 210.27834992220414, 213.75162338846883, 211.98032513208977, 212.26561072533116, 211.6544640727264, 212.96110559915797, 209.93088524282894, 208.86475690147074, 207.54530234478568, 208.99474186957633, 211.3458401604846, 220.56428981802912, 212.40591689251391, 212.343917471214, 213.10138622423221, 212.65352431835242, 211.88752462913953, 210.48949601117005, 208.8883522253044, 203.02695700357071, 201.64804381669194, 201.33930904672673, 200.55333768343837, 199.75114832644442, 198.24039790934893, 199.78831790922027, 199.7865755852265, 201.03580803164613, 199.81241401168847, 201.11218457433807, 200.72355870240176, 202.20578070504848, 200.88267110518404, 199.567992819192, 198.56570573873626, 200.2305225055544, 200.43942269357217, 198.32409377867526, 197.01865483570123, 197.73091210489565, 198.6623521665183, 199.17339998291385, 204.7137641023902, 205.65178096180784, 201.70107390085502, 204.24353321899537, 207.1068425224325, 210.40991782990955, 206.05603648250985, 208.3488604141633, 209.04452302452847, 209.67047287220464, 209.2611167383227, 209.11116972234635, 205.35784526256793, 201.6306008528369, 198.85896422273635, 199.38585917388605, 197.267883106034, 197.7280258668693, 197.13936734551692, 196.36833009558677, 198.4825936424346, 199.39639200997271, 199.7059947357995, 198.98321972465283, 200.8469805983733, 201.57998794479045, 197.31356423641773, 193.9287713234212, 194.03198668113356, 195.9435277918958, 195.0698638199276, 198.9695088773343, 198.94163715015554, 200.3711347976208, 203.30472154726078, 210.99658438116057, 214.4302538524837, 211.6159985368002, 209.51452870809987, 207.64303976303916, 206.1768995294267, 207.58864003218366, 198.20506911642013, 192.7945514012489, 192.95272597265398, 192.7769526800463, 192.452659931751, 192.36513863959465, 189.14862417571874, 188.02419573650195, 187.33283005810028, 185.40612955482726, 188.00983991540897, 193.44513973140351, 193.5980040339058, 193.1090208119112, 194.56337843036874, 193.93075707803052, 198.65930130109882, 194.6826313367394, 194.2303782971871, 195.98666491573232, 203.1106584287444, 209.11424966391792, 203.6943168145861, 195.4997443504535, 189.79291490946287, 182.21550397076868, 178.400151190134, 177.98660047491524, 174.94946625349377, 172.94682053415426, 170.39849370442323, 177.7882391832677, 180.2756635646548, 176.3195975281705, 178.12793010751227, 175.5691416980406, 177.2449241385345, 178.9550204585491, 183.38065328747302, 182.46590592778023, 173.15470759606012, 169.89623530031776, 170.56545985328728, 169.22967535911852, 171.09421836058175, 169.99035137337904, 163.3565043282914, 164.2263606406815, 167.16380336354547, 167.48277603438936, 166.8560431278646, 164.41103744913482, 166.75847363973622, 166.73845359212044, 165.62944340760902, 166.42675572527895, 165.66688584024268, 167.94797954165728, 169.7214378227631, 170.34104620897176, 171.82921941531117, 176.74853455961244, 175.62664671681785, 172.2392685299368, 169.30296407150541, 170.40085919711134, 168.89758904229586, 169.57113054471068, 165.18660156563703, 159.9104771908478, 158.4057192916348, 167.22533901975066, 179.32994468117676, 176.2271564540476, 175.38825644633266, 178.20113303619326, 177.67435193909216, 177.1882861703769, 177.3060588761503, 178.5164459067359, 179.11300205017693, 178.50906714044004, 179.68286649328462, 179.28852592022258, 178.6696220356455, 176.68792089362546, 172.97547228671306, 179.81633988314468, 173.9511634738459, 171.44586693717798, 173.42059252789147, 175.63027146935985, 171.5795215462744, 173.79700513751328, 162.3057382652732, 161.81562656095474, 161.5518627208196, 159.65094665612236, 160.08849332404571, 160.7627376045973, 161.66456479380298, 164.09461299692134, 163.35676585823774, 160.93744386904996, 157.76879858927424, 164.81813738801276, 162.14305044412146, 163.170953125038, 168.5599166095442, 172.32140705741153, 168.71338633810865, 166.33751235384398, 169.20753152197835, 168.6583993907281, 166.0699793756877, 165.13366538505693, 169.40649887176284, 172.59575422346228, 169.919036681484, 173.0000819004946, 174.8346521149827, 173.62036097484685, 172.86859387849717, 172.82788877871025, 171.51642844231083, 170.99608169848375, 170.87671048911733, 166.70911874529548, 165.04564393538828, 164.0780056755686, 164.343367346845, 163.95471998004206, 163.70188763615084, 158.21426574727985, 158.3585359767913, 161.1694950385645, 160.8299662832211, 161.33661307760417, 159.8008702233899, 160.1323258166462, 160.4658956446532, 158.05459664485608, 156.78286919950978, 157.64408542169548, 158.16710618128332, 156.20702829225814, 156.99965367541853, 151.88242815626876, 159.6846789018361, 162.08572860860295, 158.79369493444597, 159.2274308396597]
actual_ioc_with_co = [212.125, 215.274994, 219.475006, 213.024994, 209.625, 209.774994, 214.725006, 206.925003, 207.399994, 211.024994, 212.300003, 213.675003, 209.375, 210.024994, 211.824997, 209.875, 210.350006, 203.524994, 202.300003, 203.600006, 205.850006, 204.125, 196.75, 191.699997, 195.199997, 193.574997, 193.425003, 192.600006, 193.449997, 190.925003, 192.850006, 190.800003, 191.125, 191.725006, 193.75, 196.300003, 190.475006, 186.550003, 186.774994, 186.074997, 187.574997, 187.274994, 190.449997, 189.074997, 187.399994, 186.824997, 184.449997, 183.899994, 183.824997, 185.175003, 185.050003, 193.524994, 209.475006, 215.600006, 206.625, 207.024994, 205.149994, 205.899994, 209.725006, 211.149994, 213.149994, 213.675003, 206.600006, 211.600006, 213.425003, 214.100006, 220.350006, 217.25, 226.425003, 227.350006, 224.149994, 214.524994, 215.925003, 214.125, 216.100006, 214.574997, 214.524994, 217.449997, 207.975006, 209.850006, 207.824997, 207.824997, 209.149994, 206.975006, 206.625, 204.199997, 199.5, 199.824997, 197.824997, 198.699997, 200.25, 203.274994, 204.375, 203.375, 207.725006, 207.074997, 205.425003, 209.050003, 207.449997, 207.300003, 206.774994, 206.949997, 205.800003, 204.824997, 202.225006, 203.850006, 208.625, 218.25, 207.475006, 208.125, 207.75, 208.399994, 207.475006, 206.625, 205.324997, 200.100006, 198.5, 197.300003, 197.425003, 198.425003, 193.475006, 194.75, 196.5, 197.574997, 196.475006, 198.074997, 198.0, 197.375, 197.699997, 196.300003, 195.300003, 196.850006, 196.75, 193.925003, 195.274994, 195.25, 194.375, 195.75, 202.425003, 202.199997, 197.550003, 201.100006, 205.0, 206.574997, 204.800003, 204.949997, 205.050003, 205.475006, 205.125, 204.975006, 200.75, 196.0, 194.274994, 194.975006, 193.199997, 193.149994, 193.649994, 191.925003, 194.274994, 196.199997, 194.774994, 194.449997, 194.524994, 195.425003, 192.074997, 192.024994, 189.024994, 192.975006, 191.25, 194.699997, 195.149994, 196.149994, 199.824997, 208.125, 208.800003, 208.949997, 204.324997, 204.5, 200.649994, 202.375, 192.800003, 189.324997, 189.524994, 188.675003, 188.399994, 187.524994, 184.824997, 183.300003, 184.25, 181.600006, 184.199997, 188.350006, 188.725006, 189.649994, 190.75, 189.625, 191.100006, 189.699997, 191.050003, 192.75, 200.324997, 203.925003, 197.524994, 192.050003, 184.75, 177.899994, 173.850006, 172.449997, 170.100006, 168.850006, 167.199997, 174.850006, 176.600006, 170.449997, 174.149994, 171.149994, 173.550003, 174.25, 179.800003, 179.300003, 167.350006, 168.25, 165.350006, 165.0, 167.199997, 164.949997, 157.949997, 160.100006, 161.699997, 163.050003, 160.600006, 161.149994, 162.899994, 162.300003, 161.149994, 161.699997, 161.5, 164.300003, 166.550003, 167.199997, 167.949997, 169.100006, 168.399994, 167.199997, 165.949997, 164.75, 163.699997, 165.149994, 160.5, 153.5, 155.899994, 163.449997, 170.449997, 172.449997, 170.800003, 174.100006, 172.350006, 173.800003, 173.949997, 174.899994, 174.800003, 175.0, 173.100006, 174.149994, 175.100006, 171.550003, 167.75, 173.899994, 168.600006, 165.399994, 170.25, 170.0, 167.350006, 166.800003, 155.649994, 154.350006, 156.0, 153.699997, 155.300003, 155.25, 155.0, 156.699997, 157.300003, 155.25, 154.550003, 156.850006, 159.100006, 157.5, 164.449997, 167.199997, 165.800003, 161.050003, 162.5, 164.949997, 163.100006, 158.800003, 165.399994, 166.449997, 164.350006, 168.0, 168.699997, 168.550003, 168.149994, 166.649994, 165.899994, 165.399994, 166.350006, 160.649994, 160.699997, 158.5, 159.550003, 158.649994, 157.699997, 153.399994, 154.25, 156.100006, 155.550003, 156.5, 154.25, 155.550003, 155.0, 152.449997, 152.800003, 153.600006, 153.399994, 151.050003, 150.699997, 149.100006, 155.25, 156.550003, 152.600006, 153.550003]
accuracy(actual_ioc_with_co,predicted_ioc_with_co)

In [11]:
#SVR - Support vector regression
#TCI stock using OHL
actual_tci = [255.550003, 249.399994, 252.75, 261.25, 253.399994, 263.399994, 296.600006, 318.649994, 324.049988, 329.450012, 316.5, 323.5, 315.299988, 314.350006, 303.049988, 308.049988, 311.799988, 308.049988, 309.600006, 308.649994, 310.0, 304.75, 302.950012, 294.600006, 290.899994, 291.100006, 320.75, 329.950012, 327.549988, 334.850006, 340.600006, 343.549988, 339.850006, 337.450012, 333.950012, 339.049988, 332.799988, 324.950012, 318.649994, 325.149994, 322.950012, 324.25, 320.5, 317.899994, 320.75, 318.299988, 314.899994, 311.700012, 313.700012, 303.950012, 304.25, 301.600006, 298.350006, 297.649994, 290.5, 280.350006, 273.25, 276.600006, 282.299988, 284.649994, 285.950012, 276.950012, 273.649994, 272.549988, 272.649994, 274.649994, 287.700012, 281.600006, 283.450012, 285.850006, 279.950012, 282.25, 281.549988, 286.549988, 284.450012, 282.850006, 290.600006, 302.399994, 290.850006, 297.25, 294.799988, 294.5, 293.850006, 287.850006, 283.899994, 272.75, 260.850006, 262.75, 263.100006, 266.799988, 269.450012, 271.399994, 266.100006, 266.5, 268.950012, 263.0, 270.700012, 273.0, 272.399994, 280.049988, 283.649994, 284.0, 280.299988, 280.5, 282.350006, 278.649994, 276.25, 275.950012, 278.350006, 278.600006, 279.25, 276.649994, 281.100006, 291.5, 289.950012, 286.649994, 280.850006, 282.649994, 277.899994, 275.799988, 274.649994, 267.299988, 271.549988, 286.799988, 291.799988, 287.299988, 290.049988, 293.5, 298.299988, 308.0, 308.100006, 307.549988, 303.75, 299.649994, 297.700012, 295.350006, 293.0, 293.350006, 296.700012, 294.0, 291.799988, 288.299988, 285.399994, 287.149994, 288.649994, 291.25, 297.399994, 300.649994, 295.299988, 299.600006, 294.350006, 292.649994, 289.899994, 291.600006, 287.200012, 290.399994, 289.899994, 294.549988, 297.850006, 293.549988, 291.899994, 289.899994, 288.450012, 287.25, 283.549988, 284.0, 279.399994, 284.700012, 278.200012, 280.0, 277.950012, 274.850006, 280.799988, 276.899994, 278.399994, 281.600006, 272.5, 263.450012, 263.700012, 266.399994, 274.149994, 282.649994, 293.700012, 293.350006, 291.5, 283.5, 278.700012, 280.299988, 286.549988, 286.299988, 282.5, 279.399994, 276.600006, 272.899994, 271.0, 275.75, 276.350006, 269.950012, 270.899994, 261.950012, 256.399994, 260.450012, 261.799988, 266.450012, 264.299988, 256.549988, 258.850006, 262.350006, 264.399994, 259.649994, 263.850006, 263.5, 266.25, 276.200012, 285.100006, 291.450012, 288.350006, 284.450012, 284.200012, 284.850006, 287.399994, 279.0, 279.100006, 278.149994, 275.399994, 272.5, 272.899994, 274.149994, 272.899994, 291.100006, 276.700012, 284.5, 289.049988, 292.600006, 280.5, 279.649994, 279.600006, 279.899994, 277.0, 276.399994, 275.850006, 275.200012, 273.549988, 273.049988, 272.600006, 294.899994, 288.100006, 278.200012, 273.299988, 274.950012, 275.850006, 280.600006, 287.100006, 285.549988, 285.549988, 284.950012, 286.200012, 279.700012, 277.899994, 280.149994, 282.899994, 287.350006, 291.700012, 284.350006, 285.75, 286.0, 287.799988, 285.200012, 286.700012, 292.049988, 287.600006, 289.75, 284.350006, 286.299988, 281.700012, 279.799988, 289.100006, 284.549988, 284.899994, 284.649994, 284.600006, 286.200012, 292.299988, 290.549988, 285.399994, 289.700012, 286.899994, 283.25, 281.149994, 277.600006, 270.200012, 267.549988, 271.549988, 279.450012, 273.75, 274.350006, 277.649994, 282.549988, 281.5, 280.149994, 299.700012, 316.450012, 310.899994, 324.0, 318.5, 321.850006, 308.200012, 336.25, 337.799988, 330.350006, 356.450012, 361.700012, 359.950012, 356.049988, 364.75, 359.450012, 358.049988, 365.75, 363.299988, 365.350006, 349.950012, 338.200012, 327.549988, 335.850006, 348.450012, 341.75, 339.450012, 334.850006, 334.950012, 334.75, 335.149994, 340.600006]
#pred_tci = [255.550003, 249.399994, 252.75, 261.25, 253.399994, 263.399994, 296.600006, 318.649994, 324.049988, 329.450012, 316.5, 323.5, 315.299988, 314.350006, 303.049988, 308.049988, 311.799988, 308.049988, 309.600006, 308.649994, 310.0, 304.75, 302.950012, 294.600006, 290.899994, 291.100006, 320.75, 329.950012, 327.549988, 334.850006, 340.600006, 343.549988, 339.850006, 337.450012, 333.950012, 339.049988, 332.799988, 324.950012, 318.649994, 325.149994, 322.950012, 324.25, 320.5, 317.899994, 320.75, 318.299988, 314.899994, 311.700012, 313.700012, 303.950012, 304.25, 301.600006, 298.350006, 297.649994, 290.5, 280.350006, 273.25, 276.600006, 282.299988, 284.649994, 285.950012, 276.950012, 273.649994, 272.549988, 272.649994, 274.649994, 287.700012, 281.600006, 283.450012, 285.850006, 279.950012, 282.25, 281.549988, 286.549988, 284.450012, 282.850006, 290.600006, 302.399994, 290.850006, 297.25, 294.799988, 294.5, 293.850006, 287.850006, 283.899994, 272.75, 260.850006, 262.75, 263.100006, 266.799988, 269.450012, 271.399994, 266.100006, 266.5, 268.950012, 263.0, 270.700012, 273.0, 272.399994, 280.049988, 283.649994, 284.0, 280.299988, 280.5, 282.350006, 278.649994, 276.25, 275.950012, 278.350006, 278.600006, 279.25, 276.649994, 281.100006, 291.5, 289.950012, 286.649994, 280.850006, 282.649994, 277.899994, 275.799988, 274.649994, 267.299988, 271.549988, 286.799988, 291.799988, 287.299988, 290.049988, 293.5, 298.299988, 308.0, 308.100006, 307.549988, 303.75, 299.649994, 297.700012, 295.350006, 293.0, 293.350006, 296.700012, 294.0, 291.799988, 288.299988, 285.399994, 287.149994, 288.649994, 291.25, 297.399994, 300.649994, 295.299988, 299.600006, 294.350006, 292.649994, 289.899994, 291.600006, 287.200012, 290.399994, 289.899994, 294.549988, 297.850006, 293.549988, 291.899994, 289.899994, 288.450012, 287.25, 283.549988, 284.0, 279.399994, 284.700012, 278.200012, 280.0, 277.950012, 274.850006, 280.799988, 276.899994, 278.399994, 281.600006, 272.5, 263.450012, 263.700012, 266.399994, 274.149994, 282.649994, 293.700012, 293.350006, 291.5, 283.5, 278.700012, 280.299988, 286.549988, 286.299988, 282.5, 279.399994, 276.600006, 272.899994, 271.0, 275.75, 276.350006, 269.950012, 270.899994, 261.950012, 256.399994, 260.450012, 261.799988, 266.450012, 264.299988, 256.549988, 258.850006, 262.350006, 264.399994, 259.649994, 263.850006, 263.5, 266.25, 276.200012, 285.100006, 291.450012, 288.350006, 284.450012, 284.200012, 284.850006, 287.399994, 279.0, 279.100006, 278.149994, 275.399994, 272.5, 272.899994, 274.149994, 272.899994, 291.100006, 276.700012, 284.5, 289.049988, 292.600006, 280.5, 279.649994, 279.600006, 279.899994, 277.0, 276.399994, 275.850006, 275.200012, 273.549988, 273.049988, 272.600006, 294.899994, 288.100006, 278.200012, 273.299988, 274.950012, 275.850006, 280.600006, 287.100006, 285.549988, 285.549988, 284.950012, 286.200012, 279.700012, 277.899994, 280.149994, 282.899994, 287.350006, 291.700012, 284.350006, 285.75, 286.0, 287.799988, 285.200012, 286.700012, 292.049988, 287.600006, 289.75, 284.350006, 286.299988, 281.700012, 279.799988, 289.100006, 284.549988, 284.899994, 284.649994, 284.600006, 286.200012, 292.299988, 290.549988, 285.399994, 289.700012, 286.899994, 283.25, 281.149994, 277.600006, 270.200012, 267.549988, 271.549988, 279.450012, 273.75, 274.350006, 277.649994, 282.549988, 281.5, 280.149994, 299.700012, 316.450012, 310.899994, 324.0, 318.5, 321.850006, 308.200012, 336.25, 337.799988, 330.350006, 356.450012, 361.700012, 359.950012, 356.049988, 364.75, 359.450012, 358.049988, 365.75, 363.299988, 365.350006, 349.950012, 338.200012, 327.549988, 335.850006, 348.450012, 341.75, 339.450012, 334.850006, 334.950012, 334.75, 335.149994, 340.600006]

pred_tci = [251.35708261, 248.3875605 , 249.66612056, 260.55351536, 249.48268605, 254.00662646, 235.9382772 , 298.49568002, 325.20949992, 327.6108588 , 306.25694124, 326.65159744, 318.8601075 , 315.59686688, 304.21804476, 299.32777779, 308.29387256, 299.39367452, 308.88424271, 298.80052156, 301.79877563, 307.01407431, 304.86971815, 286.13543382, 279.23607126, 295.59511284, 232.00079367, 322.63200089, 321.32081307, 325.62652852, 319.30061514, 354.09089376, 317.85253454, 327.6691573 , 328.07967604, 325.45753948, 339.42088717, 319.9414624 , 319.11260335, 325.23215476, 323.1224903 , 321.78703283, 318.02681409, 317.41098977, 322.9586951 , 317.69224293, 317.74452217, 309.5116897 , 313.77188201, 303.99911605, 307.24990916, 291.40603681, 299.10253757, 295.76962668, 294.34328378, 279.93411133, 270.83681632, 267.59537391, 281.14523561, 288.84294832, 284.51348555, 280.25062784, 273.69123392, 268.95084313, 271.09543181, 274.71391088, 279.59145022, 282.26834561, 284.05994783, 281.71946069, 280.52226477, 282.06845101, 279.93749807, 285.65516792, 288.98959729, 282.59115447, 281.87078786, 292.04375157, 294.34728262, 288.73894417, 290.56317018, 295.77425673, 299.90191095, 292.55749289, 288.81171634, 273.5773283 , 258.95097989, 260.68948206, 265.02159976, 265.787821  , 266.66008175, 268.98536441, 274.18783619, 267.56392211, 266.30776752, 251.32321744, 266.89715901, 272.71900693, 273.07324508, 287.72382934, 287.48264756, 284.02169839, 283.8414944 , 282.846711  , 280.52880039, 279.13380072, 274.87277368, 276.90542062, 275.40076772, 276.27224343, 276.42246398, 278.05938579, 281.76199428, 292.33935617, 299.39559704, 296.01095492, 280.89828211, 283.13992279, 282.52313223, 275.49259068, 274.18798003, 272.87130112, 270.74603639, 288.63291618, 295.76654302, 295.79473022, 287.53879297, 300.42486646, 289.68970821, 308.15368177, 307.85954062, 303.91109583, 303.7178572 , 299.70819943, 296.46012752, 288.75920717, 284.33753973, 286.96913878, 294.4430469 , 291.95892069, 286.34714016, 297.52908638, 285.53082184, 290.92712364, 279.12939937, 297.92292501, 291.88541665, 296.31271757, 295.84749679, 297.43628141, 296.64036302, 288.07244715, 290.07606075, 302.96296612, 293.18199286, 299.5217122 , 300.82071355, 293.73563877, 298.49752019, 295.22964803, 295.43870634, 299.25594802, 286.0020808 , 292.83538102, 281.97872779, 282.98786676, 281.62596032, 279.54660104, 281.74695926, 277.88839132, 276.95399123, 276.5481334 , 274.80865157, 258.44533676, 273.16823777, 283.73129797, 250.23825983, 260.75177824, 246.45306533, 266.2926427 , 274.36422874, 292.210493  , 299.78338872, 292.56141335, 291.8020786 , 278.50738921, 278.78679862, 275.15183125, 286.26343186, 287.29603553, 279.707519  , 282.5771167 , 276.37103386, 271.21908022, 272.15389127, 268.01951811, 269.69983393, 271.1350501 , 269.60241445, 257.39406342, 257.54349649, 256.80500358, 261.35322308, 263.94256806, 259.60935998, 254.14489728, 260.12731368, 266.28099549, 262.595389  , 252.50641812, 262.05495714, 263.26346885, 269.01178929, 272.16630556, 278.80442082, 290.25575589, 293.175642  , 284.50175148, 284.96244702, 286.18734072, 295.31984906, 279.47312343, 281.85301099, 275.04315753, 276.22799878, 272.26928739, 273.08566242, 275.63014642, 273.00763779, 286.55488062, 276.21715029, 277.32236075, 286.5512485 , 303.52567197, 282.17008519, 276.35858633, 280.11825512, 281.87586411, 276.58502972, 277.31762477, 274.81006589, 273.42327722, 276.83290365, 274.37424396, 272.01819825, 283.47381063, 287.75174987, 280.35680413, 271.64725947, 274.849749  , 275.23747312, 278.8018462 , 288.86634081, 287.95715125, 284.77639015, 286.88543288, 289.62189066, 277.4194687 , 279.7802446 , 282.57326548, 283.47437668, 284.56658116, 296.11217967, 286.27322352, 281.10139152, 289.92017891, 287.07377636, 290.39208722, 284.92345396, 290.58150305, 301.24446606, 299.13789182, 280.2002583 , 284.22567204, 275.60928515, 273.91920017, 284.32827931, 283.26765871, 282.9302052 , 279.02405076, 284.16200457, 287.84832125, 293.16030943, 295.63091632, 285.49546549, 301.05532304, 289.35616075, 282.44760805, 280.72745326, 281.46812071, 270.78122201, 267.17106209, 271.61610066, 276.98296008, 271.72258784, 277.77339151, 278.94540309, 284.2154758 , 282.30693043, 281.14142064, 279.2375865 , 280.52863915, 305.4586475 , 277.69582963, 312.47113582, 325.90523608, 308.3895304 , 237.56418127, 315.15061601, 314.26256871, 225.4361334 , 362.59723467, 367.02806979, 349.05309986, 319.1479533 , 360.06524363, 359.60707748, 371.01991618, 351.48256554, 338.33967211, 342.77003557, 301.59083535, 322.48130987, 338.73664911, 292.39354179, 305.51121735, 312.36176556, 318.33969137, 331.86966894, 314.17611931, 343.54567744, 323.13165012]
accuracy(actual_tci,pred_tci)

As per predictions : 
sell at 251.35708261
buy at 248.3875605
sell at 260.55351536
buy at 249.48268605
sell at 254.00662646
buy at 235.9382772
sell at 327.6108588
buy at 306.25694124
sell at 326.65159744
buy at 299.32777779
sell at 308.29387256
buy at 299.39367452
sell at 308.88424271
buy at 298.80052156
sell at 307.01407431
buy at 279.23607126
sell at 295.59511284
buy at 232.00079367
sell at 322.63200089
buy at 321.32081307
sell at 325.62652852
buy at 319.30061514
sell at 354.09089376
buy at 317.85253454
sell at 328.07967604
buy at 325.45753948
sell at 339.42088717
buy at 319.11260335
sell at 325.23215476
buy at 317.41098977
sell at 322.9586951
buy at 317.69224293
sell at 317.74452217
buy at 309.5116897
sell at 313.77188201
buy at 303.99911605
sell at 307.24990916
buy at 291.40603681
sell at 299.10253757
buy at 267.59537391
sell at 288.84294832
buy at 268.95084313
sell at 284.05994783
buy at 280.52226477
sell at 282.06845101
buy at 279.93749807
sell at 288.98959729
buy at 281.87078786

In [10]:
#SVR - Support vector regression
#ONGC stock using OHL

actual_ongc = [174.649994, 173.699997, 174.149994, 175.75, 175.75, 178.649994, 177.0, 174.0, 174.0, 175.75, 171.449997, 172.050003, 170.550003, 169.199997, 168.5, 169.100006, 169.5, 167.149994, 167.149994, 166.050003, 169.100006, 164.800003, 160.449997, 158.100006, 159.949997, 158.100006, 157.850006, 157.300003, 159.100006, 159.899994, 162.800003, 161.699997, 159.600006, 162.399994, 160.25, 162.850006, 158.899994, 158.350006, 160.050003, 161.699997, 163.0, 165.850006, 164.350006, 163.25, 163.300003, 164.0, 162.899994, 164.75, 169.399994, 167.399994, 165.399994, 165.699997, 166.399994, 165.899994, 164.0, 165.699997, 163.699997, 159.649994, 161.699997, 160.949997, 160.600006, 160.899994, 157.149994, 159.449997, 159.699997, 159.449997, 160.600006, 157.25, 158.399994, 156.949997, 159.949997, 161.800003, 162.800003, 162.300003, 162.300003, 161.850006, 163.050003, 162.300003, 161.050003, 159.350006, 166.75, 165.199997, 165.899994, 167.699997, 165.899994, 164.449997, 164.800003, 171.399994, 170.199997, 171.550003, 170.949997, 170.649994, 172.100006, 171.199997, 173.850006, 170.699997, 170.350006, 169.149994, 169.699997, 169.649994, 172.050003, 171.199997, 174.449997, 172.050003, 171.699997, 176.0, 176.850006, 177.350006, 184.0, 186.600006, 191.100006, 191.0, 188.0, 191.850006, 199.25, 193.800003, 192.550003, 189.600006, 191.100006, 184.199997, 182.0, 177.550003, 179.149994, 177.600006, 180.0, 180.899994, 181.300003, 181.699997, 181.0, 184.050003, 181.25, 182.449997, 180.649994, 179.649994, 181.600006, 178.399994, 175.850006, 177.050003, 180.25, 178.899994, 183.300003, 183.899994, 184.350006, 183.0, 183.050003, 185.199997, 188.0, 188.0, 193.399994, 193.699997, 192.600006, 193.0, 195.199997, 192.350006, 196.850006, 193.449997, 199.5, 198.449997, 197.399994, 197.0, 197.899994, 197.899994, 200.75, 196.600006, 197.199997, 195.699997, 194.100006, 193.699997, 200.699997, 207.350006, 210.850006, 208.5, 205.149994, 204.800003, 203.449997, 194.899994, 192.699997, 188.75, 185.449997, 190.0, 188.699997, 187.350006, 191.0, 185.75, 188.300003, 186.899994, 185.300003, 187.0, 190.149994, 186.100006, 189.800003, 190.350006, 188.5, 188.300003, 189.699997, 185.550003, 185.399994, 181.5, 180.149994, 179.899994, 183.850006, 183.600006, 180.600006, 180.300003, 177.149994, 176.850006, 174.100006, 175.550003, 178.899994, 177.399994, 178.899994, 179.600006, 177.800003, 179.949997, 177.5, 175.399994, 177.449997, 176.949997, 178.399994, 180.199997, 182.149994, 181.75, 182.5, 181.5, 180.800003, 182.100006, 183.949997, 182.350006, 181.050003, 182.649994, 179.449997, 180.800003, 180.699997, 180.550003, 180.050003, 181.100006, 180.25, 183.899994, 183.100006, 182.850006, 188.25, 187.5, 187.25, 189.0, 186.199997, 188.600006, 185.050003, 185.75, 185.199997, 175.399994, 167.699997, 175.449997, 175.600006, 175.550003, 174.0, 177.899994, 173.199997, 171.100006, 171.199997, 170.300003, 172.649994, 173.699997, 173.300003, 171.399994, 170.0, 168.5, 165.449997, 164.600006, 164.800003, 162.75, 159.699997, 159.550003, 156.649994, 158.300003, 156.25, 153.5, 158.399994, 155.5, 157.699997, 156.0, 156.399994, 155.449997, 157.399994, 157.949997, 158.100006, 158.949997, 154.100006, 153.800003, 156.149994, 160.300003, 161.75, 157.949997, 156.550003, 157.199997, 159.050003, 162.550003, 161.300003, 164.100006, 165.699997, 167.199997, 166.399994, 166.800003, 169.25, 167.550003, 172.5, 168.899994, 169.25, 166.050003, 166.25, 164.449997, 163.050003, 168.699997, 170.199997, 171.899994, 174.75, 176.050003, 174.600006, 177.0, 177.449997, 180.050003, 177.300003, 174.350006, 173.699997, 174.5, 171.899994, 168.649994, 167.800003, 168.5, 172.550003, 171.699997, 173.899994, 177.0]
pred_ongc = [174.39447128, 175.22784166, 174.83504511, 174.62170905,  175.07457488, 178.62363025, 177.31024883, 173.791019  ,  174.01001262, 175.80475252, 172.36710424, 173.42634865,  170.90091081, 168.72234968, 169.2602145 , 168.93744744,  169.4648766 , 167.47834683, 166.70050537, 164.77498808,  169.50255763, 164.7307099 , 161.50389427, 159.4018121 ,  158.3314012 , 158.83179983, 157.79650779, 158.64218717,  157.45251373, 160.18650607, 163.45926976, 162.68984835,  159.35261079, 163.89913079, 161.7717868 , 169.52992738,  159.2602405 , 157.71328761, 159.01862236, 159.50862136,  160.82935649, 171.60217474, 167.59131092, 173.50835396,  172.90655681, 178.10409064, 175.07722779, 162.18554175,  166.53458671, 167.24743972, 163.72813319, 167.2340303 ,  166.48024557, 166.51270173, 168.95715639, 165.30907907,  168.8747034 , 159.15575643, 160.62877018, 160.38403358,  160.0621357 , 159.68373592, 161.14711329, 160.53446755,  159.80664428, 160.24499142, 159.58986396, 157.84824724,  160.30144786, 156.66809583, 160.40641125, 159.81958716,  168.92517594, 169.43216396, 173.0552504 , 160.00484565,  164.50579885, 170.56189268, 164.12930751, 158.64811617,  183.2040928 , 165.56321083, 163.91894409, 164.52231363,  163.72144354, 177.68363296, 162.98731252, 172.44689164,  170.27282828, 172.39832691, 172.49870737, 171.84144835,  171.0998457 , 171.91189984, 173.30054587, 171.65700856,  170.4668538 , 169.64585912, 168.54676273, 169.55962503,  170.63600392, 172.83435889, 173.14441131, 172.05575294,  171.61559967, 175.91334261, 176.88896201, 179.19548213,  191.58057507, 190.22076339, 191.42205792, 191.37843715,  189.05069073, 190.45395057, 196.77850166, 213.10795485,  192.12360642, 185.61985942, 192.20779714, 184.04656385,  182.62125423, 178.97272322, 177.54539906, 177.91383999,  179.9477506 , 179.31801414, 181.49252993, 181.69725853,  181.42674332, 183.09008319, 180.55524761, 181.543799  ,  180.32012704, 180.55840979, 182.09303076, 178.87363894,  177.52721959, 174.72210587, 181.37117461, 178.35747698,  180.72832012, 183.71412481, 183.79855592, 183.48692951,  177.93503831, 184.68993924, 189.86807518, 189.21702797,  192.0919491 , 195.35690205, 192.36488445, 192.94828482,  194.23481035, 193.03179259, 198.04565785, 194.93064537,  199.72612004, 201.25011781, 198.55077079, 196.48002485,  198.31776668, 198.75812374, 199.46341704, 201.81741133,  195.43198878, 196.15971451, 193.48542408, 194.6585805 ,  205.86681999, 204.080963  , 209.90371876, 208.32403045,  205.3527472 , 205.5774829 , 199.36791145, 199.07916957,  194.10700476, 189.05061837, 183.36989186, 189.12470243,  189.54179791, 188.42721145, 193.10669326, 187.0813989 ,  187.42714342, 187.65578067, 185.2071762 , 187.6357069 ,  189.26440933, 186.59417612, 189.70088092, 190.12694123,  185.5319446 , 188.2069973 , 188.59677084, 184.66585323,  185.72234068, 181.72423906, 180.73554597, 181.05978046,  182.89819836, 182.82824354, 181.17033598, 179.6554012 ,  178.0064681 , 177.10729308, 175.46528709, 176.14166058,  177.82398296, 179.66252548, 177.83422923, 178.63560839,  177.57517849, 179.94154138, 177.99737519, 176.64133369,  176.75802912, 177.59319858, 178.40900491, 179.43412  ,  183.13928371, 182.5322404 , 182.33887964, 180.66525421,  180.33464078, 182.11499733, 184.06663475, 182.33539496,  179.46991676, 182.33523167, 179.97437675, 180.0667218 ,  181.40523776, 182.87769526, 179.91822995, 180.7537907 ,  182.57820852, 182.31203924, 184.41407008, 183.83176602,  187.87353086, 187.46825009, 186.4775799 , 184.50509655,  200.48834281, 187.2232072 , 187.33089671, 187.59086449,  186.70637162, 182.46806894, 215.47015755, 192.33088776,  178.26931583, 175.28294348, 177.32690461, 175.86829306,  172.04583844, 172.10820051, 170.21145982, 171.53641029,  170.96569989, 172.79698959, 172.41393894, 173.81034278,  169.99933084, 168.0168133 , 166.12433231, 167.04117755,  167.24422578, 175.77789027, 159.48110412, 159.82445273,  155.70167152, 158.57530672, 156.50628916, 157.30804255,  156.06348955, 155.73857251, 157.17684771, 157.50953184,  156.13745595, 159.69326659, 157.21653001, 157.96203136,  159.49008834, 155.44224242, 160.33058247, 154.20515858,  154.28524506, 177.97533787, 161.28634269, 158.83473378,  156.05564174, 159.71207164, 162.27072226, 161.27571015,  162.89597249, 168.06305157, 167.40788764, 165.18819683,  165.99662362, 166.75194912, 169.02663923, 167.05557281,  172.13978895, 171.14938776, 168.99781876, 165.95943127,  167.0275633 , 168.08952914, 174.10022383, 170.45197919,  168.16663274, 170.15724499, 174.32829646, 175.07973278,  176.30675371, 177.46459777, 175.75173147, 181.0472442 ,  177.85443  , 174.08968063, 171.98042711, 172.70437226,  172.65334172, 169.20662865, 168.34140031, 169.91597539,  171.49287166, 172.08577931, 173.67278182, 177.99775791]
accuracy(actual_ongc,pred_ongc)

As per predictions : 
sell at 175.22784166
buy at 174.62170905
sell at 178.62363025
buy at 173.791019
sell at 175.80475252
buy at 172.36710424
sell at 173.42634865
buy at 168.72234968
sell at 169.2602145
buy at 168.93744744
sell at 169.4648766
buy at 164.77498808
sell at 169.50255763
buy at 158.3314012
sell at 158.83179983
buy at 157.79650779
sell at 158.64218717
buy at 157.45251373
sell at 163.45926976
buy at 159.35261079
sell at 163.89913079
buy at 161.7717868
sell at 169.52992738
buy at 157.71328761
sell at 171.60217474
buy at 167.59131092
sell at 173.50835396
buy at 172.90655681
sell at 178.10409064
buy at 162.18554175
sell at 167.24743972
buy at 163.72813319
sell at 167.2340303
buy at 166.48024557
sell at 168.95715639
buy at 165.30907907
sell at 168.8747034
buy at 159.15575643
sell at 160.62877018
buy at 159.68373592
sell at 161.14711329
buy at 159.80664428
sell at 160.24499142
buy at 157.84824724
sell at 160.30144786
buy at 156.66809583
sell at 160.40641125
buy at 159.81958716
se

In [12]:
#SVR - Support vector regression
#ioc stock using OHL

actual_ioc = [212.125, 215.274994, 219.475006, 213.024994, 209.625, 209.774994, 214.725006, 206.925003, 207.399994, 211.024994, 212.300003, 213.675003, 209.375, 210.024994, 211.824997, 209.875, 210.350006, 203.524994, 202.300003, 203.600006, 205.850006, 204.125, 196.75, 191.699997, 195.199997, 193.574997, 193.425003, 192.600006, 193.449997, 190.925003, 192.850006, 190.800003, 191.125, 191.725006, 193.75, 196.300003, 190.475006, 186.550003, 186.774994, 186.074997, 187.574997, 187.274994, 190.449997, 189.074997, 187.399994, 186.824997, 184.449997, 183.899994, 183.824997, 185.175003, 185.050003, 193.524994, 209.475006, 215.600006, 206.625, 207.024994, 205.149994, 205.899994, 209.725006, 211.149994, 213.149994, 213.675003, 206.600006, 211.600006, 213.425003, 214.100006, 220.350006, 217.25, 226.425003, 227.350006, 224.149994, 214.524994, 215.925003, 214.125, 216.100006, 214.574997, 214.524994, 217.449997, 207.975006, 209.850006, 207.824997, 207.824997, 209.149994, 206.975006, 206.625, 204.199997, 199.5, 199.824997, 197.824997, 198.699997, 200.25, 203.274994, 204.375, 203.375, 207.725006, 207.074997, 205.425003, 209.050003, 207.449997, 207.300003, 206.774994, 206.949997, 205.800003, 204.824997, 202.225006, 203.850006, 208.625, 218.25, 207.475006, 208.125, 207.75, 208.399994, 207.475006, 206.625, 205.324997, 200.100006, 198.5, 197.300003, 197.425003, 198.425003, 193.475006, 194.75, 196.5, 197.574997, 196.475006, 198.074997, 198.0, 197.375, 197.699997, 196.300003, 195.300003, 196.850006, 196.75, 193.925003, 195.274994, 195.25, 194.375, 195.75, 202.425003, 202.199997, 197.550003, 201.100006, 205.0, 206.574997, 204.800003, 204.949997, 205.050003, 205.475006, 205.125, 204.975006, 200.75, 196.0, 194.274994, 194.975006, 193.199997, 193.149994, 193.649994, 191.925003, 194.274994, 196.199997, 194.774994, 194.449997, 194.524994, 195.425003, 192.074997, 192.024994, 189.024994, 192.975006, 191.25, 194.699997, 195.149994, 196.149994, 199.824997, 208.125, 208.800003, 208.949997, 204.324997, 204.5, 200.649994, 202.375, 192.800003, 189.324997, 189.524994, 188.675003, 188.399994, 187.524994, 184.824997, 183.300003, 184.25, 181.600006, 184.199997, 188.350006, 188.725006, 189.649994, 190.75, 189.625, 191.100006, 189.699997, 191.050003, 192.75, 200.324997, 203.925003, 197.524994, 192.050003, 184.75, 177.899994, 173.850006, 172.449997, 170.100006, 168.850006, 167.199997, 174.850006, 176.600006, 170.449997, 174.149994, 171.149994, 173.550003, 174.25, 179.800003, 179.300003, 167.350006, 168.25, 165.350006, 165.0, 167.199997, 164.949997, 157.949997, 160.100006, 161.699997, 163.050003, 160.600006, 161.149994, 162.899994, 162.300003, 161.149994, 161.699997, 161.5, 164.300003, 166.550003, 167.199997, 167.949997, 169.100006, 168.399994, 167.199997, 165.949997, 164.75, 163.699997, 165.149994, 160.5, 153.5, 155.899994, 163.449997, 170.449997, 172.449997, 170.800003, 174.100006, 172.350006, 173.800003, 173.949997, 174.899994, 174.800003, 175.0, 173.100006, 174.149994, 175.100006, 171.550003, 167.75, 173.899994, 168.600006, 165.399994, 170.25, 170.0, 167.350006, 166.800003, 155.649994, 154.350006, 156.0, 153.699997, 155.300003, 155.25, 155.0, 156.699997, 157.300003, 155.25, 154.550003, 156.850006, 159.100006, 157.5, 164.449997, 167.199997, 165.800003, 161.050003, 162.5, 164.949997, 163.100006, 158.800003, 165.399994, 166.449997, 164.350006, 168.0, 168.699997, 168.550003, 168.149994, 166.649994, 165.899994, 165.399994, 166.350006, 160.649994, 160.699997, 158.5, 159.550003, 158.649994, 157.699997, 153.399994, 154.25, 156.100006, 155.550003, 156.5, 154.25, 155.550003, 155.0, 152.449997, 152.800003, 153.600006, 153.399994, 151.050003, 150.699997, 149.100006, 155.25, 156.550003, 152.600006, 153.550003]
pred_ioc = [211.15843543, 213.68921903, 211.34680911, 204.20208896, 206.9764897 , 224.0484766 , 219.0573943 , 197.65979709, 212.42657523, 220.27427107, 216.76613682, 212.52016616, 210.51560123, 219.81181369, 218.21370558, 215.68471232, 221.30196702, 194.30508421, 207.19204133, 198.3187798 , 205.86683991, 201.94102408, 191.56590755, 183.13084691, 190.62153516, 198.86908047, 192.69434461, 193.75684502, 193.70908711, 192.54377566, 192.52298544, 191.7274283 , 190.43950196, 193.08396475, 192.41200046, 198.90740877, 182.49253985, 188.02476907, 185.84752125, 187.1771891 , 187.66343859, 187.47325981, 190.00754783, 190.53062892, 188.04385884, 186.28004269, 184.6763417 , 184.6074006 , 185.73638897, 186.17110292, 184.83648151, 189.04473739, 168.91870403, 216.21710526, 185.13903808, 200.65953151, 207.3593629 , 206.99397871, 218.10577661, 219.11471789, 215.84158093, 216.14870479, 185.60155664, 223.86699841, 216.40183149, 215.45807301, 219.57105252, 217.51896909, 215.16202951, 197.3013948 , 194.4997221 , 214.61439727, 217.69307112, 215.59413907, 215.28283411, 215.2974952 , 216.84671578, 218.50550785, 174.0416181 , 195.39118376, 217.70212325, 212.72314401, 217.82692221, 210.52900353, 214.40113148, 194.38050036, 195.12366409, 182.95483919, 196.41662587, 177.6764041 , 200.32419863, 201.69382111, 203.9158036 , 198.86881534, 208.23702852, 215.9568175 , 208.69082422, 218.76069155, 215.00093381, 215.09102249, 213.93031703, 216.37003983, 208.79536596, 204.56856839, 204.61479462, 205.74139971, 211.69032992, 213.83233246, 182.49593752, 216.19720607, 219.05235391, 213.96738006, 214.01474498, 208.62775501, 205.01491197, 187.61610313, 195.25230008, 194.64987367, 196.09813759, 193.83996426, 188.19321082, 196.42558912, 195.62978252, 195.97475505, 194.89337244, 197.02478843, 193.90537967, 197.9619148 , 195.06570247, 194.87170083, 194.34434218, 196.12865515, 197.30161691, 193.20842962, 193.29892315, 193.76926736, 194.69727519, 195.86677993, 200.89369168, 198.36744268, 190.89659322, 201.06279759, 203.61166706, 209.27742631, 198.69361584, 203.17921605, 202.87838241, 204.08481373, 200.74265184, 206.48734744, 198.78239378, 192.41060833, 193.42000159, 194.87952274, 192.21177816, 193.79814308, 193.18170957, 192.80592085, 194.21755544, 195.79310569, 195.50181437, 194.59056013, 197.03394245, 197.36980856, 192.70696842, 189.8070625 , 190.45716401, 191.56254986, 190.82418177, 192.66477565, 193.87271098, 195.79712942, 199.23665771, 198.46037948, 217.10755197, 211.8419286 , 205.6710195 , 202.71156384, 199.99891344, 201.41910347, 194.40055697, 189.65920351, 189.83830062, 189.36081073, 188.69186949, 188.27108371, 184.9404721 , 184.74203294, 184.82315462, 183.4070763 , 184.09338117, 189.06646557, 189.93925012, 189.11628321, 190.64453557, 190.58161469, 193.62482152, 191.29246167, 191.00064427, 192.4286312 , 200.21361274, 203.92524839, 191.84372308, 192.03614964, 186.76957947, 179.29657373, 172.20394078, 170.77246557, 168.25668173, 166.94577925, 162.46243752, 174.94394971, 176.41621496, 160.2571955 , 172.69560816, 169.04080623, 172.08743148, 174.56355056, 180.22516856, 179.0496309 , 145.93610307, 160.96664622, 161.01404951, 164.15454261, 164.24587551, 161.88339611, 157.93188908, 159.06860602, 162.8002138 , 162.95700053, 162.0208139 , 159.56537057, 162.5903119 , 162.02657811, 161.8004251 , 161.36414477, 161.42633054, 162.84057194, 164.43692299, 164.21494108, 164.0820472 , 170.55768803, 168.91873292, 163.11954759, 162.31418909, 163.54571825, 163.6537455 , 164.09848239, 153.42659915, 152.96695897, 154.66133513, 163.20887988, 177.35346165, 169.9818019 , 167.25481836, 174.96689303, 172.30230016, 171.78229341, 172.26516982, 173.6651135 , 174.26735722, 173.61990756, 174.87794631, 174.18640127, 173.89256655, 171.0440523 , 164.10976296, 177.73811657, 164.77810993, 162.88410727, 168.77630564, 168.60192825, 161.89305951, 167.34012089, 151.31039568, 159.74073855, 155.82364511, 154.51452711, 154.64743162, 154.63935134, 156.48314139, 158.29734315, 156.42175593, 154.13854965, 154.13204874, 158.84766168, 155.23290578, 156.94235305, 163.43461795, 164.91454816, 161.55449885, 161.9534681 , 164.13315902, 163.84312964, 161.85700953, 159.52312359, 163.54725811, 165.88462609, 164.02089111, 164.56395596, 166.6749599 , 164.5810552 , 162.78950647, 163.48667288, 163.15041357, 162.15725011, 164.32801076, 155.4042737 , 160.51464148, 159.25650634, 158.09452672, 158.06925411, 157.45006005, 153.58396355, 154.041794  , 155.06796029, 154.50129616, 154.56332235, 153.97054093, 154.95673158, 153.84555126, 153.61896721, 151.39202333, 153.14509012, 153.26534694, 151.277422  , 150.71978674, 146.99122772, 155.73926649, 157.71236446, 153.62818443, 154.23237617]
accuracy(actual_ioc,pred_ioc)

As per predictions : 
sell at 213.68921903
buy at 204.20208896
sell at 224.0484766
buy at 197.65979709
sell at 220.27427107
buy at 210.51560123
sell at 219.81181369
buy at 215.68471232
sell at 221.30196702
buy at 194.30508421
sell at 207.19204133
buy at 198.3187798
sell at 205.86683991
buy at 183.13084691
sell at 198.86908047
buy at 192.69434461
sell at 193.75684502
buy at 190.43950196
sell at 193.08396475
buy at 192.41200046
sell at 198.90740877
buy at 182.49253985
sell at 188.02476907
buy at 185.84752125
sell at 187.66343859
buy at 187.47325981
sell at 190.53062892
buy at 184.6074006
sell at 186.17110292
buy at 184.83648151
sell at 189.04473739
buy at 168.91870403
sell at 216.21710526
buy at 185.13903808
sell at 207.3593629
buy at 206.99397871
sell at 219.11471789
buy at 215.84158093
sell at 216.14870479
buy at 185.60155664
sell at 223.86699841
buy at 215.45807301
sell at 219.57105252
buy at 194.4997221
sell at 217.69307112
buy at 215.28283411
sell at 218.50550785
buy at 174.0416181


In [11]:
#SVR ongc crude oil

actual_ongc_co = [174.649994, 173.699997, 174.149994, 175.75, 175.75, 178.649994, 177.0, 174.0, 174.0, 175.75, 171.449997, 172.050003, 170.550003, 169.199997, 168.5, 169.100006, 169.5, 167.149994, 167.149994, 166.050003, 169.100006, 164.800003, 160.449997, 158.100006, 159.949997, 158.100006, 157.850006, 157.300003, 159.100006, 159.899994, 162.800003, 161.699997, 159.600006, 162.399994, 160.25, 162.850006, 158.899994, 158.350006, 160.050003, 161.699997, 163.0, 165.850006, 164.350006, 163.25, 163.300003, 164.0, 162.899994, 164.75, 169.399994, 167.399994, 165.399994, 165.699997, 166.399994, 165.899994, 164.0, 165.699997, 163.699997, 159.649994, 161.699997, 160.949997, 160.600006, 160.899994, 157.149994, 159.449997, 159.699997, 159.449997, 160.600006, 157.25, 158.399994, 156.949997, 159.949997, 161.800003, 162.800003, 162.300003, 162.300003, 161.850006, 163.050003, 162.300003, 161.050003, 159.350006, 166.75, 165.199997, 165.899994, 167.699997, 165.899994, 164.449997, 164.800003, 171.399994, 170.199997, 171.550003, 170.949997, 170.649994, 172.100006, 171.199997, 173.850006, 170.699997, 170.350006, 169.149994, 169.699997, 169.649994, 172.050003, 171.199997, 174.449997, 172.050003, 171.699997, 176.0, 176.850006, 177.350006, 184.0, 186.600006, 191.100006, 191.0, 188.0, 191.850006, 199.25, 193.800003, 192.550003, 189.600006, 191.100006, 184.199997, 182.0, 177.550003, 179.149994, 177.600006, 180.0, 180.899994, 181.300003, 181.699997, 181.0, 184.050003, 181.25, 182.449997, 180.649994, 179.649994, 181.600006, 178.399994, 175.850006, 177.050003, 180.25, 178.899994, 183.300003, 183.899994, 184.350006, 183.0, 183.050003, 185.199997, 188.0, 188.0, 193.399994, 193.699997, 192.600006, 193.0, 195.199997, 192.350006, 196.850006, 193.449997, 199.5, 198.449997, 197.399994, 197.0, 197.899994, 197.899994, 200.75, 196.600006, 197.199997, 195.699997, 194.100006, 193.699997, 200.699997, 207.350006, 210.850006, 208.5, 205.149994, 204.800003, 203.449997, 194.899994, 192.699997, 188.75, 185.449997, 190.0, 188.699997, 187.350006, 191.0, 185.75, 188.300003, 186.899994, 185.300003, 187.0, 190.149994, 186.100006, 189.800003, 190.350006, 188.5, 188.300003, 189.699997, 185.550003, 185.399994, 181.5, 180.149994, 179.899994, 183.850006, 183.600006, 180.600006, 180.300003, 177.149994, 176.850006, 174.100006, 175.550003, 178.899994, 177.399994, 178.899994, 179.600006, 177.800003, 179.949997, 177.5, 175.399994, 177.449997, 176.949997, 178.399994, 180.199997, 182.149994, 181.75, 182.5, 181.5, 180.800003, 182.100006, 183.949997, 182.350006, 181.050003, 182.649994, 179.449997, 180.800003, 180.699997, 180.550003, 180.050003, 181.100006, 180.25, 183.899994, 183.100006, 182.850006, 188.25, 187.5, 187.25, 189.0, 186.199997, 188.600006, 185.050003, 185.75, 185.199997, 175.399994, 167.699997, 175.449997, 175.600006, 175.550003, 174.0, 177.899994, 173.199997, 171.100006, 171.199997, 170.300003, 172.649994, 173.699997, 173.300003, 171.399994, 170.0, 168.5, 165.449997, 164.600006, 164.800003, 162.75, 159.699997, 159.550003, 156.649994, 158.300003, 156.25, 153.5, 158.399994, 155.5, 157.699997, 156.0, 156.399994, 155.449997, 157.399994, 157.949997, 158.100006, 158.949997, 154.100006, 153.800003, 156.149994, 160.300003, 161.75, 157.949997, 156.550003, 157.199997, 159.050003, 162.550003, 161.300003, 164.100006, 165.699997, 167.199997, 166.399994, 166.800003, 169.25, 167.550003, 172.5, 168.899994, 169.25, 166.050003, 166.25, 164.449997, 163.050003, 168.699997, 170.199997, 171.899994, 174.75, 176.050003, 174.600006, 177.0, 177.449997, 180.050003, 177.300003, 174.350006, 173.699997, 174.5, 171.899994, 168.649994, 167.800003, 168.5, 172.550003, 171.699997, 173.899994, 177.0]
pred_ongc_co = [174.27470124, 174.20752473, 174.09389557, 176.0909655 , 173.44207938, 176.16789197, 175.56203838, 173.0738651 , 173.58235362, 176.91182174, 171.68289484, 173.07914227, 169.56013776, 168.3734088 , 168.92257033, 168.35341759, 168.78466344, 167.93282444, 166.7507027 , 165.28042843, 168.19309604, 165.49889842, 161.36748722, 158.26881074, 157.18270305, 158.36686796, 158.42507154, 157.22036442, 157.67882422, 160.65946884, 162.05894545, 160.89630736, 159.78770951, 162.17979751, 160.93674254, 164.32535208, 159.53658585, 158.31464148, 159.54934379, 160.55531689, 161.15849223, 166.06709155, 164.07530929, 162.17158768, 162.16383684, 163.65283182, 162.31315747, 162.96083316, 168.26285827, 167.20035725, 164.47006371, 164.52085634, 165.0004394 , 165.8179843 , 163.14704978, 164.22366449, 163.17353861, 159.23142304, 161.4321597 , 159.84988409, 160.65258609, 160.6469612 , 159.46021653, 159.29428733, 159.63949175, 159.24645505, 160.33513066, 157.47866619, 158.17360371, 156.86712321, 159.11794202, 160.63807746, 162.0136951 , 162.8151361 , 161.7301821 , 160.45516597, 162.28545405, 161.96443589, 161.39238235, 159.23442483, 164.66519179, 165.53892984, 165.48730704, 167.24691949, 164.44727941, 163.28506749, 162.45671801, 170.17977745, 170.65710517, 169.71764613, 171.80142595, 170.47879103, 171.31370722, 170.83932525, 173.28483245, 170.68004644, 169.90778669, 168.84865759, 168.37224624, 168.91568098, 171.47584558, 170.48407856, 173.00342138, 172.0994925 , 171.10894641, 174.8869539 , 176.06865726, 178.27124154, 182.10014276, 188.5792319 , 188.68918461, 191.19347796, 188.92619761, 189.77233689, 197.13465385, 197.24316773, 192.51125847, 189.0368419 , 191.23199938, 186.01948369, 182.64557465, 178.01392307, 178.66626539, 179.97961009, 180.36604434, 180.77908411, 183.36872982, 183.63096608, 183.12680355, 185.11667425, 182.73991646, 184.37528166, 181.79021889, 182.57295766, 184.29312001, 180.33512633, 177.60570629, 178.1720111 , 181.9096626 , 180.37795149, 184.25688417, 186.12213084, 184.09411425, 184.23162818, 179.61065822, 186.21336881, 189.13568661, 188.94426805, 192.55400738, 195.38756938, 193.46553808, 193.96407275, 194.58860574, 193.71679233, 196.94665658, 193.37517404, 197.00063061, 196.89601793, 197.30862456, 193.74352538, 194.14940387, 193.18958262, 194.93691249, 192.79219059, 191.33148749, 191.18380858, 190.0383576 , 189.52656074, 197.91269531, 199.51503718, 205.27339835, 205.67868236, 201.76037272, 200.84706398, 196.31863967, 195.62506642, 187.97083772, 184.81184445, 178.89063733, 187.50045162, 188.42951494, 188.69222148, 193.16990952, 187.7618776 , 186.53935762, 186.57935008, 179.96596644, 182.73448413, 184.35338252, 181.850482  , 183.71320272, 185.83033292, 184.84928934, 183.29515375, 186.18145829, 181.61928789, 181.02765614, 179.50681729, 179.97547055, 178.28392358, 179.87817941, 182.91625564, 180.57940226, 177.75830019, 174.22838316, 173.96594666, 171.82703607, 173.14533041, 175.49361695, 175.5601128 , 175.06686042, 176.06362567, 173.83501529, 175.31257008, 173.60947142, 172.28684292, 172.63536669, 174.4570346 , 173.5927539 , 176.64512017, 181.84404903, 181.68211592, 182.05631721, 180.16427768, 179.75397567, 182.26671878, 184.36099222, 183.04602576, 181.44024986, 182.49148319, 180.55139665, 179.74567669, 181.51244129, 180.53804326, 180.48042896, 181.31530801, 182.73632969, 183.2933333 , 182.55898543, 183.82094663, 187.49073584, 187.30271507, 186.41690128, 187.90867346, 180.45754586, 187.66091416, 187.94194028, 186.82143217, 186.03440439, 178.34333541, 163.32759855, 170.93174829, 176.71095419, 175.47837918, 175.79808569, 177.99056644, 173.54529982, 172.56459394, 171.14058732, 170.86508966, 172.53719484, 173.8907727 , 173.77646355, 173.68078221, 170.72453492, 169.32923172, 167.39104755, 166.29778062, 166.24552908, 164.47510051, 161.94344158, 159.93537203, 158.6946855 , 158.70066593, 157.15084643, 156.22188312, 158.75895367, 156.20040665, 160.74099025, 157.2202847 , 157.12940932, 157.34316889, 158.37189802, 159.75272376, 157.97485844, 157.8361609 , 156.12743266, 155.38445268, 156.26778487, 163.11739609, 163.71687732, 159.46313259, 157.39354741, 157.90447508, 160.31893436, 162.69546362, 161.71983847, 165.29398648, 165.0533687 , 167.58844091, 168.05172928, 168.3783988 , 168.93767944, 167.95416565, 171.00927635, 170.36916745, 169.68656787, 166.95267486, 166.06954398, 165.41589114, 163.93304769, 168.05808618, 169.03423899, 170.81313948, 174.95466643, 175.95623218, 175.90801628, 176.86931255, 180.05855166, 179.65852686, 177.90132715, 175.25033354, 172.28020077, 173.12651316, 172.11707646, 169.29791216, 169.22728929, 170.13531939, 171.53893781, 172.69744754, 173.92275183, 177.29966501]
accuracy(actual_ongc_co,pred_ongc_co)

As per predictions : 
sell at 174.27470124
buy at 174.09389557
sell at 176.0909655
buy at 173.44207938
sell at 176.16789197
buy at 173.0738651
sell at 176.91182174
buy at 171.68289484
sell at 173.07914227
buy at 168.3734088
sell at 168.92257033
buy at 168.35341759
sell at 168.78466344
buy at 165.28042843
sell at 168.19309604
buy at 157.18270305
sell at 158.42507154
buy at 157.22036442
sell at 162.05894545
buy at 159.78770951
sell at 162.17979751
buy at 160.93674254
sell at 164.32535208
buy at 158.31464148
sell at 166.06709155
buy at 162.16383684
sell at 163.65283182
buy at 162.31315747
sell at 168.26285827
buy at 164.47006371
sell at 165.8179843
buy at 163.14704978
sell at 164.22366449
buy at 159.23142304
sell at 161.4321597
buy at 159.84988409
sell at 160.65258609
buy at 159.29428733
sell at 159.63949175
buy at 159.24645505
sell at 160.33513066
buy at 157.47866619
sell at 158.17360371
buy at 156.86712321
sell at 162.8151361
buy at 160.45516597
sell at 162.28545405
buy at 159.23442483


In [13]:
#ioc crude
act = [212.125, 215.274994, 219.475006, 213.024994, 209.625, 209.774994, 214.725006, 206.925003, 207.399994, 211.024994, 212.300003, 213.675003, 209.375, 210.024994, 211.824997, 209.875, 210.350006, 203.524994, 202.300003, 203.600006, 205.850006, 204.125, 196.75, 191.699997, 195.199997, 193.574997, 193.425003, 192.600006, 193.449997, 190.925003, 192.850006, 190.800003, 191.125, 191.725006, 193.75, 196.300003, 190.475006, 186.550003, 186.774994, 186.074997, 187.574997, 187.274994, 190.449997, 189.074997, 187.399994, 186.824997, 184.449997, 183.899994, 183.824997, 185.175003, 185.050003, 193.524994, 209.475006, 215.600006, 206.625, 207.024994, 205.149994, 205.899994, 209.725006, 211.149994, 213.149994, 213.675003, 206.600006, 211.600006, 213.425003, 214.100006, 220.350006, 217.25, 226.425003, 227.350006, 224.149994, 214.524994, 215.925003, 214.125, 216.100006, 214.574997, 214.524994, 217.449997, 207.975006, 209.850006, 207.824997, 207.824997, 209.149994, 206.975006, 206.625, 204.199997, 199.5, 199.824997, 197.824997, 198.699997, 200.25, 203.274994, 204.375, 203.375, 207.725006, 207.074997, 205.425003, 209.050003, 207.449997, 207.300003, 206.774994, 206.949997, 205.800003, 204.824997, 202.225006, 203.850006, 208.625, 218.25, 207.475006, 208.125, 207.75, 208.399994, 207.475006, 206.625, 205.324997, 200.100006, 198.5, 197.300003, 197.425003, 198.425003, 193.475006, 194.75, 196.5, 197.574997, 196.475006, 198.074997, 198.0, 197.375, 197.699997, 196.300003, 195.300003, 196.850006, 196.75, 193.925003, 195.274994, 195.25, 194.375, 195.75, 202.425003, 202.199997, 197.550003, 201.100006, 205.0, 206.574997, 204.800003, 204.949997, 205.050003, 205.475006, 205.125, 204.975006, 200.75, 196.0, 194.274994, 194.975006, 193.199997, 193.149994, 193.649994, 191.925003, 194.274994, 196.199997, 194.774994, 194.449997, 194.524994, 195.425003, 192.074997, 192.024994, 189.024994, 192.975006, 191.25, 194.699997, 195.149994, 196.149994, 199.824997, 208.125, 208.800003, 208.949997, 204.324997, 204.5, 200.649994, 202.375, 192.800003, 189.324997, 189.524994, 188.675003, 188.399994, 187.524994, 184.824997, 183.300003, 184.25, 181.600006, 184.199997, 188.350006, 188.725006, 189.649994, 190.75, 189.625, 191.100006, 189.699997, 191.050003, 192.75, 200.324997, 203.925003, 197.524994, 192.050003, 184.75, 177.899994, 173.850006, 172.449997, 170.100006, 168.850006, 167.199997, 174.850006, 176.600006, 170.449997, 174.149994, 171.149994, 173.550003, 174.25, 179.800003, 179.300003, 167.350006, 168.25, 165.350006, 165.0, 167.199997, 164.949997, 157.949997, 160.100006, 161.699997, 163.050003, 160.600006, 161.149994, 162.899994, 162.300003, 161.149994, 161.699997, 161.5, 164.300003, 166.550003, 167.199997, 167.949997, 169.100006, 168.399994, 167.199997, 165.949997, 164.75, 163.699997, 165.149994, 160.5, 153.5, 155.899994, 163.449997, 170.449997, 172.449997, 170.800003, 174.100006, 172.350006, 173.800003, 173.949997, 174.899994, 174.800003, 175.0, 173.100006, 174.149994, 175.100006, 171.550003, 167.75, 173.899994, 168.600006, 165.399994, 170.25, 170.0, 167.350006, 166.800003, 155.649994, 154.350006, 156.0, 153.699997, 155.300003, 155.25, 155.0, 156.699997, 157.300003, 155.25, 154.550003, 156.850006, 159.100006, 157.5, 164.449997, 167.199997, 165.800003, 161.050003, 162.5, 164.949997, 163.100006, 158.800003, 165.399994, 166.449997, 164.350006, 168.0, 168.699997, 168.550003, 168.149994, 166.649994, 165.899994, 165.399994, 166.350006, 160.649994, 160.699997, 158.5, 159.550003, 158.649994, 157.699997, 153.399994, 154.25, 156.100006, 155.550003, 156.5, 154.25, 155.550003, 155.0, 152.449997, 152.800003, 153.600006, 153.399994, 151.050003, 150.699997, 149.100006, 155.25, 156.550003, 152.600006, 153.550003]
pred = [211.80063159, 214.78749022, 216.52617378, 211.73804545, 211.73879046, 212.24417893, 214.06296619, 208.82013761, 207.37431435, 209.72773959, 212.07957741, 211.63942464, 210.87514191, 209.29942283, 210.01911175, 209.94199003, 209.37321567, 205.12721705, 204.5478876 , 202.3259432 , 204.42144288, 203.3929548 , 198.90988369, 191.80950263, 191.08940987, 196.1756494 , 192.91150083, 192.89530675, 193.11882698, 193.19309898, 191.55974478, 191.29371083, 189.56057178, 192.04739232, 191.84743587, 197.13904547, 192.50448692, 188.14705839, 186.17280124, 187.10829884, 187.7678794 , 187.64100509, 189.26148499, 190.04445212, 188.37450678, 187.19960915, 185.68636461, 183.79187757, 185.55728787, 186.47299241, 184.9194597 , 192.86360005, 206.45971533, 212.10571502, 208.5149792 , 208.25246295, 206.03344363, 204.99347148, 208.63728161, 211.5731969 , 212.5211623 , 214.74871489, 210.80647313, 211.36762053, 214.30984595, 214.1281343 , 216.99771404, 218.06773773, 222.02550135, 226.24456509, 224.32778514, 216.80788157, 215.81257401, 214.92284814, 214.66869016, 215.10303989, 215.2217942 , 216.1347425 , 210.82086008, 211.08373218, 209.36783692, 208.53280822, 208.66839078, 207.46402   , 207.90946845, 202.72319999, 200.78883333, 196.87014192, 200.06904478, 193.79666156, 200.12431575, 202.31626999, 205.92452617, 202.34481983, 206.12829814, 208.26579111, 206.55942496, 209.36969279, 208.72876583, 208.53405706, 207.96027915, 208.66485865, 206.54450004, 205.4690879 , 204.4099129 , 204.83355683, 207.19545085, 215.11329613, 211.29072968, 208.09082092, 208.53024155, 208.28959213, 207.95006586, 206.21699276, 203.82905008, 199.84975328, 197.6676949 , 197.09147244, 195.92147379, 195.5504497 , 195.39345541, 195.48611625, 195.74475164, 196.97984095, 195.90616184, 196.56586802, 196.3863042 , 196.29992694, 195.49442177, 194.88473311, 193.96042193, 195.62239775, 195.87575623, 193.87260809, 192.82535208, 193.37349943, 194.35557127, 194.87971563, 199.12817809, 201.33582883, 198.41388494, 199.03688444, 202.2257496 , 205.304026  , 202.59696796, 203.56277709, 203.76795837, 203.93280873, 203.62620001, 202.46251725, 199.95130612, 196.87805396, 193.50845154, 193.44346535, 192.45268209, 191.20673617, 190.45496993, 190.48370604, 191.16183145, 191.00482878, 191.13808618, 189.3298725 , 190.85916879, 190.58032052, 188.73102444, 185.9753523 , 186.24365229, 187.18814933, 187.75483859, 190.01667835, 189.169402  , 189.23990326, 191.30713751, 198.39762828, 202.47265592, 199.26992023, 197.61734314, 196.89763538, 195.22185671, 198.96066114, 191.30685608, 188.4901766 , 188.51924559, 187.99888952, 186.53339228, 185.95486139, 182.5718255 , 180.92709073, 180.80969681, 178.35670206, 178.65807678, 182.70050898, 184.53696057, 184.15728528, 186.87142874, 186.0380294 , 188.58539089, 187.82612718, 187.85713622, 187.92315045, 193.8187677 , 200.06416771, 197.35148298, 187.93042575, 184.33554126, 177.56505743, 171.48172829, 169.52606642, 168.04479447, 163.3501898 , 163.10577753, 168.0265381 , 171.05006446, 171.10178653, 169.79278026, 169.55129301, 169.73509732, 172.71950228, 174.06897978, 174.28493735, 167.17643724, 160.53156741, 162.99685269, 160.64047808, 162.05502163, 161.75301381, 155.82701324, 155.08378282, 157.97581253, 158.06331866, 158.30981268, 156.256166  , 157.91987603, 158.81593656, 157.87209557, 157.03049027, 157.02709042, 157.58809214, 159.63808885, 159.25314063, 161.48944272, 164.89176821, 164.69552749, 162.07569605, 159.84391228, 160.2867486 , 158.72934249, 158.84184651, 156.77927276, 151.69862503, 149.91585268, 156.29756576, 167.53972932, 166.46065085, 165.15247178, 166.91152022, 168.25697712, 167.98851263, 167.98967143, 168.77290287, 169.23723755, 168.44360423, 169.06810265, 168.92142554, 168.73499755, 167.72905355, 164.83033836, 168.04138089, 166.09093928, 162.93994688, 163.16041751, 165.39442949, 162.86760254, 162.34081394, 153.9522124 , 150.36739929, 151.04251363, 149.91893213, 149.73015961, 150.32544183, 150.83032327, 152.84927247, 152.69778253, 151.11052035, 149.62684842, 155.62760097, 153.4681005 , 154.73654054, 158.48386936, 161.65073982, 159.69285771, 157.99220512, 159.15021311, 159.01189439, 157.34737019, 156.18535434, 158.25638512, 161.42178663, 160.02571268, 162.60196036, 164.37573004, 163.15280291, 163.14443017, 162.26871109, 162.07226687, 162.29781804, 161.20865195, 159.56023163, 156.45919222, 156.24768007, 155.52659117, 155.38852562, 154.87262369, 150.92048947, 149.75696879, 151.77417831, 152.06961534, 151.76872389, 150.38603064, 150.33335644, 151.0679169 , 149.02456462, 147.79454218, 149.09813699, 149.53537695, 147.81474439, 147.26256154, 143.49024261, 149.3500042 , 151.32985687, 149.87826294, 148.69684763]
accuracy(act,pred)

As per predictions : 
sell at 216.52617378
buy at 211.73804545
sell at 214.06296619
buy at 207.37431435
sell at 212.07957741
buy at 209.29942283
sell at 210.01911175
buy at 202.3259432
sell at 204.42144288
buy at 191.08940987
sell at 196.1756494
buy at 192.89530675
sell at 193.19309898
buy at 189.56057178
sell at 192.04739232
buy at 191.84743587
sell at 197.13904547
buy at 186.17280124
sell at 187.7678794
buy at 187.64100509
sell at 190.04445212
buy at 183.79187757
sell at 186.47299241
buy at 184.9194597
sell at 212.10571502
buy at 204.99347148
sell at 214.74871489
buy at 210.80647313
sell at 214.30984595
buy at 214.1281343
sell at 226.24456509
buy at 214.66869016
sell at 216.1347425
buy at 210.82086008
sell at 211.08373218
buy at 208.53280822
sell at 208.66839078
buy at 207.46402
sell at 207.90946845
buy at 196.87014192
sell at 200.06904478
buy at 193.79666156
sell at 205.92452617
buy at 202.34481983
sell at 208.26579111
buy at 206.55942496
sell at 209.36969279
buy at 207.96027915
sel

In [2]:
#tci crude
actual_tci = [255.550003, 249.399994, 252.75, 261.25, 253.399994, 263.399994, 296.600006, 318.649994, 324.049988, 329.450012, 316.5, 323.5, 315.299988, 314.350006, 303.049988, 308.049988, 311.799988, 308.049988, 309.600006, 308.649994, 310.0, 304.75, 302.950012, 294.600006, 290.899994, 291.100006, 320.75, 329.950012, 327.549988, 334.850006, 340.600006, 343.549988, 339.850006, 337.450012, 333.950012, 339.049988, 332.799988, 324.950012, 318.649994, 325.149994, 322.950012, 324.25, 320.5, 317.899994, 320.75, 318.299988, 314.899994, 311.700012, 313.700012, 303.950012, 304.25, 301.600006, 298.350006, 297.649994, 290.5, 280.350006, 273.25, 276.600006, 282.299988, 284.649994, 285.950012, 276.950012, 273.649994, 272.549988, 272.649994, 274.649994, 287.700012, 281.600006, 283.450012, 285.850006, 279.950012, 282.25, 281.549988, 286.549988, 284.450012, 282.850006, 290.600006, 302.399994, 290.850006, 297.25, 294.799988, 294.5, 293.850006, 287.850006, 283.899994, 272.75, 260.850006, 262.75, 263.100006, 266.799988, 269.450012, 271.399994, 266.100006, 266.5, 268.950012, 263.0, 270.700012, 273.0, 272.399994, 280.049988, 283.649994, 284.0, 280.299988, 280.5, 282.350006, 278.649994, 276.25, 275.950012, 278.350006, 278.600006, 279.25, 276.649994, 281.100006, 291.5, 289.950012, 286.649994, 280.850006, 282.649994, 277.899994, 275.799988, 274.649994, 267.299988, 271.549988, 286.799988, 291.799988, 287.299988, 290.049988, 293.5, 298.299988, 308.0, 308.100006, 307.549988, 303.75, 299.649994, 297.700012, 295.350006, 293.0, 293.350006, 296.700012, 294.0, 291.799988, 288.299988, 285.399994, 287.149994, 288.649994, 291.25, 297.399994, 300.649994, 295.299988, 299.600006, 294.350006, 292.649994, 289.899994, 291.600006, 287.200012, 290.399994, 289.899994, 294.549988, 297.850006, 293.549988, 291.899994, 289.899994, 288.450012, 287.25, 283.549988, 284.0, 279.399994, 284.700012, 278.200012, 280.0, 277.950012, 274.850006, 280.799988, 276.899994, 278.399994, 281.600006, 272.5, 263.450012, 263.700012, 266.399994, 274.149994, 282.649994, 293.700012, 293.350006, 291.5, 283.5, 278.700012, 280.299988, 286.549988, 286.299988, 282.5, 279.399994, 276.600006, 272.899994, 271.0, 275.75, 276.350006, 269.950012, 270.899994, 261.950012, 256.399994, 260.450012, 261.799988, 266.450012, 264.299988, 256.549988, 258.850006, 262.350006, 264.399994, 259.649994, 263.850006, 263.5, 266.25, 276.200012, 285.100006, 291.450012, 288.350006, 284.450012, 284.200012, 284.850006, 287.399994, 279.0, 279.100006, 278.149994, 275.399994, 272.5, 272.899994, 274.149994, 272.899994, 291.100006, 276.700012, 284.5, 289.049988, 292.600006, 280.5, 279.649994, 279.600006, 279.899994, 277.0, 276.399994, 275.850006, 275.200012, 273.549988, 273.049988, 272.600006, 294.899994, 288.100006, 278.200012, 273.299988, 274.950012, 275.850006, 280.600006, 287.100006, 285.549988, 285.549988, 284.950012, 286.200012, 279.700012, 277.899994, 280.149994, 282.899994, 287.350006, 291.700012, 284.350006, 285.75, 286.0, 287.799988, 285.200012, 286.700012, 292.049988, 287.600006, 289.75, 284.350006, 286.299988, 281.700012, 279.799988, 289.100006, 284.549988, 284.899994, 284.649994, 284.600006, 286.200012, 292.299988, 290.549988, 285.399994, 289.700012, 286.899994, 283.25, 281.149994, 277.600006, 270.200012, 267.549988, 271.549988, 279.450012, 273.75, 274.350006, 277.649994, 282.549988, 281.5, 280.149994, 299.700012, 316.450012, 310.899994, 324.0, 318.5, 321.850006, 308.200012, 336.25, 337.799988, 330.350006, 356.450012, 361.700012, 359.950012, 356.049988, 364.75, 359.450012, 358.049988, 365.75, 363.299988, 365.350006, 349.950012, 338.200012, 327.549988, 335.850006, 348.450012, 341.75, 339.450012, 334.850006, 334.950012, 334.75, 335.149994, 340.600006]
pred_tci_co = [250.46855522, 250.95452354, 249.27843788, 258.48157477, 249.81613167, 256.48741421, 285.02008332, 310.70991184, 325.89661038, 329.7794676 , 319.50515085, 325.89142349, 314.70706975, 316.02001765, 304.71873745, 308.94139551, 310.88502615, 306.64894294, 311.60668773, 308.28231804, 301.72050857, 305.61629572, 303.96479015, 291.27999237, 293.40000859, 290.53221993, 313.62993749, 327.79135682, 326.83825251, 337.80280517, 340.29597743, 344.76678777, 338.31738689, 340.19857153, 335.44173806, 335.11166745, 332.68919066, 325.86482883, 320.1297242 , 325.18019027, 325.37513695, 325.33410889, 321.60700369, 319.23293234, 323.10488031, 319.76968136, 318.22429724, 311.91155225, 315.0721243 , 307.39600924, 307.51583879, 306.13231443, 301.70615817, 300.10005435, 292.40592424, 281.64298392, 274.14702002, 272.73466558, 283.55071597, 287.99220808, 284.64618469, 279.2279211 , 273.08753924, 267.48693942, 269.81846848, 275.59287883, 283.00220477, 285.01997369, 284.21055297, 287.07202273, 281.96544779, 280.17319035, 281.87334846, 283.75139059, 286.2178614 , 282.38738651, 287.10313864, 305.0457562 , 294.08063519, 296.75091602, 296.21443995, 294.44527626, 297.63147771, 288.57817244, 285.71351387, 273.82933471, 260.29583757, 262.07226226, 267.24032343, 266.78806912, 267.4594456 , 269.02884952, 268.04760299, 268.77960073, 269.14483668, 262.0694143 , 267.71647602, 272.64454694, 273.19241042, 282.7043437 , 286.68072974, 286.36061298, 282.33171583, 281.36563675, 279.79340138, 280.79731473, 275.51753142, 276.35851736, 276.91022793, 277.4609243 , 275.84712943, 275.3616396 , 280.81359706, 290.39606798, 287.24105553, 286.00203718, 281.40010656, 281.46402517, 278.70477589, 275.23814729, 272.81096086, 268.2155674 , 269.15869917, 281.49732802, 289.96729298, 291.09333344, 283.78897152, 288.36521013, 292.58027558, 304.60880638, 305.71547845, 302.74107012, 300.04579709, 298.92926517, 294.44604474, 290.64006588, 289.31762232, 292.22288627, 295.38426454, 289.70892213, 289.71572564, 287.12381455, 283.39182987, 284.58462922, 283.45245383, 289.79661082, 292.5877245 , 296.16376641, 293.4767515 , 295.93735793, 292.61308691, 290.79958934, 287.09991211, 288.68952628, 285.36200744, 286.23874997, 286.68226756, 289.10861444, 296.68969528, 290.25182439, 288.96904016, 288.59454233, 283.86571931, 285.72259473, 281.11466559, 280.19812341, 281.64545041, 279.94093769, 277.88518024, 279.299448  , 275.92488077, 275.31873131, 275.81983048, 281.74129384, 274.616083  , 283.70569482, 267.65170679, 262.64196482, 260.65075043, 266.39456579, 272.36864963, 281.24396266, 288.9869643 , 290.99445123, 291.61770493, 283.15467444, 275.87770562, 274.55121017, 283.14012616, 284.43208427, 281.69531416, 279.37070757, 276.87498967, 272.54499897, 271.41628545, 267.87433395, 273.88325107, 269.47928413, 268.32196603, 264.80121505, 258.94275729, 258.8879812 , 261.21936913, 263.23875798, 261.71982055, 255.69648362, 260.25796541, 265.54805419, 264.66918449, 259.90346257, 263.47163069, 264.87332205, 268.54684238, 273.16292428, 279.70415933, 286.69202716, 287.06602181, 282.2840775 , 283.56942604, 283.62610221, 287.30587617, 281.33085123, 278.67572268, 277.31873106, 277.60962633, 272.36504231, 272.50649218, 273.54238301, 271.74074694, 282.14095647, 282.37658128, 277.27285762, 285.30340161, 286.86674624, 281.69545621, 276.08731308, 276.7821104 , 276.40043857, 275.40343911, 275.16962598, 273.02119193, 271.82222074, 273.85142359, 269.37206514, 267.9797853 , 282.32588941, 281.11968867, 278.6733818 , 268.06741218, 270.68680987, 271.18438231, 278.54176064, 283.79334175, 283.02214085, 281.32953206, 283.16033056, 284.47497934, 275.98032661, 278.94520271, 281.35652988, 282.33856683, 283.47261908, 289.00728202, 284.7801805 , 281.83083772, 284.02605208, 283.9981057 , 284.32548819, 282.94490541, 285.75039522, 287.0883155 , 285.13970201, 280.34614249, 281.13176238, 273.55031943, 270.8066505 , 277.92273451, 276.82432358, 275.48709044, 273.67278173, 276.86409258, 277.77039585, 281.13518475, 281.38711147, 278.76900884, 283.23392169, 279.75249147, 283.89852183, 277.60291148, 275.81683283, 268.90031169, 266.39824039, 267.69969439, 274.15215881, 269.16337189, 271.5703944 , 273.35868386, 279.66023201, 276.76498021, 277.17782515, 287.36807988, 302.26288222, 302.1348853 , 309.34598711, 312.25343724, 313.65147826, 303.18851494, 310.45308971, 321.46214636, 317.92029408, 334.41548431, 335.81863786, 339.92819531, 330.57503359, 330.21110664, 335.00808082, 332.32750402, 336.88572243, 333.69159982, 333.25897226, 330.73023035, 321.15385961, 308.72470609, 315.14823253, 321.90897346, 320.25240752, 318.13519849, 314.66257907, 315.63267049, 315.1706229 , 318.87256442, 317.96143652]
accuracy(actual_tci,pred_tci_co)

As per predictions : 
sell at 250.95452354
buy at 249.27843788
sell at 258.48157477
buy at 249.81613167
sell at 329.7794676
buy at 319.50515085
sell at 325.89142349
buy at 314.70706975
sell at 316.02001765
buy at 304.71873745
sell at 310.88502615
buy at 306.64894294
sell at 311.60668773
buy at 301.72050857
sell at 305.61629572
buy at 291.27999237
sell at 293.40000859
buy at 290.53221993
sell at 327.79135682
buy at 326.83825251
sell at 344.76678777
buy at 338.31738689
sell at 340.19857153
buy at 320.1297242
sell at 325.37513695
buy at 319.23293234
sell at 323.10488031
buy at 311.91155225
sell at 315.0721243
buy at 307.39600924
sell at 307.51583879
buy at 272.73466558
sell at 287.99220808
buy at 267.48693942
sell at 285.01997369
buy at 284.21055297
sell at 287.07202273
buy at 280.17319035
sell at 286.2178614
buy at 282.38738651
sell at 305.0457562
buy at 294.08063519
sell at 296.75091602
buy at 294.44527626
sell at 297.63147771
buy at 260.29583757
sell at 267.24032343
buy at 266.78806912